In [ ]:
!pip install kmodes

In [ ]:
import random
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pandas as pd
from itertools import cycle, islice
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
from kmodes.kprototypes import KPrototypes
import missingno as msno 
from sklearn.decomposition import PCA
import time
from sklearn import metrics
from itertools import permutations  
from sklearn.metrics import confusion_matrix
from sklearn.utils.linear_assignment_ import linear_assignment
import csv 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import os
import urllib.request
urllib.request.urlretrieve('https://raw.githubusercontent.com/jainsee24/Datasets/main/adult1.csv', 'adult.csv')
urllib.request.urlretrieve('https://raw.githubusercontent.com/jainsee24/Datasets/main/sat.csv', 'satellite.csv')
urllib.request.urlretrieve('https://raw.githubusercontent.com/jainsee24/Datasets/main/GermanData1.csv', 'german.csv')
urllib.request.urlretrieve('https://raw.githubusercontent.com/jainsee24/Datasets/main/heart.csv', 'heart.csv')
urllib.request.urlretrieve('https://raw.githubusercontent.com/jainsee24/Datasets/main/kddcup991.csv', 'kddcup.csv')
urllib.request.urlretrieve('https://raw.githubusercontent.com/jainsee24/Datasets/main/shuttle.csv', 'shuttle.csv')
urllib.request.urlretrieve('https://raw.githubusercontent.com/jainsee24/Datasets/main/phenotypic.csv', 'soybean.csv')

['.config',
 '.ipynb_checkpoints',
 'shuttle.csv',
 'adult.csv',
 'german.csv',
 'satellite.csv',
 'kddcup.csv',
 'soybean.csv',
 'heart.csv',
 'sample_data']

In [ ]:
def accuracy(true_row_labels, predicted_row_labels):

    cm = confusion_matrix(true_row_labels, predicted_row_labels)
    indexes = linear_assignment(_make_cost_m(cm))
    total = 0
    for row, column in indexes:
        value = cm[row][column]
        total += value

    return (total * 1. / np.sum(cm))

def _make_cost_m(cm):
    s = np.max(cm)
    return (- cm + s)

In [ ]:
def InclusionProposedApproach(x1,n):
    pca = PCA(n_components=1)
    x = pca.fit_transform(x1)
    mi=0
    for i in range(0,len(x)):
        if x[i][0]<mi:
            mi=x[i][0]
    for i in range(0,len(x)):
        x[i][0]-=mi
        x[i][0]+=1
    N=len(x)
    f=[0]*len(x)
    m=0
    m1=-1
    pi=[0]*N
    while m1!=m:
        m1=m
        xx=0
        for i in range(N):
            xx+=x[i][0]*(1-f[i])
        c=n-m
        for i in range(N):
            if f[i]==0:
                pi[i]=c*x[i][0]/xx
                if pi[i]>=1:
                    f[i]=1
            else:
                pi[i]=1
        m=sum(f)
    return pi

def InclusionShastri(x1,n):
    x=np.array(x1)
    x=np.transpose(x)
    for i in range(0,len(x)):
        x[i] = [float(j)/sum(x[i]) for j in x[i]]
    ma=[]
    for i in range(0,len(x)):
        ma.append(max(x[i]))
    x=np.transpose(x)
    dev=[0]*len(x)
    
    for i in range(0,len(x)):
        for j in range(0,len(x[i])):
            dev[i]+=(ma[j]-x[i][j])
    pi=[]
    su=0
    for i in dev:
        su+=1/i
    for i in dev:
        pi.append(n*(1/i)/su)
        
    return pi

def InclusionNigam(x1,n):
    x=np.array(x1)
    x=np.transpose(x)
    for i in range(0,len(x)):
        x[i] = [float(j)/sum(x[i]) for j in x[i]]
    x=np.transpose(x)
    dev=[0]*len(x)
    for i in range(0,len(x)):
        for j in range(0,len(x[i])):
            dev[i]+=x[i][j]
    pi=[]
    su=0
    for i in dev:
        su+=i
    for i in dev:
        pi.append(n*i/su)
    return pi

In [ ]:
def row_echelon(A):
    r, c = A.shape
    if r == 0 or c == 0:
        return A
    for i in range(len(A)):
        if A[i,0] != 0:
            break
    else:
        B = row_echelon(A[:,1:])
        return np.hstack([A[:,:1], B])

    if i > 0:
        ith_row = A[i].copy()
        A[i] = A[0]
        A[0] = ith_row
    A[0] = A[0] / A[0,0]
    return A
def fastflightcube(p,X):
    nc=len(X[0])
    nr=len(X)
    u=[0]*nc
    uset=[0]*nc
    la1=la2=10**200
    la=eps=10**-9
    v=free=-1
    X=np.array(X)
    X=row_echelon(X)
    for i in range(nr-1,-1,-1):
        lead=0
        for j in range(0,nc):
            if X[i][j]==0:
                lead+=1
            else:
                break
    if lead<nc:
        v=0.0
        for j in range(lead+1,nc):
            if uset[j]==0:
                uset[j]=1
                free*=-1
                u[j]=free
            v-=u[j]*X[i][j]
        u[lead]=v/X[i][lead]
        uset[lead]=1
    for i in range(0,nc):
        if uset[i]==0:
            free*=-1
            u[i]=free
        else:
            break
    for i in range(0,nc):
        if u[i]>0:
            la1=min(la1,(1-p[i])/u[i])
            la2=min(la2,p[i]/u[i])
        if u[i]<0:
            la1=min(la1,-p[i]/u[i])
            la2=min(la2,(p[i]-1)/u[i])
    if la2/(la1+la2)>random.uniform(0, 1):
        la=la1
    else:
        la=-la2
    for i in range(0,nc):
        p[i]+=la*u[i]
        if p[i]<eps:
            p[i]=0
        if p[i]>1-eps:
            p[i]=1
    return p
def flightphase(X,P):
    n=len(P)
    NN=len(X[0])
    index=[]
    p=P.copy()
    for i in range(0,n):
        index.append(i)
    e=10**(-12)
    d=0
    random.shuffle(index)
    for i in range(d,n):
        if p[index[i]]<e or p[index[i]]>1-e:
            index[d],index[i]=index[i],index[d]
            d+=1
    while d<n:
        hm=min(NN+1,n-d)
        if hm<=NN:
            d=n
            break
        if hm>1:
            p_s=[0]*hm
            i_s=[0]*hm
            B=[[0 for i in range(hm)] for j in range(hm-1)]
            for i in range(0,hm):
                i_s[i]=index[d+i]
                for j in range(0,hm-1):
                    B[j][i]=X[i_s[i]][j]/P[i_s[i]]
                p_s[i]=p[i_s[i]]
            p_s=fastflightcube(p_s,B)
            for i in range(0,hm):
                p[i_s[i]]=p_s[i]
            hl=d+hm
            for i in range(d,hl):
                if p[index[i]]<e or p[index[i]]>1-e:
                    index[d],index[i]=index[i],index[d]
                    d+=1
        else:
            if p[index[d]]>random.uniform(0, 1):
                p[index[d]]=1
            else:
                p[index[d]]=0
            d=n
    for i in range(0,n):
        if p[index[i]]>1-e:
            p[index[i]]=1
        if p[index[i]]<e:
            p[index[i]]=0
    return p

def cube(X,P):
    n=len(P)
    NN=len(X[0])
    index=[]
    p=P.copy()
    for i in range(0,n):
        index.append(i)
    e=10**(-12)
    d=0
    random.shuffle(index)
    for i in range(d,n):
        if p[index[i]]<e or p[index[i]]>1-e:
            index[d],index[i]=index[i],index[d]
            d+=1
    while d<n:
        hm=min(NN+1,n-d)
        if hm>1:
            p_s=[0]*hm
            i_s=[0]*hm
            B=[[0 for i in range(hm)] for j in range(hm-1)]
            for i in range(0,hm):
                i_s[i]=index[d+i]
                for j in range(0,hm-1):
                    B[j][i]=X[i_s[i]][j]/P[i_s[i]]
                p_s[i]=p[i_s[i]]
            p_s=fastflightcube(p_s,B)
            for i in range(0,hm):
                p[i_s[i]]=p_s[i]
            hl=d+hm
            for i in range(d,hl):
                if p[index[i]]<e or p[index[i]]>1-e:
                    index[d],index[i]=index[i],index[d]
                    d+=1
        else:
            if p[index[d]]>random.uniform(0, 1):
                p[index[d]]=1
            else:
                p[index[d]]=0
            d=n
    n1=round(sum(p))
    s=[0]*n1
    count=0
    for i in range(0,n):
        if p[index[i]]>1-e:
            s[count]=index[i]+1
            count+=1
    return s

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Feb 16 13:34:13 2021

@author: sj
"""

s='''563.225,56.748,0
564.887,58.119,0
565.434,68.061,0
565.926,79.953,0
566.762,69.405,0
567.99,81.669,0
569.194,62.483,0
569.791,55.458,0
570.576,73.136,0
572.167,73.306,0
573.101,47.248,0
573.649,88.769,0
575.847,55.477,0
576.019,55.179,0
576.686,66.819,0
577.379,44.799,0
577.513,51.055,0
578.727,80.574,0
579.476,39.921,0
580.633,44.737,0
581.278,65.616,0
582.023,54.002,0
583.593,76.708,0
583.796,79.86,0
584.366,53.483,0
586.459,59.779,0
587.023,85.774,0
587.679,71.899,0
588.232,39.995,0
588.672,51.99,0
589.312,38.724,0
591.027,44.019,0
591.4,91.172,0
592.217,89.381,0
593.16,66.805,0
594.063,66.893,0
594.76,74.765,0
595.356,80.827,0
595.79,75.361,0
596.355,83.586,0
596.668,86.122,0
598.529,58.733,0
599.626,59.223,0
600.848,85.828,0
601.795,80.356,0
602.177,43.649,0
603.088,68.333,0
604.009,86.323,0
605.044,65.636,0
605.162,87.564,0
605.689,81.98,0
606.666,65.305,0
609.273,46.702,0
609.805,79.983,0
610.426,66.076,0
611.214,67.693,0
612.12,50.623,0
612.611,80.34,0
615.622,67.058,0
616.511,69.774,0
185.148,187.84,1
186.379,210.891,1
190.146,203.939,1
192.725,200.574,1
193.58,191.945,1
196.524,209.577,1
198.153,194.02,1
199.639,189.935,1
200.871,193.293,1
202.992,198.882,1
206.631,195.446,1
208.041,192.715,1
209.812,189.853,1
211.981,200.95,1
214.217,207.195,1
216.775,196.353,1
218.926,212.944,1
220.754,190.068,1
221.351,191.195,1
222.246,185.421,1
223.856,211.078,1
224.959,201.679,1
226.243,191.191,1
227.497,199.323,1
232.63,208.815,1
233.817,198.346,1
236.097,201.321,1
241.224,206.679,1
242.439,191.389,1
244.763,189.957,1
245.674,206.491,1
247.434,208.13,1
249.004,200.664,1
249.977,198.39,1
250.814,208.034,1
253.545,215.389,1
256.788,192.156,1
257.962,208.085,1
260.626,210.016,1
264.383,201.5,1
265.424,194.869,1
267.976,208.111,1
272.218,187.568,1
275.199,196.881,1
281.512,193.436,1
282.56,198.533,1
283.104,202.56,1
284.464,215.55,1
286.95,198.506,1
288.923,213.113,1
289.933,208.837,1
290.501,200.067,1
292.782,209.527,1
295.307,185.377,1
296.686,208.971,1
300.462,210.472,1
301.987,213.201,1
303.553,213.944,1
13.354,264.529,2
16.103,255.247,2
17.339,249.372,2
18.137,258.655,2
19.608,265.248,2
21.247,262.794,2
22.098,281.747,2
22.758,242.831,2
23.345,251.326,2
23.635,245.377,2
24.691,273.256,2
26.117,244.136,2
27.203,257.622,2
28.94,244.752,2
29.936,234.543,2
30.809,233.232,2
31.169,284.702,2
32.489,266.777,2
32.93,271.838,2
33.591,283.933,2
33.773,281.922,2
34.513,276.181,2
34.652,252.158,2
35.524,243.571,2
35.876,243.583,2
37.343,233.625,2
38.62,246.751,2
40.455,255.385,2
41.524,284.287,2
42.817,248.056,2
44.067,281.238,2
45.476,271.387,2
46.187,286.244,2
47.739,252.702,2
48.338,258.763,2
48.994,234.513,2
49.857,253.511,2
50.245,237.932,2
50.976,267.189,2
51.328,270.145,2
51.66,262.632,2
53.837,292.781,2
54.617,291.696,2
54.983,230.195,2
55.211,290.125,2
56.785,252.76,2
58.204,276.242,2
59.035,272.023,2
59.624,282.485,2
60.396,231.942,2
61.014,242.437,2
61.448,245.848,2
62.185,263.182,2
62.725,245.593,2
63.76,274.755,2
64.487,237.96,2
64.713,252.737,2
65.05,227.462,2
65.823,290.489,2
66.296,267.64,2
67.832,281.385,2
68.888,226.002,2
69.967,246.5,2
70.286,260.492,2
71.834,228.468,2
72.206,243.232,2
72.789,287.178,2
73.157,293.775,2
73.85,264.688,2
74.708,264.807,2
75.394,254.897,2
76.311,243.142,2
76.972,287.451,2
77.989,230.004,2
78.233,266.766,2
78.502,237.794,2
79.202,287.122,2
79.272,242.156,2
79.722,264.361,2
80.199,298.082,2
81.069,261.888,2
81.534,276.974,2
81.913,237.585,2
83.034,247.402,2
83.679,240.902,2
83.855,297.607,2
84.888,253.396,2
85.499,236.795,2
86.941,232.489,2
87.406,299.783,2
88.392,237.048,2
88.701,250.651,2
88.982,299.489,2
89.319,226.7,2
90.41,257.448,2
90.58,231.072,2
91.23,262.604,2
92.209,222.15,2
92.786,227.418,2
93.554,293.737,2
93.768,293.591,2
94.697,258.708,2
95.536,237.686,2
96.182,228.088,2
96.715,223.257,2
97.527,273.821,2
97.685,278.438,2
98.38,284.748,2
99.108,296.813,2
99.767,224.9,2
100.165,222.073,2
100.778,278.911,2
101.931,266.622,2
102.84,240.178,2
103.131,239.699,2
103.347,251.559,2
103.919,268.385,2
104.593,279.342,2
104.851,266.407,2
105.187,268.4,2
105.396,267.244,2
105.877,248.509,2
106.187,280.048,2
106.267,221.253,2
106.699,250.833,2
107.251,233.71,2
107.427,264.254,2
107.743,239.853,2
108.98,262.284,2
109.713,272.873,2
109.967,298.248,2
110.534,224.346,2
110.823,229.572,2
111.309,226.288,2
111.731,278.228,2
113.107,268.624,2
113.726,289.755,2
113.971,233.849,2
114.502,226.788,2
114.867,299.812,2
115.386,274.375,2
116.25,285.816,2
116.901,239.932,2
117.588,279.468,2
117.808,296.526,2
117.991,296.388,2
118.793,226.169,2
119.412,292.521,2
119.916,279.182,2
120.435,281.88,2
121.329,271.627,2
122.254,274.67,2
122.962,272.851,2
123.476,263.763,2
123.817,290.177,2
124.895,225.312,2
125.876,237.528,2
126.299,248.847,2
127.881,277.424,2
128.782,258.494,2
129.474,259.443,2
129.986,263.765,2
130.248,254.418,2
130.953,288.629,2
132.874,236.928,2
133.637,287.142,2
134.173,262.139,2
135.02,242.398,2
135.631,288.789,2
136.99,279.42,2
137.468,248.605,2
138.208,250.879,2
139.394,251.962,2
140.295,288.066,2
140.989,255.222,2
141.315,270.173,2
141.852,235.406,2
142.609,245.018,2
143.009,238.399,2
143.719,276.511,2
144.26,231.112,2
145.175,256.033,2
145.439,275.743,2
145.89,252.356,2
146.859,269.907,2
147.058,258.492,2
147.155,235.835,2
147.203,236.173,2
148.38,258.14,2
149.106,257.056,2
149.431,255.84,2
150.654,287.67,2
151.7,255.743,2
152.675,261.621,2
153.978,271.106,2
155.048,286.331,2
156.319,265.01,2
157.05,285.03,2
158.213,278.882,2
158.962,243.196,2
161.138,237.596,2
161.646,243.994,2
163.331,280.859,2
164.213,281.869,2
164.773,282.94,2
165.059,242.019,2
165.281,280.402,2
166.042,248.031,2
167.623,242.272,2
168.128,276.198,2
168.464,268.973,2
169.483,267.895,2
170.249,249.485,2
170.979,273.265,2
171.513,253.62,2
171.783,269.142,2
172.844,257.272,2
173.952,265.78,2
174.138,270.815,2
175.856,272.269,2
177.18,255.497,2
177.257,264.482,2
178.95,243.888,2
181.542,281.885,2
182.722,251.107,2
183.453,267.579,2
184.185,246.987,2
185.341,270.505,2
186.511,242.978,2
188.91,241.019,2
191.666,243.181,2
192.627,275.928,2
194.441,269.436,2
195.761,262.548,2
196.945,247.552,2
197.076,274.643,2
198.511,272.877,2
200.067,270.562,2
200.776,269.116,2
202.842,259.868,2
203.896,244.526,2
204.787,250.219,2
205.469,248.346,2
208.056,258.037,2
208.684,259.333,2
210.158,265.929,2
210.741,245.633,2
213.053,239.042,2
213.685,254.623,2
214.879,253.977,2
216.027,264.996,2
217.35,266.199,2
218.58,255.904,2
218.852,243.589,2
219.705,242.826,2
221.495,236.117,2
223.711,242.644,2
224.354,259.331,2
225.566,248.478,2
229.354,237.398,2
233.129,236.923,2
233.311,258.28,2
234.903,256.115,2
236.158,240.551,2
237.976,253.329,2
239.34,251.177,2
240.348,244.397,2
243.132,254.789,2
243.505,248.39,2
244.731,247.555,2
245.48,255.893,2
247.504,258.892,2
249.927,234.034,2
251.727,260.918,2
252.992,248.801,2
254.104,251.732,2
255.41,243.183,2
258.885,236.008,2
260.153,258.965,2
261.686,253.904,2
264.961,245.82,2
267.765,254.765,2
271.931,253.244,2
272.186,246.484,2
273.159,248.205,2
274.679,256.092,2
276.12,238.06,2
277.313,240.625,2
278.117,262.051,2
280.711,258.076,2
281.716,246.601,2
281.94,236.785,2
282.711,249.692,2
283.115,263.584,2
283.983,257.348,2
284.543,263.316,2
287.046,262.162,2
288.927,260.541,2
289.315,255.218,2
289.943,250.361,2
291.608,267.339,2
291.955,267.281,2
292.398,252.325,2
293.017,255.158,2
295.004,260.365,2
296.183,237.238,2
297.623,252.161,2
298.424,264.142,2
299.807,263.045,2
300.557,269.025,2
304.204,271.556,2
307.425,274.677,2
308.161,273.136,2
308.928,247.987,2
310.057,251.42,2
313.136,266.045,2
313.672,247.666,2
314.848,254.205,2
317.095,273.844,2
318.449,266.77,2
318.858,277.693,2
320.149,245.128,2
320.88,254.928,2
321.587,243.537,2
323.345,257.64,2
324.518,243.11,2
325.259,254.743,2
326.596,254.017,2
327.833,249.55,2
328.233,263.65,2
329.356,264.379,2
331.098,264.241,2
333.338,276.376,2
334.236,263.988,2
335.585,269.761,2
336.776,245.851,2
337.25,243.518,2
338.656,262.053,2
339.143,237.759,2
340.372,273.623,2
341.517,282.249,2
342.655,280.406,2
343.522,269.317,2
344.314,252.462,2
344.967,243.995,2
347.038,255.476,2
347.652,285.927,2
348.419,273.871,2
348.735,244.499,2
349.448,246.822,2
349.757,286.903,2
349.886,271.844,2
351.663,274.561,2
352.21,235.047,2
352.711,252.672,2
354.288,235.336,2
354.766,253.537,2
356.356,250.344,2
356.632,270.577,2
357.71,243.756,2
358.554,270.689,2
358.852,276.35,2
360.086,259.86,2
360.788,271.746,2
362.224,276.787,2
362.946,254.075,2
363.594,272.175,2
363.749,296.325,2
364.26,266.056,2
365.446,264.773,2
365.78,289.731,2
367.157,276.706,2
367.698,278.978,2
368.132,265.631,2
368.667,235.202,2
369.802,277.56,2
370.829,225.197,2
372.405,228.113,2
373.835,270.518,2
374.024,269.535,2
374.702,226.626,2
375.341,276.45,2
376.196,241.588,2
376.595,237.067,2
377.026,266.93,2
377.092,248.07,2
377.393,220.006,2
377.831,248.255,2
378.371,274.317,2
378.5,240.02,2
379.145,225.451,2
379.454,269.72,2
379.639,234.918,2
381.229,286.043,2
381.421,267.62,2
381.813,230.739,2
382.338,262.01,2
382.845,234.506,2
383.098,241.973,2
383.557,292.124,2
384.073,230.751,2
384.686,262.532,2
385.559,291.054,2
386.265,300.405,2
386.731,275.618,2
387.253,226.958,2
387.448,226.627,2
387.822,289.82,2
388.107,260.38,2
388.736,233.743,2
388.97,291.242,2
389.399,243.022,2
390.715,226.094,2
392.665,264.428,2
393.032,288.458,2
394.425,298.366,2
394.91,252.065,2
395.928,301.856,2
396.136,276.809,2
396.765,263.377,2
397.189,235.132,2
397.463,229.396,2
397.862,259.51,2
398.33,243.374,2
398.63,235.251,2
399.487,228.996,2
400.102,245.483,2
400.213,246.726,2
400.6,276.768,2
401.105,273.561,2
401.597,254.28,2
402.147,286.638,2
402.891,297.832,2
404.28,297.829,2
404.364,291.647,2
404.952,289.539,2
405.164,293.44,2
405.487,295.939,2
405.846,282.933,2
406.319,231.514,2
406.818,266.06,2
408.352,273.228,2
409.058,243.858,2
409.322,255.152,2
409.623,289.933,2
409.888,269.07,2
410.876,287.778,2
411.207,262.768,2
411.46,247.458,2
412.459,236.835,2
413.229,275.968,2
413.701,227.171,2
414.651,238.275,2
414.779,235.505,2
415.267,261.513,2
416.077,257.898,2
416.318,282.917,2
416.889,291.509,2
417.761,268.451,2
418.445,259.976,2
419.217,268.525,2
420.531,291.445,2
420.635,232.499,2
420.819,276.785,2
421.354,250.434,2
421.532,231.376,2
421.91,293.441,2
422.133,292.868,2
423.156,234.235,2
424.009,252.468,2
424.881,241.292,2
425.677,279.174,2
426.503,250.416,2
426.682,246.638,2
427.451,233.268,2
429.061,266.166,2
429.557,277.432,2
430.013,234.895,2
430.835,247.6,2
432.029,267.743,2
432.396,237.085,2
433.005,254.057,2
433.448,245.556,2
434.83,289.099,2
435.734,248.008,2
436.958,238.21,2
437.521,286.103,2
437.817,256.183,2
438.218,231.246,2
438.6,236.97,2
439.448,240.77,2
440.103,251.244,2
440.686,266.626,2
441.449,238.77,2
441.867,233.186,2
442.566,237.036,2
443.651,239.602,2
444.093,253.062,2
445.336,273.26,2
446.434,246.71,2
446.862,244.73,2
447.864,238.768,2
449.021,240.921,2
449.613,253.74,2
451.683,265.286,2
452.213,242.38,2
453.724,258.44,2
454.921,263.766,2
455.359,280.24,2
456.722,272.881,2
457.717,265.404,2
458.617,240.381,2
459.11,263.377,2
459.967,275.32,2
460.796,266.038,2
461.696,277.385,2
463.472,260.844,2
464.852,273.706,2
466.705,261.183,2
20.026,88.356,3
20.925,67.44,3
22.487,51.786,3
22.816,79.416,3
23.583,45.891,3
24.731,71.788,3
25.363,83.041,3
25.619,35.879,3
25.866,104.418,3
26.267,98.659,3
27.143,86.854,3
27.569,63.439,3
28.136,52.202,3
28.708,70.1,3
30.002,86.534,3
30.529,50.283,3
31.439,59.06,3
31.778,31.64,3
32.362,60.976,3
33.102,42.576,3
33.816,63.395,3
34.528,42.59,3
35.083,82.876,3
35.587,35.094,3
35.876,84.028,3
36.332,44.874,3
38.016,63.307,3
38.977,101.356,3
39.977,35.651,3
40.739,101.634,3
41.284,99.463,3
41.571,70.988,3
42.029,96.977,3
42.703,42.245,3
43.616,99.011,3
44.274,97.434,3
44.549,96.843,3
45.227,64.844,3
46.879,105.358,3
47.229,41.339,3
47.812,73.61,3
48.172,37.776,3
48.445,101.597,3
48.882,96.462,3
49.213,37.181,3
50.837,68.983,3
52.087,39.576,3
52.683,97.493,3
52.851,70.693,3
53.586,44.759,3
53.813,74.661,3
54.275,39.558,3
54.604,68.79,3
55.701,41.258,3
56.304,103.478,3
56.559,48.611,3
57.978,73.95,3
58.987,83.099,3
59.405,72.465,3
60.345,84.563,3
60.88,56.087,3
61.323,91.239,3
61.968,78.091,3
62.304,41.51,3
63.227,76.938,3
64.075,76.453,3
65.2,45.738,3
65.767,74.248,3
66.027,59.45,3
66.414,49.308,3
67.068,54.264,3
67.789,33.516,3
68.005,90.836,3
68.44,84.09,3
69.243,73.506,3
69.941,67.454,3
70.515,76.681,3
70.619,68.005,3
71.443,62.345,3
72.225,73.661,3
72.823,55.906,3
73.081,57.281,3
74.597,88.148,3
74.891,92.013,3
75.352,42.168,3
76.402,46.526,3
76.924,97.468,3
77.354,42.696,3
77.609,49.547,3
77.998,51.452,3
78.273,67.935,3
78.672,81.344,3
79.841,37.471,3
80.276,99.507,3
80.807,66.626,3
81.264,37.297,3
82.573,43.318,3
82.845,62.738,3
84.769,33.369,3
84.95,46.428,3
85.31,69.209,3
85.528,43.019,3
86.174,87.7,3
87.064,45.155,3
87.264,41.933,3
87.621,79.009,3
88.04,42.651,3
88.358,83.503,3
88.661,50.492,3
89.681,62.352,3
89.819,35.2,3
90.149,80.837,3
90.417,86.464,3
91.178,93.144,3
92.234,58.324,3
93.43,69.424,3
93.843,43.443,3
94.526,86.049,3
95.324,53.04,3
95.511,60.079,3
96.416,65.625,3
97.569,67.92,3
98.308,100.305,3
99.007,36.435,3
99.209,63.846,3
99.495,57.019,3
100.154,79.893,3
101.63,77.737,3
102.822,77.927,3
103.06,40.985,3
103.86,81.621,3
104.769,81.393,3
106.071,100.828,3
106.567,76.381,3
107.638,40.477,3
108.238,49.247,3
108.426,78.291,3
109.009,50.542,3
109.879,93.396,3
111.307,36.873,3
111.949,77.803,3
112.188,68.448,3
112.91,82.451,3
113.441,72.41,3
113.821,42.075,3
114.246,94.967,3
114.784,36.421,3
115.213,73.147,3
115.343,41.516,3
115.8,47.128,3
116.291,63.513,3
116.894,38.559,3
117.562,65.505,3
117.75,43.494,3
118.667,84.298,3
119.687,80.956,3
120.488,97.901,3
121.28,94.189,3
122.338,60.263,3
123.958,88.871,3
124.635,97.045,3
124.823,56.081,3
126.011,49.435,3
126.736,61.51,3
126.961,78.826,3
127.27,76.188,3
128.959,67.193,3
129.687,75.267,3
131.082,49.239,3
131.513,92.707,3
131.874,48.172,3
132.207,70.671,3
132.473,78.561,3
132.883,82.874,3
133.179,90.397,3
133.755,93.225,3
134.411,80.027,3
136.888,63.247,3
137.557,82.487,3
138.322,52.055,3
139.097,88.59,3
139.247,52.484,3
139.864,81.763,3
140.329,90.265,3
140.81,52.179,3
141.162,61.581,3
141.492,92.94,3
142.308,63.537,3
143.042,57.887,3
144.075,50.124,3
145.616,70.385,3
146.775,77.422,3
147.703,84.201,3
148.922,42.972,3
149.038,75.295,3
149.587,42.701,3
149.917,59.291,3
152.073,43.234,3
152.474,95.42,3
152.883,71.206,3
153.719,87.626,3
154.131,96.51,3
154.392,46.112,3
154.686,39.223,3
155.18,69.093,3
156.204,81.21,3
156.851,91.208,3
157.62,88.902,3
158.011,88.12,3
158.285,88.965,3
158.606,42.57,3
161.177,77.037,3
161.465,71.657,3
162.31,89.163,3
163.53,88.975,3
163.767,65.833,3
165.344,62.451,3
167.697,82.038,3
168.233,60.035,3
168.664,54.343,3
169.22,83.732,3
170.695,75.392,3
170.993,67.087,3
171.253,69.994,3
171.6,72.319,3
172.086,54.77,3
174.307,63.59,3
174.961,51.181,3
175.306,57.132,3
175.675,85.154,3
176.225,60.676,3
177.422,48.042,3
178.532,38.528,3
179.289,38.267,3
180.322,61.561,3
182.166,40.855,3
182.69,50.331,3
183.39,85.896,3
183.61,68.512,3
184.591,76.374,3
185.472,74.38,3
185.61,77.854,3
185.938,60.224,3
186.136,61.699,3
186.442,64.75,3
186.704,91.046,3
187.69,62.835,3
187.95,91.721,3
188.717,59.453,3
189.725,55.706,3
190.862,47.475,3
191.617,93.426,3
192.774,57.081,3
193.39,69.729,3
194.738,88.856,3
194.948,85.251,3
196.382,61.135,3
197.382,91.451,3
198.151,70.459,3
198.896,92.014,3
199.346,45.374,3
200.184,90.579,3
200.288,71.313,3
201.004,94.303,3
202.27,44.882,3
203.374,66.095,3
205.069,75.118,3
205.719,50.086,3
206.605,39.759,3
207.196,44.224,3
208.581,88.991,3
209.378,74.42,3
210.508,66.553,3
211.211,87.259,3
212.047,62.406,3
213.648,56.196,3
214.136,52.492,3
215.333,62.461,3
215.711,51.128,3
216.664,89.533,3
216.957,58.002,3
217.56,89.709,3
218.142,62.487,3
218.712,62.188,3
219.774,43.489,3
220.216,86.898,3
220.599,79.91,3
222.071,52.333,3
223.48,72.01,3
224.339,72.518,3
225.532,41.204,3
226.199,90.138,3
226.773,53.108,3
227.091,49.349,3
229.049,72.15,3
229.812,56.575,3
230.128,74.606,3
232.164,81.685,3
233.352,61.882,3
234.184,72.322,3
234.561,68.49,3
235.12,63.464,3
236.365,58.842,3
237.963,43.493,3
239.496,52.424,3
240.379,79.313,3
241.285,56.515,3
242.707,86.626,3
243.848,73.486,3
244.328,50.502,3
245.503,58.238,3
247.604,88.236,3
249.017,75.836,3
249.916,41.877,3
250.626,76.032,3
251.214,75.53,3
252.941,78.521,3
256.725,44.221,3
257.692,79.27,3
258.448,43.265,3
259.4,60.697,3
260.526,64.885,3
260.949,61.861,3
262.137,65.716,3
263.49,68.789,3
264.564,83.02,3
265.939,85.714,3
267.149,71.208,3
267.577,64.794,3
268.566,64.402,3
268.832,62.947,3
270.193,61.842,3
270.907,75.431,3
272.251,76.767,3
274.057,57.467,3
274.713,56.921,3
275.826,85.578,3
276.736,55.24,3
276.994,50.804,3
278.203,44.936,3
278.654,80.64,3
279.981,85.693,3
280.538,46.481,3
281.017,46.503,3
281.581,52.053,3
281.775,56.956,3
282.448,83.449,3
284.784,85.639,3
285.346,84.195,3
285.816,83.059,3
286.705,87.379,3
287.922,68.7,3
288.775,55.605,3
290.721,72.844,3
291.944,48.623,3
293.111,66.996,3
295.278,57.284,3
297.164,81.626,3
297.622,77.822,3
300.619,54.293,3
301.24,67.744,3
302.009,45.413,3
302.212,71.154,3
303.245,71.639,3
304.637,46.342,3
305.991,68.517,3
307.776,63.824,3
308.773,64.917,3
309.122,66.194,3
310.161,77.431,3
311.262,80.928,3
312.144,76.836,3
312.881,78.538,3
314.322,61.281,3
315.441,46.009,3
316.111,67.115,3
318.039,67.135,3
320.769,74.333,3
322.16,78.744,3
323.978,50.357,3
326.427,72.615,3
327.47,59.251,3
328.444,61.955,3
329.155,45.002,3
330.036,80.932,3
331.706,48.948,3
334.272,46.132,3
335.207,50.155,3
336.559,52.831,3
337.243,55.359,3
338.274,74.516,3
339.262,48.903,3
340.858,74.988,3
341.722,79.242,3
342.302,67.469,3
343.49,81.079,3
344.244,49.789,3
346.019,48.244,3
347.603,67.166,3
348.298,63.024,3
349.743,76.385,3
350.902,48.533,3
351.801,78.05,3
352.975,72.493,3
354.22,70.427,3
355.118,71.092,3
356.428,78.265,3
357.412,70.52,3
357.851,57.674,3
361.262,61.678,3
362.116,74.956,3
363.833,69.576,3
364.162,50.608,3
365.168,70.519,3
366.817,79.558,3
367.494,63.749,3
368.624,70.051,3
369.742,53.672,3
370.865,76.36,3
371.122,77.172,3
371.939,70.299,3
372.808,54.15,3
373.894,74.103,3
375.607,75.367,3
375.795,64.978,3
378.162,76.049,3
379.772,73.963,3
381.339,78.152,3
381.711,64.804,3
384.332,69.501,3
386.181,48.463,3
387.533,49.403,3
388.474,48.509,3
389.93,73.889,3
390.384,59.937,3
391.786,72.13,3
392.926,63.488,3
393.553,67.943,3
394.626,78.583,3
395.643,65.311,3
396.715,61.735,3
399.499,55.847,3
400.204,61.959,3
400.882,48.857,3
401.445,53.554,3
401.961,72.637,3
402.865,50.064,3
404.213,58.824,3
405.133,59.514,3
406.462,75.161,3
407.734,61.151,3
409.517,76.146,3
410.738,55.98,3
412.46,48.352,3
414.745,59.642,3
416.578,58.682,3
417.472,72.913,3
419.479,61.671,3
421.539,65.509,3
422.855,57.896,3
423.308,72.312,3
424.876,63.659,3
427.301,52.866,3
428.306,56.496,3
429.491,63.914,3
431.457,51.741,3
431.817,52.7,3
433.348,54.709,3
434.718,55.796,3
435.7,59.384,3
436.188,52.454,3
438.046,50.671,3
439.489,51.076,3
440.297,51.044,3
442.533,59.002,3
444.359,57.847,3
445.847,63.321,3
447.009,53.556,3
448.248,59.269,3
449.252,55.079,3
451.058,56.084,3
452.231,60.751,3
455.288,73.972,3
480.872,334.831,4
485.216,241.477,4
486.035,325.079,4
486.356,232.863,4
486.759,278.795,4
487.7,327.89,4
488.121,266.233,4
488.824,188.719,4
489.661,208.594,4
490.141,317.891,4
491.006,262.795,4
491.867,273.072,4
492.522,349.435,4
493.272,348.466,4
493.839,175.772,4
494.508,372.029,4
494.83,174.778,4
495.311,262.529,4
495.699,232.135,4
495.73,175.982,4
496.058,311.919,4
496.213,346.151,4
497.08,251.411,4
497.592,277.648,4
498.615,321.004,4
499.348,327.046,4
500.261,264.723,4
500.896,228.076,4
502.061,308.524,4
502.412,273.52,4
503.806,193.192,4
505.413,338.048,4
506.234,203.63,4
506.661,272.982,4
507.103,198.025,4
507.867,231.304,4
508.158,194.442,4
508.522,181.015,4
509.631,331.831,4
510.06,299.833,4
510.995,164.336,4
511.274,185.284,4
512.052,330.97,4
512.474,312.885,4
512.707,363.166,4
513.433,272.337,4
513.942,309.618,4
514.896,338.554,4
515.489,181.802,4
515.637,172.259,4
515.971,168.484,4
516.882,243.334,4
517.255,357.003,4
517.496,170.072,4
517.887,375.45,4
518.631,258.059,4
518.912,262.173,4
519.05,275.865,4
521.297,365.091,4
521.577,324.102,4
521.922,265.562,4
522.173,361.623,4
522.66,266.664,4
523.306,331.784,4
523.525,289.952,4
524.095,255.166,4
525.058,322.106,4
525.683,310.979,4
526.41,204.544,4
526.837,185.48,4
527.096,366.064,4
527.808,306.467,4
528.442,254.458,4
529.27,198.847,4
529.626,263.193,4
530.131,166.028,4
530.802,293.405,4
531.122,365.629,4
531.457,227.77,4
531.702,296.293,4
532.02,283.149,4
532.132,199.358,4
532.647,326.492,4
532.878,234.675,4
533.192,160.673,4
533.877,220.304,4
534.15,274.941,4
534.923,201.415,4
535.435,167.846,4
535.984,204.646,4
536.212,234.512,4
536.616,227.524,4
536.745,174.493,4
537.329,309.338,4
537.521,244.539,4
538.817,379.651,4
539.194,278.559,4
539.774,172.602,4
539.844,314.595,4
540.037,277,4
540.395,329.338,4
540.609,163.443,4
540.875,203.393,4
541.6,359.202,4
541.962,207.813,4
543.076,233.018,4
543.266,165.854,4
544.265,350.96,4
544.443,172.071,4
545.03,301.581,4
545.303,301.635,4
545.665,172.952,4
545.98,364.976,4
546.914,359.652,4
548.245,348.933,4
548.338,342.74,4
548.628,227.947,4
549.035,290.121,4
549.559,170.365,4
549.987,290.017,4
552.43,368.72,4
20.163,135.455,5
22.54,133.792,5
23.601,139.432,5
27.562,131.582,5
29.526,136.135,5
32.592,135.646,5
34.601,149.79,5
38.274,145.324,5
40.537,149.816,5
46.921,131.675,5
48.715,151.548,5
54.139,129.718,5
54.596,127.531,5
55.752,131.06,5
56.923,131.517,5
57.837,132.443,5
58.408,132.9,5
59.909,131.622,5
61.463,133.798,5
63.28,128.257,5
65.549,138.53,5
68.142,144.341,5
70.974,135.507,5
71.675,126.72,5
73.756,131.251,5
75.71,129.162,5
76.753,144.234,5
78.764,145.557,5
81.256,146.509,5
82.402,142.087,5
83.626,142.361,5
84.094,124.809,5
88.248,151.101,5
89.82,147.771,5
90.21,124.418,5
91.216,127.902,5
92.038,147.397,5
94.98,123.231,5
97.174,149.764,5
99.236,140.842,5
99.415,132.784,5
100.697,148.987,5
101.718,151.445,5
102.192,128.267,5
102.345,156.875,5
104.671,131.178,5
105.947,120.576,5
106.423,153.835,5
109.149,137.643,5
110.612,143.044,5
112.654,145.227,5
113.741,153.157,5
117.786,150.827,5
120.133,119.53,5
122.344,126.998,5
122.97,130.338,5
124.424,123.629,5
126.069,150.593,5
128.502,137.405,5
129.8,148.397,5
130.996,142.945,5
132.985,139.085,5
138.954,151.225,5
139.725,142.143,5
141.033,136.615,5
143.928,119.499,5
145.088,127.996,5
147.137,145.576,5
150.874,143.991,5
151.57,132.643,5
152.641,126.352,5
154.816,124.228,5
155.562,137.586,5
156.685,116.021,5
159.459,159.694,5
160.652,135.253,5
161.822,145.008,5
163.865,133.157,5
164.448,146.123,5
166.413,127.977,5
167.876,124.219,5
168.858,116.553,5
170.047,154.26,5
174.158,123.771,5
176.448,134.225,5
178.697,124.466,5
179.603,125.417,5
180.723,152.672,5
183.14,149.104,5
185.67,143.05,5
189.056,120.449,5
190.099,136.46,5
191.786,116.852,5
192.382,128.013,5
193.626,133.698,5
194.39,114.376,5
195.738,135.992,5
198.6,155.586,5
199.467,135.352,5
200.434,114.484,5
203.988,141.826,5
206.186,119.585,5
206.79,126.155,5
207.826,144.105,5
208.591,129.315,5
211.667,148.61,5
212.739,151.245,5
213.784,159.809,5
216.198,138.852,5
217.273,132.778,5
219.627,133.476,5
220.024,150.138,5
221.087,124.243,5
221.448,112.89,5
222.243,128.54,5
223.322,146.254,5
225.3,119.518,5
226.192,115.616,5
227.152,111.342,5
228.003,115.449,5
229.062,118.111,5
231.865,119.5,5
232.564,123.819,5
233.056,159.382,5
233.971,122.505,5
235.537,139.907,5
236.322,150.657,5
237.016,144.844,5
238.188,151.559,5
238.685,156.991,5
240.516,150.911,5
242.275,148.436,5
242.966,156.867,5
243.745,144.246,5
244.482,156.06,5
244.964,156.2,5
246.113,148.951,5
246.408,153.451,5
248.583,110.92,5
249.141,141.036,5
249.995,130.426,5
251.757,134.636,5
252.769,149.487,5
254.687,119.377,5
254.895,159.468,5
255.668,135.436,5
256.683,135.129,5
257.022,148.442,5
258.886,143.29,5
261.13,150.724,5
261.35,123.095,5
262.475,151.48,5
263.424,154.39,5
264.568,132.859,5
265.271,136.056,5
265.645,148.981,5
266.535,126.489,5
268.389,149.466,5
268.992,128.756,5
270.461,159.535,5
272.095,116.329,5
272.8,131.06,5
273.96,112.132,5
274.629,126.368,5
275.433,159.832,5
276.017,155.483,5
276.4,147.304,5
277.327,152.237,5
278.876,160.939,5
280.239,133.911,5
280.972,135.362,5
281.773,151.874,5
282.664,137.674,5
285.095,110.859,5
286.109,146.389,5
287.04,161.174,5
287.998,162.888,5
288.741,127.745,5
289.269,146.349,5
290.278,120.803,5
290.594,126.369,5
291.729,141.952,5
292.221,147.908,5
293.231,160.734,5
294.203,111.993,5
294.478,144.722,5
297.582,126.619,5
298.336,152.056,5
300.213,149.197,5
300.479,111.716,5
301.491,125.282,5
301.988,147.122,5
303.514,128.15,5
304.495,148.12,5
304.844,150.956,5
305.023,146.022,5
305.32,117.454,5
306.886,158.827,5
308.903,133.036,5
309.514,108.966,5
310.989,149.599,5
312.728,131.854,5
313.972,138.096,5
314.744,123.795,5
315.034,162.594,5
318.113,119.776,5
318.642,125.897,5
320.046,142.62,5
321.235,110.232,5
323.767,137.205,5
325.108,149.343,5
326.088,157.696,5
326.944,115.167,5
327.722,108.153,5
328.09,119.36,5
328.568,125.463,5
328.882,133.027,5
329.07,130.705,5
331.298,150.178,5
332.78,117.426,5
332.962,139.875,5
333.176,142.43,5
334.072,145.903,5
334.732,118.522,5
335.609,112.523,5
336.048,106.842,5
336.464,109.636,5
337.301,121.584,5
338.975,142.92,5
340.076,140.209,5
340.733,130.657,5
341.432,148.501,5
342.143,135.803,5
342.561,118.268,5
342.76,159.119,5
343.347,120.624,5
345.759,112.959,5
346.177,142.695,5
347.01,162.25,5
348.324,155.248,5
348.847,121.319,5
349.27,104.102,5
350.268,155.018,5
351.355,130.904,5
352.184,116.145,5
352.6,153.59,5
353.529,165.407,5
353.965,109.725,5
354.898,104.62,5
355.471,131.498,5
356.412,115.175,5
357.903,159.996,5
358.739,148.101,5
359.707,110.247,5
360.877,135.827,5
361.789,151.029,5
362.986,105.904,5
363.991,161.374,5
364.349,124.248,5
365.072,123.332,5
365.712,109.745,5
366.139,161.967,5
367.357,157.376,5
368.109,148.878,5
370.159,132.535,5
370.341,122.888,5
371.905,151.124,5
372.141,146.876,5
372.809,101.883,5
373.572,124.196,5
374.806,110.627,5
375.012,166.118,5
375.493,148.985,5
375.997,128.038,5
376.54,115.948,5
377.539,134.109,5
379.408,133.052,5
379.881,147.462,5
380.49,164.797,5
380.935,116.036,5
381.989,104.546,5
382.408,128.653,5
383.107,109.26,5
383.366,158.673,5
383.876,166.926,5
384.802,112.571,5
385.689,126.766,5
386.499,126.074,5
386.875,149.056,5
388.105,141.878,5
389.451,102.493,5
391.529,106.554,5
392.474,158.959,5
393.363,160.461,5
393.913,162.008,5
394.391,144.514,5
394.913,107.693,5
395.405,137.845,5
395.852,135.215,5
396.776,119.499,5
397.641,139.632,5
399.164,105.794,5
399.918,165.557,5
401.254,134.953,5
402.096,162.859,5
402.61,108.187,5
403.609,147.19,5
405.196,157.791,5
405.949,154.721,5
406.65,161.996,5
408.269,136.817,5
409.474,109.904,5
411.081,110.178,5
412.274,164.404,5
413.416,142.217,5
413.929,153.953,5
414.675,155.706,5
415.76,158.422,5
415.947,148.758,5
417.508,159.855,5
418.315,141.355,5
419.537,148.749,5
420.437,134.559,5
420.996,156.976,5
421.305,104.089,5
421.528,107.615,5
423.015,155.494,5
423.708,124.388,5
424.648,154.398,5
425.93,147.334,5
426.891,113.192,5
427.803,109.824,5
429.384,98.402,5
429.747,111.753,5
430.327,117.361,5
431.114,166.059,5
431.724,114.938,5
432.267,156.525,5
432.536,117.196,5
433.09,104.252,5
434.12,115.038,5
435.246,158.305,5
435.818,162.484,5
436.959,147.122,5
438.152,137.398,5
438.75,98.508,5
439.579,169.047,5
439.691,141.413,5
440.249,117.888,5
442.137,130.39,5
442.7,167.665,5
443.017,137.065,5
443.897,163.984,5
444.519,96.789,5
445.233,127.048,5
445.52,112.532,5
446.048,104.647,5
446.788,124.089,5
447.624,107.356,5
448.034,107.531,5
448.515,169.29,5
449.34,132.426,5
449.89,94.756,5
450.602,154.409,5
451.759,127.084,5
452.486,136.176,5
452.895,98.269,5
454.448,155.473,5
454.853,146.684,5
19.63,349.476,6
22.691,367.157,6
23.878,357.674,6
25.846,359.834,6
26.461,342.387,6
27.403,350.942,6
29.272,353.44,6
31.412,383.076,6
32.358,371.62,6
33.132,371.771,6
33.892,337.602,6
35.6,383.954,6
37.084,333.971,6
37.494,335.178,6
38.315,359.756,6
38.604,396.243,6
39.737,361.884,6
40.679,388.344,6
41.653,319.618,6
42.345,394.862,6
43.797,340.708,6
44.178,332.407,6
44.706,365.156,6
46.428,355.147,6
47.515,356.629,6
48.115,342.562,6
49.459,350.855,6
50.221,334.949,6
51.055,379.387,6
52.212,327.486,6
53.565,363.259,6
54.699,345.209,6
56.059,385.797,6
57.262,381.326,6
59.588,357.893,6
61.471,379.243,6
61.998,387.088,6
63.099,339.609,6
64.309,352.468,6
64.903,367.009,6
65.812,393.432,6
66.669,364.468,6
67.612,375.903,6
68.008,383.416,6
68.789,372.091,6
71.037,360.484,6
73.94,345.452,6
74.735,339.028,6
75.428,379.489,6
76.856,362.964,6
78.884,351.998,6
79.65,373.082,6
81.327,329.31,6
82.628,386.988,6
83.23,367.24,6
83.711,341.459,6
84.957,388.822,6
85.27,389.035,6
86.209,368.775,6
87.567,398.195,6
91.831,373.494,6
92.391,365.471,6
92.829,375.571,6
94.331,374.994,6
95.679,355.861,6
98.554,327.855,6
100.356,343.139,6
101.355,392.531,6
102.682,396.718,6
103.281,392.931,6
104.011,363.262,6
104.786,331.155,6
105.121,344.324,6
106.15,372.175,6
108.516,334.433,6
108.74,389.766,6
109.958,400.941,6
111.22,345.926,6
112.781,389.315,6
113.842,391.853,6
114.926,340.23,6
115.261,349.419,6
116.562,346.507,6
116.922,394.277,6
118.894,334.391,6
119.6,375.124,6
120.381,370.344,6
120.825,334.534,6
122.749,338.957,6
125.508,353.338,6
126.19,382.555,6
127.186,365.922,6
127.912,353.864,6
128.304,353.219,6
129.391,328.166,6
130.542,349.603,6
130.862,372.719,6
131.724,353.369,6
132.643,327.94,6
133.877,379.068,6
134.885,329.977,6
135.315,357.724,6
137.015,340.944,6
137.442,367.477,6
138.355,394.453,6
139.28,378.471,6
140.046,371.254,6
140.794,348.667,6
141.719,382.129,6
143.696,389.256,6
144.511,391.301,6
145.283,382.486,6
146.735,363.463,6
147.796,323.203,6
148.646,373.737,6
149.749,394.613,6
150.137,343.403,6
152.39,373.589,6
152.761,398.566,6
154.49,389.909,6
155.075,357.295,6
157.744,325.818,6
159.24,376.091,6
160.202,380.371,6
161.99,365.087,6
162.862,339.485,6
164.314,341.107,6
164.805,318.129,6
166.374,340.962,6
168.254,391.485,6
169.13,392.663,6
171.751,373.785,6
173.256,389.621,6
173.481,384.481,6
174.257,314.826,6
174.701,329.036,6
175.309,345.168,6
175.93,353.58,6
177.14,368.486,6
177.867,371.715,6
178.184,317.635,6
179.134,323.988,6
179.369,326.522,6
180.012,381.236,6
180.725,330.761,6
181.608,343.612,6
183.608,384.239,6
184.551,356.24,6
184.854,326.746,6
185.522,318.843,6
185.979,381.037,6
186.061,333.279,6
187.069,317.164,6
187.73,337.019,6
188.529,309.77,6
189.777,336.671,6
192.081,334.49,6
192.965,370.724,6
193.226,383.806,6
194.004,345.934,6
194.615,317.75,6
195.402,318.689,6
195.7,341.63,6
197.125,358.948,6
197.687,384.699,6
198.43,396.933,6
199.645,359.01,6
200.389,367.112,6
201.408,333.623,6
202.13,369.695,6
203.031,304.539,6
203.492,344.555,6
203.626,332.286,6
205.471,360.552,6
205.983,354.271,6
207.846,313.296,6
208.828,347.435,6
210.178,381.39,6
211.081,346.189,6
212.218,352.546,6
212.441,335.582,6
212.709,346.273,6
212.975,355.365,6
213.517,328.602,6
214.337,383.732,6
214.778,352.766,6
215.837,395.339,6
216.275,344.837,6
216.691,299.908,6
217.011,348.172,6
217.927,376.302,6
218.218,369.903,6
219.168,338.646,6
219.975,381.722,6
220.203,342.417,6
221.014,313.268,6
221.483,335.171,6
222.342,310.161,6
223.5,331.001,6
223.912,299.244,6
224.663,400.324,6
224.856,382.039,6
226.199,343.957,6
227.477,336.655,6
228.534,347.878,6
229.163,368.545,6
229.257,397.519,6
229.736,305.988,6
232.237,391.361,6
232.926,374.289,6
233.61,292.224,6
234.479,299.916,6
234.85,371.915,6
235.435,317.97,6
235.89,309.186,6
236.433,366.293,6
237.501,326.768,6
238.334,302.021,6
238.515,307.758,6
239.226,356.51,6
239.499,341.685,6
240.196,336.499,6
240.473,368.099,6
241.59,342.149,6
241.891,381.061,6
242.966,390.113,6
243.324,285.619,6
244.477,382.524,6
244.779,381.582,6
245.247,285.977,6
246.079,326.126,6
247.13,318.996,6
247.822,333.245,6
248.125,346.789,6
248.591,292.215,6
249.807,311.445,6
250.294,351.677,6
251.711,366.227,6
252.868,284.662,6
253.215,385.3,6
253.658,282.454,6
253.755,322.149,6
254.04,389.891,6
254.356,327.074,6
255.184,364.71,6
255.626,399.492,6
255.924,311.035,6
256.963,381.075,6
257.238,289.789,6
258.056,330.499,6
259.155,373.736,6
259.768,309.523,6
259.905,281.671,6
261.23,299.372,6
262.001,361.898,6
262.411,357.32,6
263.693,305.648,6
263.977,305.759,6
264.761,377.055,6
265.614,399.174,6
266.554,304.575,6
266.705,350.153,6
267.081,313.846,6
267.452,290.026,6
267.748,352.889,6
268.215,334.102,6
268.604,347.187,6
269.071,331.948,6
269.557,319.773,6
270.599,360.545,6
270.766,348.803,6
271.375,341.773,6
271.554,292.768,6
272.273,396.064,6
273.29,293.129,6
273.852,297.201,6
274.456,393.318,6
275.018,390.815,6
275.965,341.212,6
276.379,364.671,6
277.239,319.648,6
277.636,287.894,6
277.865,295.799,6
278.699,327.014,6
279.889,382.823,6
280.581,351.827,6
281.568,307.436,6
281.765,373.226,6
282.162,293.951,6
282.322,342.371,6
282.66,351.256,6
283.452,376.83,6
284.714,302.182,6
285.139,346.886,6
285.723,363.942,6
285.857,345.885,6
286.249,346.898,6
286.682,351.613,6
287.241,384.163,6
288.326,384.467,6
288.669,377.337,6
289.253,338.963,6
289.52,333.788,6
290.63,361.425,6
291.376,308.967,6
292.198,358.362,6
292.543,298.838,6
293.355,324.01,6
293.751,306.411,6
294.182,378.559,6
294.614,325.861,6
296.555,306.575,6
296.958,382.22,6
298.207,299.327,6
298.671,377.606,6
300.118,353.163,6
300.996,375.288,6
301.39,338.523,6
302.585,360.149,6
303.038,377.79,6
303.779,378.984,6
304.234,301.891,6
305.172,359.986,6
305.648,303.526,6
307.581,397.806,6
308.518,400.249,6
309.301,336.652,6
309.991,306.869,6
310.802,389.039,6
311.134,343.202,6
311.648,387.505,6
312.849,337.028,6
313.275,394.552,6
313.773,392.062,6
314.734,372.732,6
315.007,365.965,6
317.379,371.384,6
317.912,382.673,6
318.992,315.658,6
319.649,372.035,6
320.654,321.73,6
321.207,387.65,6
321.734,373.914,6
322.403,326.447,6
323.134,398.075,6
324.948,366.022,6
325.339,347.075,6
326.037,371.664,6
327.784,334.158,6
328.515,323.344,6
329.41,340.427,6
330.088,350.123,6
330.401,371.878,6
331.529,351.491,6
332.463,327.332,6
334.078,339.272,6
334.874,311.798,6
335.542,367.419,6
337.687,384.806,6
338.011,390.244,6
339.267,342.463,6
339.761,334.525,6
340.794,318.997,6
341.5,322.826,6
342.018,342.896,6
342.468,358.592,6
342.961,346.798,6
343.296,315.26,6
343.613,386.365,6
343.854,369.105,6
344.749,348.701,6
345.608,349.167,6
347.346,373.371,6
347.688,359.336,6
348.557,397.396,6
349.286,394.957,6
349.841,331.45,6
351.698,350.463,6
353.635,338.307,6
354.384,365.922,6
355.433,317.592,6
356.506,324.225,6
356.941,374.429,6
357.544,360.146,6
358.707,394.035,6
359.02,386.164,6
359.226,369.82,6
359.884,365.271,6
360.191,319.999,6
360.986,370.84,6
361.176,323.365,6
361.983,340.434,6
363.552,350.567,6
364.138,389.774,6
364.31,400.75,6
364.868,371.305,6
365.271,394.511,6
367.192,377.158,6
368.095,322.415,6
368.875,381.008,6
370.46,385.538,6
371.19,373.831,6
373.018,364.119,6
373.965,322.584,6
374.933,379.476,6
375.286,385.352,6
376.258,337.189,6
379.053,325.039,6
379.967,376.652,6
381.251,382.6,6
382.274,330.678,6
382.877,361.738,6
383.532,392.374,6
385.117,359.703,6
386.335,338.236,6
386.801,364.83,6
387.565,393.354,6
387.701,383.558,6
388.808,325.055,6
389.141,383.174,6
389.596,366.305,6
390.214,325.705,6
391.248,391.097,6
392.615,351.181,6
392.911,350.535,6
394.155,345.312,6
395.88,377.841,6
396.936,371.517,6
397.628,321.44,6
398.174,386.012,6
399.255,393.952,6
400.306,357.416,6
401.899,349.845,6
404.031,348.756,6
404.279,389.834,6
405.065,348.706,6
406.237,362.886,6
406.969,352.447,6
408.012,393.706,6
408.556,383.086,6
409.45,398.005,6
410.003,346.277,6
410.776,351.986,6
412.663,382.547,6
413.546,365.278,6
415.726,363.253,6
417.072,375.138,6
417.484,355.655,6
418.502,373.409,6
420.889,384.816,6
421.985,369.462,6
424.566,372.556,6
425.837,342.892,6
427.491,356.414,6
429.937,337.217,6
430.432,351.46,6
431.508,375.668,6
432.453,387.599,6
433.123,365.432,6
434.134,370.212,6
434.78,349.47,6
435.925,369.221,6
436.832,390.254,6
438.054,344.923,6
438.312,354.268,6
440.18,344.595,6
441.388,374.623,6
443.013,375.385,6
445.226,339.88,6
446.536,337.299,6
446.721,350.344,6
447.578,342.993,6
451.617,372.611,6
453.138,368.993,6
454.069,369.96,6
456.558,342.868,6
458.943,375.9,6
460.569,367.608,6
461.574,358.047,6
462.669,356.61,6
463.312,352.626,6
465.684,360.702,6
466.62,368.253,6
468.655,352.672,6
504.028,55.227,7
510.743,61.946,7
513.031,52.542,7
513.689,53.478,7
516.584,62.239,7
517.117,61.721,7
519.542,71.158,7
519.789,66.485,7
519.829,81.693,7
520.208,53.968,7
520.663,84.578,7
521.054,64.937,7
523.901,80.038,7
524.201,63.799,7
524.789,72.777,7
526.183,92.373,7
527.265,65.492,7
528.743,59.36,7
530.023,98.811,7
533.453,90.643,7
533.642,88.332,7
534.593,81.006,7
536.044,62.266,7
537.817,72.697,7
538.113,64.413,7
538.659,111.169,7
539.337,75.1,7
540.131,73.454,7
540.19,76.69,7
541.149,109.533,7
541.823,89.65,7
542.869,65.846,7
543.921,112.783,7
546.255,83.295,7
547.246,112.738,7
548.017,111.445,7
548.185,126.194,7
548.547,104.378,7
551.222,104.242,7
551.976,116.697,7
552.784,110.715,7
554.142,122.696,7
554.637,106.999,7
554.814,90.925,7
556.079,120.762,7
556.591,133.378,7
556.841,121.529,7
557.139,125.728,7
557.441,100.475,7
558.661,134.89,7
559.491,112.552,7
559.991,125.34,7
560.38,142.154,7
561.281,120.642,7
562.481,108.078,7
563.169,103.119,7
563.799,314.189,7
564.012,341.848,7
564.681,143.322,7
564.849,148.328,7
565.067,299.003,7
565.246,317.064,7
565.441,146.231,7
566.003,325.107,7
566.114,345.167,7
566.22,126.081,7
566.523,163.97,7
566.62,347.415,7
566.86,174.176,7
566.956,179.487,7
567.22,125.456,7
567.29,328.439,7
567.449,323.849,7
567.561,349.433,7
567.88,209.984,7
567.952,311.985,7
568.045,185.988,7
568.391,312.555,7
568.56,253.928,7
568.724,284.118,7
568.829,236.282,7
568.915,123.298,7
569.107,302.503,7
569.372,124.686,7
569.488,239.794,7
569.685,201.226,7
570.207,158.788,7
570.513,234.551,7
570.883,191.138,7
570.987,151.505,7
571.027,168.509,7
571.197,291.11,7
571.532,182.715,7
571.883,216.723,7
572.064,271.436,7
572.263,185.926,7
572.338,198.167,7
572.566,308.345,7
572.816,183.224,7
572.948,357.202,7
573.264,243.103,7
573.385,152.264,7
573.915,135.807,7
574.091,293.097,7
574.571,245.353,7
574.859,362.817,7
575.025,286.235,7
575.128,231.212,7
575.463,220.713,7
575.766,123.604,7
575.963,346.134,7
576.146,274.988,7
576.292,127.909,7
576.377,292.105,7
576.849,295.941,7
577.072,259.181,7
577.221,300.215,7
577.36,230.811,7
577.5,221.246,7
577.551,145.808,7
577.804,320.07,7
577.937,331.004,7
578.177,296.176,7
578.199,250.657,7
578.284,155.48,7
578.397,205.368,7
578.487,291.93,7
578.853,137.311,7
579.048,360.826,7
579.176,379.559,7
579.282,360.992,7
579.402,223.282,7
579.676,313.344,7
579.843,344.934,7
579.96,211.026,7
580.044,219.853,7
580.409,172.457,7
580.45,300.179,7
580.726,217.399,7
580.92,148.427,7
580.991,217.541,7
581.035,377.345,7
581.328,213.396,7
581.662,298.031,7
581.908,134.275,7
582.187,191.983,7
582.493,129.271,7
582.583,131.719,7
582.876,153.416,7
583.156,133.197,7
583.455,374.931,7
583.61,155.124,7
583.787,329.809,7
583.834,200.943,7
583.946,274.24,7
584.068,242.799,7
584.171,226.237,7
584.614,359.845,7
584.684,267.184,7
584.721,217.028,7
584.92,321.48,7
585.22,358.711,7
585.356,318.592,7
585.679,320.137,7
586.012,246.075,7
586.193,129.596,7
586.391,150.932,7
586.575,249.375,7
586.822,131.428,7
587.086,199.17,7
587.262,240.233,7
587.363,221.569,7
587.439,205.984,7
587.594,170.538,7
587.807,194.82,7
587.925,254.724,7
588.057,354.096,7
588.194,257.233,7
588.262,247.201,7
588.288,128.991,7
588.495,283.926,7
588.976,229.133,7
589.019,340.729,7
589.259,232.24,7
589.537,209.663,7
589.792,290.991,7
590.027,338.037,7
590.252,198.071,7
590.399,155.03,7
590.562,305.318,7
590.798,317.45,7
591.114,180.042,7
591.156,368.362,7
591.35,304.522,7
591.729,184.134,7
591.975,370.581,7
592.212,171.273,7
592.51,159.02,7
592.682,368.321,7
592.784,282.055,7
592.91,129.215,7
592.978,169.718,7
593.103,361.339,7
593.174,382.389,7
593.584,300.721,7
593.664,277.234,7
593.807,155.468,7
594.023,138.445,7
594.21,238.329,7
594.33,185.75,7
594.502,324.827,7
594.554,191.805,7
594.706,335.182,7
594.986,254.981,7
595.124,180.942,7
595.352,167.91,7
595.426,321.672,7
595.622,225.465,7
595.659,175.996,7
595.783,253.859,7
595.848,201.982,7
596.162,300.781,7
596.366,268.087,7
596.576,212.327,7
596.822,131.08,7
596.967,171.221,7
597.105,139.119,7
597.228,247.438,7
597.332,348.475,7
597.527,208.034,7
597.694,294.257,7
597.795,203.989,7
598.029,150.373,7
598.355,266.608,7
598.66,339.573,7
598.936,308.396,7
599.25,174.396,7
599.427,315.671,7
599.584,374.813,7
599.745,146.914,7
599.843,141.491,7
600.073,317.714,7
600.177,276.823,7
600.414,344.423,7
600.582,139.046,7
600.707,239.265,7
600.859,302.851,7
600.957,379.96,7
601.285,356.755,7
601.49,251.212,7
601.604,153.194,7
601.977,365.213,7
602.191,335.238,7
602.733,284.104,7
602.912,354.94,7
603.15,324.281,7
603.373,360.198,7
603.558,331.077,7
603.917,221.381,7
603.995,302.343,7
604.082,130.127,7
604.23,385.626,7
604.465,370.468,7
604.631,262.499,7
605.185,138.466,7
605.358,215.403,7
605.6,308.35,7
605.856,279.482,7
606.103,374.552,7
606.362,262.293,7
606.761,360.66,7
607.099,311.835,7
607.169,312.57,7
607.354,349.464,7
607.547,333.494,7
607.835,367.067,7
608.015,139.512,7
608.174,176.187,7
608.496,237.74,7
608.728,272.892,7
608.915,350.525,7
609.239,223.566,7
609.287,179.413,7
609.635,278.034,7
609.773,147.967,7
609.962,319.468,7
610.173,319.096,7
610.265,203.962,7
610.402,231.619,7
610.71,298.128,7
610.928,176.848,7
611.057,209.789,7
611.292,347.619,7
611.407,127.679,7
611.549,365.704,7
611.839,177.838,7
612.234,268.038,7
612.521,203.777,7
612.663,285.108,7
612.759,346.325,7
613.265,211.308,7
613.42,267.645,7
613.73,369.371,7
613.927,168.511,7
614.175,162.138,7
614.349,316.53,7
614.877,357.195,7
615.224,153.596,7
615.307,166.364,7
615.343,305.332,7
615.473,243.94,7
615.647,170.137,7
615.857,348.684,7
615.976,297.076,7
616.189,294.695,7
616.567,234.514,7
616.625,234.837,7
616.789,157.771,7
616.984,304.459,7
617.082,309.042,7
617.209,166.945,7
617.359,340.698,7
617.576,138.562,7
617.786,206.85,7
618.009,307.813,7
618.304,214.915,7
618.357,202.963,7
618.506,198.618,7
618.613,236.333,7
618.863,162.446,7
619.34,276.388,7
619.418,175.424,7
619.453,228.333,7
619.66,262.822,7
619.905,177.5,7
620.082,359.154,7
620.327,337.93,7
620.758,155.255,7
620.875,172.556,7
620.931,216.611,7
621.161,256.027,7
621.269,210.442,7
621.755,358.685,7
621.881,156.251,7
621.987,150.63,7
622.036,359.822,7
622.132,167.384,7
622.57,277.084,7
622.811,325.948,7
622.995,293.109,7
623.156,344.808,7
623.327,319.128,7
623.618,290.868,7
623.724,277.57,7
623.768,270.129,7
624.131,208.597,7
624.215,200.356,7
624.39,337.081,7
624.7,179.035,7
624.888,211.609,7
624.934,258.239,7
625.317,181.469,7
625.415,227.66,7
625.65,167.6,7
625.769,140.199,7
625.928,253.514,7
626.089,307.353,7
626.213,284.3,7
626.377,273.54,7
626.545,237.412,7
626.826,294.339,7
626.985,140.82,7
627.252,273.933,7
627.39,272.513,7
627.436,125.475,7
627.515,314.964,7
627.756,237.811,7
627.971,188.982,7
628.566,262.255,7
628.944,130.454,7
629.396,169.871,7
629.657,140.104,7
629.783,140.089,7
630.164,198.043,7
630.334,167.752,7
630.471,293.513,7
630.876,313.745,7
631.366,155.726,7
631.727,135.269,7
631.99,187.382,7
632.044,191.112,7
632.247,144.479,7
633.195,164.1,7
633.8,104.46,7
634.52,157.802,7
635.519,151.448,7
636.629,98.137,7
637.155,133.187,7
637.985,129.206,7
638.43,123.668,7
639.189,146.078,7
639.913,101.291,7
640.398,146.905,7
640.874,88.532,7
641.526,71.193,7
641.833,130.098,7
642.096,134.914,7
642.65,66.026,7
642.931,72.046,7
643.305,70.147,7
644.189,105.402,7
644.326,87.593,7
644.568,67.488,7
644.897,97.698,7
646.183,73.295,7
646.309,134.465,7
647.109,96.616,7
647.336,62.26,7
647.832,77.936,7
648.663,101.896,7
649.185,115.621,7
649.603,78.791,7
651.034,102.031,7
651.791,79.026,7
651.904,69.127,7
653.323,82.183,7
653.676,50.756,7
654.241,50.978,7
654.597,61.231,7
654.814,62.109,7
655.425,108.417,7
656.026,65.508,7
656.46,74.5,7
656.56,72.458,7
656.91,86.43,7
657.276,115.197,7
657.623,47.035,7
657.918,79.095,7
658.613,91.878,7
659.224,101.421,7
659.818,73.144,7
660.509,76.084,7
661.103,54.241,7
661.84,85.89,7
662.95,41.143,7
663.395,94.227,7
664.898,49.236,7
665.146,78.25,7
665.792,57.659,7
666.548,86.523,7
667.492,57.099,7
668.872,40.505,7
669.373,81.047,7
670.007,58.479,7
671.358,75.474,7
672.982,50.898,7
674.599,57.592,7'''
s='''664159.0,550946.0,14
665845.0,557965.0,14
597173.0,575538.0,14
618600.0,551446.0,14
635690.0,608046.0,14
588100.0,557588.0,14
582015.0,546191.0,14
604678.0,574577.0,14
572029.0,518313.0,14
604737.0,574591.0,14
577728.0,587566.0,14
602013.0,574722.0,14
627968.0,574625.0,14
607269.0,536961.0,14
603145.0,574795.0,14
671919.0,571761.0,14
612184.0,570393.0,14
600032.0,575310.0,14
627912.0,593892.0,14
601967.0,604428.0,14
591851.0,569051.0,14
601444.0,572693.0,14
629718.0,558104.0,14
661430.0,603567.0,14
597551.0,556737.0,14
601182.0,582584.0,14
562704.0,570596.0,14
605107.0,563429.0,14
607214.0,575069.0,14
568824.0,570203.0,14
612485.0,518009.0,14
589244.0,573777.0,14
625579.0,551084.0,14
560237.0,500154.0,14
626224.0,569687.0,14
610666.0,551701.0,14
597428.0,569940.0,14
600582.0,599535.0,14
604168.0,555003.0,14
613871.0,550423.0,14
617310.0,551945.0,14
625728.0,579460.0,14
606300.0,566708.0,14
638559.0,558807.0,14
582176.0,630383.0,14
544056.0,577786.0,14
631297.0,578351.0,14
561574.0,621747.0,14
604973.0,574773.0,14
605284.0,556134.0,14
617910.0,592293.0,14
555904.0,640401.0,14
526559.0,509417.0,14
603849.0,572396.0,14
598634.0,619900.0,14
603359.0,589262.0,14
657455.0,567754.0,14
614279.0,599251.0,14
578496.0,631434.0,14
645171.0,595209.0,14
688015.0,533535.0,14
656477.0,573007.0,14
540029.0,563888.0,14
597418.0,538362.0,14
613545.0,574733.0,14
628696.0,542447.0,14
604157.0,574191.0,14
603126.0,497987.0,14
605850.0,533310.0,14
596179.0,529556.0,14
610330.0,552628.0,14
598184.0,531473.0,14
620438.0,511694.0,14
600202.0,622079.0,14
603021.0,577991.0,14
515612.0,540643.0,14
606548.0,574376.0,14
593490.0,623110.0,14
607084.0,595500.0,14
583728.0,563146.0,14
616044.0,533807.0,14
624452.0,584622.0,14
604438.0,574666.0,14
625709.0,660145.0,14
601376.0,579831.0,14
565148.0,557305.0,14
639418.0,537903.0,14
612799.0,537497.0,14
583653.0,652182.0,14
599808.0,596484.0,14
605250.0,573272.0,14
606738.0,601356.0,14
611044.0,571814.0,14
575440.0,568511.0,14
616329.0,529516.0,14
605927.0,572533.0,14
609286.0,586782.0,14
647354.0,579190.0,14
606929.0,571420.0,14
562920.0,559223.0,14
680357.0,620811.0,14
707100.0,601463.0,14
604294.0,574368.0,14
614917.0,600946.0,14
611508.0,503220.0,14
603789.0,574474.0,14
551740.0,588768.0,14
546031.0,570429.0,14
569197.0,559307.0,14
688712.0,568754.0,14
604067.0,574091.0,14
606809.0,572101.0,14
588597.0,622592.0,14
613840.0,534167.0,14
576263.0,547239.0,14
597077.0,568196.0,14
601339.0,571609.0,14
606011.0,569426.0,14
593398.0,559764.0,14
593166.0,622725.0,14
609796.0,532982.0,14
712648.0,607541.0,14
585048.0,505097.0,14
525967.0,565687.0,14
660327.0,585901.0,14
607604.0,573951.0,14
604001.0,573587.0,14
591484.0,571087.0,14
613868.0,585089.0,14
645464.0,660478.0,14
538571.0,654649.0,14
555429.0,604551.0,14
597620.0,586226.0,14
586097.0,567445.0,14
545235.0,539822.0,14
649088.0,575733.0,14
599123.0,616977.0,14
563763.0,559454.0,14
617717.0,613939.0,14
618869.0,577243.0,14
567188.0,560044.0,14
636628.0,533309.0,14
560362.0,567914.0,14
595014.0,632844.0,14
605876.0,550640.0,14
604352.0,573425.0,14
600679.0,599545.0,14
596891.0,579476.0,14
614943.0,540976.0,14
604415.0,571400.0,14
603698.0,573585.0,14
658199.0,568743.0,14
598759.0,582942.0,14
671269.0,590258.0,14
593317.0,534085.0,14
657985.0,453405.0,13
608557.0,590278.0,14
623004.0,508050.0,14
601900.0,569391.0,14
641031.0,593074.0,14
589658.0,640498.0,14
567565.0,573952.0,14
559078.0,570088.0,14
639431.0,588155.0,14
566609.0,537592.0,14
568570.0,579335.0,14
660650.0,615038.0,14
667639.0,571164.0,14
596796.0,529933.0,14
606111.0,617846.0,14
616060.0,590779.0,14
602873.0,527864.0,14
571334.0,594145.0,14
602167.0,543919.0,14
605100.0,573307.0,14
546640.0,560043.0,14
585195.0,555011.0,14
651740.0,589474.0,14
589112.0,536573.0,14
582057.0,543124.0,14
690968.0,555952.0,14
609030.0,552436.0,14
632125.0,541413.0,14
603445.0,630598.0,14
628264.0,616214.0,14
577242.0,574073.0,14
604090.0,574392.0,14
607911.0,576556.0,14
600665.0,638890.0,14
600117.0,539079.0,14
631182.0,565466.0,14
695646.0,652231.0,14
619445.0,567491.0,14
613191.0,571942.0,14
567428.0,580005.0,14
590583.0,623314.0,14
606289.0,670243.0,14
585175.0,574133.0,14
634967.0,579020.0,14
614615.0,574437.0,14
593061.0,579188.0,14
606933.0,576409.0,14
606587.0,573873.0,14
594084.0,629464.0,14
683697.0,581516.0,14
601530.0,577183.0,14
603055.0,589818.0,14
616561.0,619007.0,14
607463.0,477085.0,13
611581.0,572630.0,14
579712.0,568354.0,14
602887.0,595245.0,14
600039.0,557139.0,14
605913.0,579526.0,14
579773.0,568698.0,14
606324.0,544357.0,14
605958.0,573728.0,14
606144.0,575076.0,14
667472.0,570571.0,14
580048.0,565009.0,14
597802.0,576113.0,14
610263.0,574422.0,14
603855.0,575180.0,14
624369.0,532153.0,14
599870.0,591110.0,14
577167.0,617095.0,14
658403.0,611332.0,14
563460.0,480559.0,13
610980.0,522819.0,14
569422.0,567790.0,14
627727.0,578737.0,14
592804.0,584539.0,14
540783.0,626497.0,14
591823.0,639096.0,14
637323.0,549567.0,14
638078.0,629421.0,14
611315.0,560757.0,14
647103.0,573853.0,14
522993.0,595581.0,14
601187.0,531825.0,14
533799.0,564377.0,14
649034.0,528813.0,14
620975.0,564344.0,14
622407.0,583192.0,14
597026.0,642918.0,14
551291.0,576721.0,14
608768.0,610525.0,14
697959.0,614701.0,14
658281.0,582264.0,14
546632.0,599204.0,14
672565.0,564468.0,14
636500.0,587401.0,14
696144.0,548705.0,14
533551.0,582247.0,14
526802.0,551524.0,14
553038.0,606418.0,14
696601.0,603624.0,14
588964.0,597435.0,14
716139.0,621721.0,14
608529.0,586311.0,14
557378.0,567033.0,14
589994.0,610890.0,14
589344.0,563119.0,14
624415.0,515911.0,14
565483.0,551512.0,14
584895.0,574666.0,14
582777.0,542670.0,14
639394.0,576728.0,14
608298.0,573748.0,14
663578.0,586265.0,14
628080.0,562986.0,14
597421.0,554767.0,14
595193.0,576508.0,14
658103.0,572250.0,14
659529.0,498474.0,14
606415.0,581386.0,14
641976.0,577433.0,14
587466.0,564953.0,14
589180.0,595099.0,14
601885.0,573083.0,14
580561.0,528916.0,14
621726.0,578003.0,14
558817.0,576255.0,14
620213.0,574468.0,14
594010.0,570095.0,14
541976.0,512639.0,14
593840.0,578685.0,14
602667.0,575857.0,14
631446.0,561156.0,14
596481.0,567449.0,14
611318.0,581584.0,14
568546.0,564911.0,14
593865.0,572987.0,14
597017.0,575588.0,14
578804.0,573582.0,14
673750.0,594081.0,14
592714.0,553958.0,14
594029.0,586701.0,14
668502.0,563937.0,14
629533.0,537542.0,14
801539.0,318482.0,3
801667.0,318542.0,3
814045.0,332998.0,3
800479.0,321640.0,3
757470.0,224976.0,3
844536.0,424646.0,6
797105.0,316002.0,3
779956.0,271501.0,3
797205.0,313318.0,3
801323.0,316240.0,3
783666.0,262902.0,3
806131.0,312752.0,3
830658.0,368694.0,3
802482.0,335269.0,3
801910.0,325434.0,3
808062.0,337340.0,3
789833.0,304422.0,3
803603.0,327255.0,3
800865.0,312968.0,3
802237.0,318526.0,3
803959.0,315173.0,3
800123.0,312312.0,3
780927.0,285057.0,3
797242.0,324588.0,3
801551.0,320544.0,3
809182.0,319115.0,3
801931.0,317202.0,3
870801.0,476330.0,6
820327.0,352701.0,3
801093.0,317509.0,3
798523.0,324500.0,3
805237.0,318655.0,3
803375.0,311620.0,3
801441.0,323812.0,3
797313.0,317527.0,3
788220.0,301915.0,3
790395.0,300024.0,3
817135.0,349925.0,3
797787.0,301470.0,3
804630.0,332333.0,3
808710.0,330379.0,3
802111.0,320817.0,3
796375.0,309563.0,3
819768.0,346596.0,3
814605.0,354382.0,3
800565.0,319364.0,3
802089.0,313456.0,3
814401.0,341234.0,3
796609.0,319691.0,3
776131.0,248465.0,3
803464.0,329093.0,3
802646.0,317894.0,3
809550.0,327365.0,3
795322.0,311178.0,3
830450.0,393096.0,3
778414.0,260729.0,3
800041.0,318331.0,3
802814.0,320130.0,3
802738.0,312234.0,3
823913.0,369439.0,3
803386.0,318982.0,3
812869.0,346251.0,3
798904.0,324420.0,3
800447.0,331643.0,3
807446.0,342555.0,3
804251.0,329956.0,3
797594.0,302109.0,3
803448.0,314797.0,3
765502.0,240340.0,3
795527.0,315510.0,3
794640.0,313497.0,3
808717.0,335740.0,3
790932.0,305223.0,3
816852.0,352965.0,3
826228.0,376658.0,3
795299.0,312036.0,3
808686.0,332209.0,3
809594.0,344872.0,3
799275.0,324108.0,3
878184.0,487392.0,6
798472.0,322218.0,3
824822.0,367091.0,3
746769.0,205186.0,3
756701.0,207223.0,3
795489.0,287279.0,3
800483.0,320713.0,3
783491.0,288029.0,3
797162.0,306135.0,3
800272.0,316170.0,3
806827.0,324616.0,3
802853.0,314938.0,3
800804.0,316615.0,3
813173.0,354777.0,3
801588.0,319814.0,3
797400.0,315778.0,3
823840.0,368410.0,3
811391.0,343836.0,3
801055.0,321833.0,3
804038.0,306622.0,3
803780.0,334441.0,3
834173.0,386159.0,3
802784.0,313853.0,3
807293.0,347456.0,3
817119.0,336333.0,3
786635.0,269352.0,3
824624.0,381716.0,3
797415.0,319840.0,3
800772.0,325023.0,3
806626.0,318966.0,3
800845.0,311838.0,3
769386.0,259892.0,3
797272.0,324228.0,3
807409.0,325526.0,3
795836.0,291870.0,3
798321.0,319665.0,3
805372.0,313397.0,3
790738.0,308028.0,3
800667.0,315995.0,3
795205.0,316433.0,3
809054.0,345541.0,3
808452.0,320117.0,3
779897.0,279572.0,3
782642.0,275308.0,3
797569.0,321799.0,3
801751.0,318110.0,3
787644.0,297715.0,3
807401.0,314610.0,3
800677.0,310656.0,3
800394.0,301473.0,3
795571.0,310093.0,3
814158.0,335578.0,3
811913.0,325091.0,3
806982.0,316680.0,3
775604.0,274485.0,3
800133.0,327081.0,3
806511.0,313841.0,3
803796.0,314893.0,3
802012.0,318315.0,3
826789.0,377676.0,3
814108.0,344096.0,3
804354.0,309671.0,3
807017.0,330432.0,3
803774.0,334756.0,3
817410.0,346944.0,3
841928.0,398948.0,3
816145.0,349032.0,3
818622.0,370753.0,3
786245.0,292352.0,3
784164.0,285702.0,3
805390.0,319676.0,3
790774.0,303642.0,3
803277.0,319525.0,3
833182.0,379714.0,3
773023.0,251826.0,3
801609.0,320399.0,3
778965.0,282290.0,3
814317.0,333112.0,3
805080.0,323708.0,3
801080.0,318049.0,3
799724.0,318884.0,3
798059.0,316113.0,3
806176.0,320301.0,3
769876.0,257669.0,3
825716.0,373098.0,3
791983.0,298565.0,3
801386.0,313545.0,3
804871.0,317801.0,3
810903.0,332925.0,3
838276.0,395673.0,3
810543.0,341180.0,3
777978.0,252419.0,3
820258.0,351165.0,3
801997.0,319385.0,3
779437.0,281959.0,3
810253.0,323873.0,3
794826.0,309589.0,3
814280.0,344223.0,3
832651.0,373469.0,3
794142.0,306704.0,3
802041.0,319504.0,3
804190.0,320706.0,3
798983.0,325682.0,3
840462.0,402858.0,3
799912.0,323363.0,3
804119.0,328544.0,3
804358.0,320802.0,3
803057.0,331037.0,3
803031.0,325811.0,3
784151.0,283760.0,3
800444.0,311497.0,3
802004.0,317772.0,3
803470.0,319069.0,3
811277.0,335577.0,3
803332.0,315488.0,3
800037.0,317963.0,3
813436.0,360446.0,3
800694.0,316789.0,3
838129.0,404012.0,3
764997.0,249477.0,3
800316.0,311699.0,3
806848.0,320541.0,3
805462.0,317833.0,3
798308.0,312788.0,3
802655.0,316263.0,3
801456.0,320080.0,3
801309.0,318213.0,3
840886.0,397888.0,3
829529.0,375007.0,3
830805.0,370801.0,3
804979.0,334141.0,3
804013.0,316083.0,3
808489.0,317021.0,3
770113.0,241615.0,3
804332.0,319006.0,3
833661.0,389733.0,3
797834.0,297329.0,3
794956.0,309980.0,3
801232.0,307078.0,3
809552.0,342258.0,3
794615.0,305632.0,3
809259.0,345634.0,3
811502.0,355939.0,3
801301.0,316893.0,3
803370.0,318247.0,3
804702.0,315881.0,3
797009.0,293194.0,3
813791.0,334609.0,3
812965.0,330224.0,3
826048.0,374236.0,3
800959.0,322309.0,3
803405.0,317316.0,3
802254.0,320673.0,3
783116.0,265635.0,3
798633.0,317300.0,3
801482.0,327644.0,3
798612.0,326404.0,3
799001.0,302014.0,3
802537.0,312497.0,3
799794.0,315080.0,3
793892.0,313943.0,3
804691.0,332987.0,3
793855.0,298525.0,3
796952.0,318633.0,3
800655.0,320227.0,3
801780.0,318992.0,3
805977.0,320805.0,3
796916.0,323332.0,3
820904.0,351763.0,3
808405.0,318841.0,3
765283.0,249459.0,3
824212.0,363776.0,3
798981.0,311092.0,3
797178.0,310439.0,3
800582.0,316980.0,3
803839.0,310353.0,3
796283.0,315582.0,3
801145.0,317601.0,3
801662.0,318423.0,3
801782.0,318281.0,3
803377.0,319212.0,3
803932.0,318781.0,3
806192.0,337849.0,3
789583.0,287700.0,3
801586.0,318947.0,3
804251.0,331304.0,3
799667.0,316830.0,3
759789.0,229116.0,3
799021.0,326404.0,3
793684.0,312065.0,3
806320.0,315242.0,3
786234.0,267976.0,3
821151.0,366545.0,3
801903.0,312794.0,3
782506.0,279930.0,3
803627.0,319323.0,3
794991.0,291216.0,3
794667.0,315069.0,3
799291.0,317500.0,3
798243.0,324160.0,3
804408.0,337683.0,3
818119.0,348676.0,3
810502.0,336169.0,3
793560.0,285392.0,3
801326.0,315779.0,3
808644.0,326320.0,3
813645.0,328765.0,3
800966.0,317165.0,3
797111.0,302781.0,3
802431.0,322048.0,3
811061.0,333595.0,3
786719.0,283364.0,3
773955.0,239849.0,3
803701.0,321815.0,3
783110.0,289805.0,3
800235.0,316652.0,3
807421.0,317527.0,3
797947.0,319734.0,3
805035.0,317469.0,3
801166.0,322298.0,3
846693.0,429031.0,6
801493.0,322249.0,3
806087.0,326636.0,3
810657.0,329468.0,3
801313.0,300606.0,3
800390.0,309948.0,3
801634.0,318965.0,3
799197.0,314215.0,3
795254.0,296488.0,3
807696.0,320548.0,3
800461.0,326821.0,3
801174.0,320442.0,3
809781.0,326904.0,3
826585.0,366934.0,3
804429.0,312164.0,3
788005.0,292268.0,3
801183.0,316191.0,3
378274.0,816341.0,1
406233.0,813072.0,1
401478.0,785370.0,1
448651.0,847994.0,1
434440.0,822866.0,1
369199.0,734895.0,1
413241.0,771595.0,1
369418.0,733394.0,1
378563.0,789670.0,1
424190.0,780052.0,1
415993.0,776671.0,1
421352.0,799127.0,1
432656.0,783776.0,1
383032.0,788008.0,1
414377.0,771258.0,1
425260.0,773661.0,1
469139.0,810971.0,1
442979.0,803024.0,1
347079.0,803582.0,1
412797.0,803180.0,1
352874.0,807952.0,1
442286.0,755194.0,1
416433.0,786159.0,1
419700.0,793447.0,1
422961.0,821595.0,1
400008.0,789447.0,1
429586.0,775621.0,1
457561.0,788245.0,1
415417.0,785827.0,1
389552.0,750700.0,1
415598.0,786499.0,1
443447.0,783977.0,1
404792.0,749210.0,1
389392.0,730768.0,1
473485.0,847098.0,1
491614.0,778997.0,1
437483.0,785130.0,1
452570.0,848127.0,1
417197.0,778243.0,1
405986.0,787439.0,1
437568.0,794323.0,1
420319.0,794333.0,1
418963.0,746595.0,1
469422.0,822619.0,1
397688.0,762902.0,1
418149.0,790096.0,1
407289.0,798870.0,1
360319.0,778673.0,1
473799.0,795167.0,1
445199.0,763833.0,1
446537.0,828924.0,1
410019.0,732704.0,1
396540.0,815936.0,1
407694.0,718592.0,1
417091.0,792545.0,1
442260.0,754050.0,1
411526.0,784658.0,1
407814.0,765816.0,1
394022.0,803914.0,1
481133.0,781413.0,1
396118.0,794614.0,1
458538.0,786673.0,1
419721.0,789724.0,1
424476.0,781552.0,1
377720.0,727349.0,1
448336.0,789445.0,1
412227.0,787450.0,1
408745.0,824896.0,1
417016.0,774869.0,1
395274.0,786698.0,1
412541.0,806622.0,1
412109.0,735920.0,1
407023.0,787385.0,1
376660.0,815936.0,1
396548.0,784388.0,1
508535.0,830710.0,1
414014.0,816460.0,1
440782.0,782395.0,1
436951.0,814456.0,1
393758.0,750953.0,1
418410.0,795929.0,1
425991.0,787029.0,1
414562.0,787355.0,1
408712.0,803987.0,1
377858.0,811790.0,1
417169.0,792321.0,1
460371.0,780609.0,1
388873.0,746909.0,1
395641.0,808571.0,1
386436.0,791388.0,1
440667.0,768761.0,1
438437.0,730300.0,1
370839.0,727884.0,1
422917.0,781112.0,1
391555.0,789079.0,1
402666.0,814060.0,1
474368.0,812039.0,1
406176.0,760860.0,1
463768.0,793154.0,1
446307.0,796523.0,1
402309.0,786873.0,1
413946.0,786971.0,1
425053.0,790566.0,1
416926.0,785113.0,1
424023.0,737453.0,1
417447.0,782955.0,1
415672.0,787824.0,1
416019.0,787562.0,1
417515.0,787648.0,1
459230.0,803720.0,1
447833.0,819282.0,1
418199.0,851439.0,1
417393.0,771537.0,1
441415.0,780207.0,1
399634.0,751940.0,1
373360.0,784232.0,1
419227.0,779215.0,1
434717.0,808225.0,1
397669.0,792218.0,1
412028.0,773494.0,1
407504.0,803363.0,1
365147.0,790525.0,1
459328.0,805961.0,1
429472.0,860855.0,1
480128.0,785580.0,1
344915.0,702496.0,1
410969.0,770451.0,1
407460.0,791404.0,1
420476.0,750495.0,1
397747.0,792207.0,1
408428.0,771823.0,1
397806.0,749559.0,1
426040.0,825049.0,1
407565.0,766203.0,1
414317.0,828815.0,1
410610.0,765111.0,1
412388.0,784064.0,1
406337.0,750353.0,1
422419.0,754048.0,1
416039.0,730594.0,1
414512.0,793966.0,1
398478.0,822297.0,1
424375.0,721868.0,1
403890.0,792420.0,1
400902.0,772920.0,1
402367.0,782997.0,1
472917.0,830363.0,1
418621.0,840463.0,1
329002.0,727685.0,1
418618.0,811618.0,1
425356.0,745471.0,1
470922.0,770669.0,1
400598.0,790903.0,1
412851.0,774986.0,1
492415.0,829655.0,1
400079.0,749617.0,1
417673.0,785965.0,1
424720.0,768456.0,1
386415.0,801444.0,1
426689.0,788562.0,1
430747.0,776261.0,1
405714.0,781619.0,1
429220.0,787348.0,1
427850.0,762201.0,1
434080.0,785264.0,1
473514.0,788780.0,1
412386.0,777312.0,1
430686.0,844506.0,1
409142.0,811927.0,1
422198.0,724557.0,1
392185.0,713763.0,1
412677.0,786951.0,1
399149.0,784376.0,1
382693.0,749680.0,1
527515.0,831478.0,1
448519.0,805219.0,1
430779.0,770145.0,1
413556.0,758080.0,1
392140.0,769924.0,1
416855.0,794023.0,1
389713.0,772742.0,1
398498.0,776720.0,1
473727.0,807244.0,1
389410.0,788256.0,1
377230.0,790154.0,1
415564.0,784464.0,1
456799.0,844976.0,1
444336.0,820061.0,1
404359.0,783919.0,1
453154.0,776082.0,1
425019.0,759577.0,1
415895.0,775793.0,1
420083.0,787815.0,1
506612.0,858859.0,1
436648.0,859634.0,1
403522.0,785432.0,1
401407.0,759648.0,1
403852.0,792473.0,1
367398.0,701756.0,1
412453.0,784453.0,1
382124.0,786262.0,1
416884.0,786845.0,1
448865.0,836781.0,1
383821.0,808301.0,1
414249.0,796590.0,1
457338.0,789952.0,1
396276.0,785999.0,1
390282.0,789593.0,1
409417.0,758123.0,1
397561.0,815245.0,1
397137.0,742547.0,1
402891.0,801910.0,1
385257.0,791581.0,1
409628.0,833942.0,1
369249.0,797068.0,1
460942.0,779073.0,1
423146.0,737571.0,1
420509.0,788730.0,1
422736.0,790806.0,1
401492.0,790336.0,1
394018.0,799506.0,1
385140.0,796221.0,1
430243.0,781020.0,1
416775.0,806826.0,1
425796.0,795608.0,1
411488.0,837715.0,1
433989.0,765343.0,1
415806.0,783594.0,1
417023.0,771194.0,1
415270.0,775605.0,1
391163.0,811977.0,1
405350.0,762342.0,1
414860.0,773921.0,1
371779.0,814715.0,1
415631.0,805010.0,1
387951.0,813315.0,1
472691.0,802678.0,1
471930.0,844046.0,1
432892.0,815223.0,1
366654.0,802105.0,1
411721.0,717873.0,1
426115.0,814461.0,1
375997.0,806621.0,1
385561.0,718324.0,1
375573.0,809249.0,1
437958.0,784801.0,1
363857.0,784086.0,1
430138.0,818594.0,1
435085.0,791564.0,1
414036.0,742189.0,1
425611.0,793420.0,1
439793.0,740773.0,1
467107.0,817914.0,1
387631.0,749416.0,1
430001.0,765803.0,1
384225.0,784381.0,1
417146.0,822655.0,1
427371.0,819417.0,1
399587.0,792459.0,1
474648.0,778810.0,1
345692.0,788190.0,1
409515.0,756149.0,1
469782.0,780552.0,1
468229.0,827985.0,1
414951.0,852935.0,1
409701.0,750741.0,1
442826.0,795974.0,1
459556.0,818479.0,1
423311.0,813081.0,1
476144.0,772131.0,1
391976.0,789416.0,1
448649.0,813885.0,1
496658.0,816784.0,1
410336.0,767856.0,1
430167.0,811490.0,1
357467.0,797530.0,1
404220.0,785798.0,1
446368.0,763341.0,1
439514.0,758221.0,1
424145.0,743433.0,1
470062.0,771204.0,1
421187.0,779726.0,1
396617.0,801039.0,1
394708.0,739061.0,1
394912.0,755947.0,1
425950.0,788228.0,1
408283.0,806161.0,1
419520.0,778231.0,1
441950.0,785934.0,1
409814.0,836083.0,1
395817.0,788858.0,1
422776.0,808328.0,1
363959.0,810632.0,1
422943.0,804253.0,1
407626.0,771304.0,1
347933.0,797148.0,1
457554.0,795861.0,1
394274.0,837428.0,1
445674.0,768552.0,1
416359.0,786189.0,1
419485.0,804779.0,1
376176.0,795539.0,1
349378.0,805118.0,1
456566.0,743012.0,1
484407.0,809641.0,1
403329.0,717870.0,1
392795.0,817694.0,1
372621.0,772258.0,1
461751.0,817277.0,1
489182.0,773934.0,1
427388.0,815258.0,1
418155.0,819422.0,1
437035.0,755506.0,1
416444.0,783624.0,1
777811.0,751059.0,5
823974.0,725025.0,5
749723.0,781756.0,5
875160.0,698897.0,5
828194.0,742074.0,5
800913.0,749332.0,5
831165.0,783383.0,5
793019.0,764859.0,5
810467.0,706465.0,5
803540.0,754792.0,5
844662.0,654835.0,5
803962.0,731228.0,5
823065.0,733789.0,5
822156.0,732296.0,5
811389.0,723618.0,5
779936.0,774397.0,5
854769.0,713941.0,5
803457.0,735855.0,5
790712.0,745884.0,5
815652.0,733433.0,5
841289.0,701474.0,5
832054.0,751330.0,5
786032.0,701427.0,5
824835.0,704997.0,5
832886.0,781206.0,5
818063.0,797960.0,5
857620.0,709386.0,5
828589.0,652430.0,5
836727.0,737798.0,5
776000.0,780347.0,5
824098.0,714042.0,5
830583.0,710990.0,5
862189.0,735684.0,5
815131.0,704478.0,5
828478.0,655405.0,5
784838.0,771499.0,5
880362.0,706363.0,5
767482.0,727417.0,5
825070.0,734215.0,5
813781.0,738344.0,5
840863.0,737972.0,5
840943.0,739611.0,5
828276.0,762021.0,5
822883.0,728314.0,5
854886.0,769396.0,5
829623.0,732301.0,5
835428.0,731189.0,5
818338.0,800932.0,5
823417.0,701523.0,5
812152.0,689907.0,5
829969.0,699565.0,5
861563.0,661642.0,5
816955.0,747629.0,5
774526.0,757135.0,5
803577.0,712231.0,5
798056.0,714380.0,5
881789.0,727577.0,5
858867.0,700187.0,5
834996.0,728517.0,5
839121.0,736259.0,5
841045.0,738224.0,5
840100.0,728839.0,5
874544.0,686096.0,5
862334.0,747768.0,5
820253.0,714625.0,5
817862.0,732248.0,5
844729.0,682981.0,5
846484.0,719360.0,5
793669.0,724513.0,5
821906.0,732848.0,5
826465.0,646980.0,5
820350.0,748217.0,5
814540.0,707227.0,5
838223.0,766380.0,5
869471.0,745767.0,5
816054.0,733183.0,5
801177.0,737548.0,5
848783.0,748980.0,5
814105.0,723255.0,5
838036.0,749612.0,5
840270.0,725456.0,5
789601.0,714058.0,5
827169.0,708345.0,5
789448.0,738035.0,5
835030.0,668528.0,5
848151.0,741869.0,5
816091.0,747764.0,5
801511.0,761094.0,5
822670.0,689090.0,5
873672.0,734440.0,5
808153.0,737136.0,5
755829.0,751858.0,5
814697.0,729209.0,5
832758.0,707452.0,5
848264.0,762550.0,5
799903.0,784535.0,5
802674.0,727804.0,5
828147.0,808738.0,5
820707.0,705318.0,5
824594.0,741664.0,5
804474.0,747122.0,5
823108.0,733302.0,5
839514.0,728923.0,5
827711.0,732798.0,5
851644.0,721350.0,5
816390.0,711340.0,5
798007.0,723995.0,5
790612.0,738038.0,5
821909.0,728448.0,5
865754.0,723314.0,5
735295.0,814058.0,4
877711.0,703850.0,5
792990.0,685472.0,5
809917.0,762429.0,5
789388.0,712210.0,5
879221.0,655497.0,5
876935.0,684572.0,5
819439.0,735945.0,5
807929.0,730950.0,5
850451.0,764015.0,5
811748.0,766105.0,5
824765.0,761114.0,5
848586.0,789052.0,5
812231.0,739235.0,5
873532.0,712802.0,5
842590.0,716861.0,5
832798.0,734777.0,5
817640.0,730018.0,5
824322.0,744720.0,5
777697.0,798253.0,5
868493.0,677155.0,5
805658.0,698630.0,5
840349.0,790707.0,5
796953.0,726378.0,5
832589.0,782799.0,5
817441.0,709397.0,5
856029.0,730586.0,5
805089.0,762512.0,5
854628.0,773470.0,5
846090.0,651306.0,5
796943.0,674692.0,5
827192.0,785316.0,5
818209.0,719382.0,5
795200.0,786424.0,5
817777.0,724927.0,5
846512.0,683647.0,5
825080.0,746712.0,5
824674.0,806300.0,5
822616.0,735941.0,5
805391.0,732659.0,5
823250.0,732088.0,5
850376.0,739749.0,5
821337.0,733356.0,5
796786.0,724615.0,5
862425.0,741010.0,5
825690.0,703854.0,5
800872.0,672603.0,5
825395.0,715780.0,5
843911.0,730036.0,5
820238.0,732928.0,5
848570.0,703697.0,5
825515.0,738284.0,5
820367.0,732364.0,5
822025.0,749214.0,5
837718.0,761480.0,5
871723.0,685520.0,5
850904.0,720163.0,5
816775.0,739989.0,5
822103.0,732911.0,5
815455.0,755486.0,5
835648.0,756631.0,5
868626.0,737331.0,5
825997.0,730652.0,5
817252.0,730254.0,5
877906.0,722361.0,5
808440.0,729985.0,5
791470.0,751113.0,5
841654.0,764836.0,5
825429.0,732758.0,5
751215.0,828365.0,4
804706.0,788001.0,5
820194.0,734233.0,5
869991.0,705259.0,5
809779.0,663654.0,5
822078.0,711840.0,5
866139.0,717138.0,5
829379.0,718721.0,5
837582.0,763266.0,5
783999.0,732487.0,5
800231.0,732106.0,5
862569.0,719773.0,5
838764.0,652792.0,5
849343.0,713993.0,5
828297.0,715480.0,5
821259.0,750089.0,5
874643.0,729921.0,5
819058.0,729125.0,5
817018.0,729596.0,5
797804.0,715342.0,5
838515.0,768484.0,5
829398.0,724900.0,5
754841.0,796342.0,5
878690.0,739021.0,5
821908.0,732077.0,5
786495.0,759000.0,5
823987.0,687499.0,5
787988.0,741203.0,5
839339.0,726322.0,5
822425.0,727248.0,5
854412.0,725575.0,5
788351.0,737539.0,5
838587.0,700545.0,5
840425.0,762290.0,5
817395.0,734126.0,5
807913.0,737341.0,5
872285.0,750708.0,5
788166.0,702817.0,5
836225.0,709204.0,5
822293.0,786837.0,5
888892.0,702505.0,5
822623.0,745025.0,5
795124.0,696429.0,5
832238.0,725434.0,5
815593.0,689874.0,5
843572.0,784897.0,5
833716.0,758772.0,5
822474.0,780882.0,5
801102.0,759330.0,5
841752.0,731595.0,5
808779.0,732990.0,5
822533.0,758955.0,5
817807.0,715863.0,5
837977.0,728814.0,5
801204.0,755177.0,5
819628.0,810748.0,5
814732.0,734183.0,5
832086.0,716249.0,5
876503.0,659112.0,5
823856.0,749355.0,5
819737.0,737829.0,5
818756.0,743887.0,5
811971.0,664520.0,5
801526.0,705057.0,5
827031.0,734324.0,5
843891.0,721571.0,5
810460.0,735109.0,5
782368.0,759352.0,5
801172.0,715857.0,5
866928.0,716961.0,5
829673.0,730398.0,5
850888.0,736810.0,5
786935.0,732999.0,5
769656.0,721159.0,5
804925.0,735492.0,5
870054.0,738900.0,5
818932.0,728481.0,5
853692.0,746285.0,5
807072.0,687932.0,5
750459.0,800215.0,5
823959.0,737866.0,5
835804.0,710711.0,5
790206.0,747165.0,5
813092.0,739043.0,5
801810.0,728106.0,5
825404.0,733788.0,5
820947.0,730529.0,5
827081.0,694553.0,5
822664.0,720677.0,5
768565.0,762193.0,5
823113.0,733720.0,5
821079.0,800206.0,5
828531.0,731300.0,5
820757.0,738235.0,5
781260.0,737962.0,5
819438.0,662327.0,5
833504.0,744564.0,5
789552.0,731946.0,5
822843.0,763117.0,5
775555.0,719683.0,5
865084.0,713697.0,5
820455.0,729132.0,5
849547.0,738979.0,5
762269.0,792243.0,5
821464.0,700760.0,5
824013.0,753991.0,5
803369.0,698753.0,5
823589.0,728083.0,5
796353.0,710294.0,5
820083.0,728310.0,5
829133.0,735488.0,5
823455.0,747394.0,5
818542.0,740451.0,5
809623.0,734937.0,5
841762.0,777572.0,5
802184.0,751958.0,5
803874.0,675499.0,5
830065.0,739162.0,5
824140.0,729074.0,5
852730.0,769968.0,5
866760.0,753547.0,5
811031.0,703822.0,5
825414.0,734466.0,5
816615.0,770855.0,5
786227.0,765634.0,5
817269.0,733788.0,5
809453.0,736879.0,5
757421.0,774445.0,5
818251.0,731938.0,5
788020.0,765562.0,5
823618.0,734216.0,5
812295.0,728049.0,5
837545.0,696979.0,5
797918.0,694723.0,5
822224.0,742716.0,5
821895.0,732158.0,5
797233.0,713358.0,5
800258.0,733812.0,5
813319.0,737376.0,5
860951.0,162251.0,15
860387.0,157731.0,15
857363.0,154639.0,15
863391.0,126214.0,15
839094.0,159543.0,15
849248.0,164090.0,15
875990.0,133950.0,15
905936.0,150106.0,15
895835.0,123485.0,15
818079.0,108536.0,15
898976.0,198452.0,15
869087.0,153957.0,15
849653.0,143827.0,15
834293.0,161575.0,15
844234.0,153197.0,15
835128.0,212354.0,15
828179.0,155649.0,15
875410.0,128878.0,15
850965.0,156224.0,15
800816.0,176612.0,15
832621.0,150748.0,15
876863.0,176593.0,15
826192.0,172906.0,15
862679.0,188399.0,15
839974.0,154358.0,15
854338.0,135067.0,15
855233.0,141357.0,15
850538.0,160159.0,15
868442.0,160118.0,15
888714.0,205560.0,15
864185.0,161232.0,15
851280.0,156049.0,15
875825.0,204959.0,15
840375.0,155757.0,15
892340.0,149050.0,15
873527.0,126426.0,15
836433.0,200181.0,15
826499.0,142732.0,15
854922.0,159650.0,15
883537.0,210648.0,15
807420.0,168787.0,15
840299.0,118907.0,15
834602.0,213597.0,15
864053.0,139325.0,15
856454.0,117143.0,15
857972.0,160214.0,15
850046.0,108829.0,15
804936.0,148980.0,15
864374.0,150149.0,15
858587.0,150101.0,15
870247.0,121460.0,15
855656.0,162100.0,15
850176.0,127908.0,15
850057.0,157848.0,15
828005.0,145089.0,15
861994.0,160156.0,15
849537.0,157510.0,15
867125.0,168762.0,15
838142.0,156770.0,15
847644.0,164348.0,15
856349.0,158150.0,15
860606.0,152843.0,15
869288.0,168662.0,15
838328.0,185708.0,15
860986.0,140320.0,15
829229.0,143653.0,15
866051.0,172932.0,15
871556.0,194747.0,15
865489.0,161762.0,15
851840.0,110475.0,15
846256.0,154064.0,15
836081.0,178429.0,15
859707.0,202806.0,15
849938.0,156042.0,15
894124.0,202052.0,15
864325.0,170593.0,15
848108.0,161786.0,15
817619.0,181929.0,15
858058.0,196183.0,15
861242.0,161722.0,15
885793.0,172115.0,15
850577.0,157049.0,15
877125.0,161227.0,15
883864.0,111281.0,15
891699.0,170415.0,15
802394.0,87835.0,15
862759.0,157988.0,15
824185.0,167369.0,15
813124.0,157130.0,15
840335.0,164912.0,15
858042.0,178196.0,15
846257.0,145306.0,15
858516.0,151301.0,15
851282.0,153750.0,15
905005.0,164126.0,15
857309.0,139276.0,15
845589.0,192916.0,15
895245.0,150977.0,15
863995.0,150435.0,15
862642.0,159379.0,15
861631.0,169376.0,15
858934.0,162874.0,15
912023.0,160368.0,15
842824.0,177832.0,15
824560.0,133679.0,15
842281.0,175525.0,15
860845.0,155375.0,15
841096.0,188742.0,15
821428.0,153468.0,15
840357.0,169437.0,15
827802.0,189078.0,15
844734.0,164570.0,15
847812.0,114745.0,15
887269.0,166179.0,15
830159.0,134329.0,15
850925.0,157811.0,15
847276.0,121785.0,15
870599.0,149782.0,15
780153.0,136449.0,15
814379.0,132655.0,15
834483.0,173379.0,15
852423.0,159874.0,15
854012.0,157449.0,15
883375.0,158482.0,15
850219.0,110573.0,15
836450.0,151205.0,15
881217.0,172320.0,15
813239.0,139752.0,15
871255.0,140340.0,15
789356.0,159383.0,15
873690.0,156226.0,15
858834.0,149259.0,15
778867.0,140736.0,15
872057.0,122354.0,15
882097.0,161493.0,15
787336.0,144875.0,15
849751.0,167033.0,15
843833.0,170416.0,15
850663.0,157840.0,15
812240.0,177846.0,15
838609.0,140215.0,15
850959.0,157920.0,15
859703.0,156782.0,15
854753.0,190254.0,15
895723.0,126593.0,15
857833.0,170565.0,15
850200.0,164752.0,15
845303.0,150476.0,15
877422.0,134877.0,15
850153.0,155701.0,15
819256.0,201704.0,15
871689.0,155981.0,15
818014.0,140185.0,15
815298.0,167390.0,15
864088.0,171351.0,15
902529.0,162041.0,15
908069.0,146564.0,15
799326.0,106384.0,15
854996.0,81095.0,15
843813.0,182032.0,15
888504.0,147016.0,15
877964.0,242679.0,15
851866.0,156896.0,15
821361.0,175823.0,15
810775.0,160434.0,15
843150.0,156573.0,15
844088.0,150085.0,15
830620.0,117906.0,15
835790.0,127898.0,15
819541.0,117296.0,15
858703.0,157931.0,15
857846.0,153281.0,15
876408.0,183646.0,15
842576.0,153072.0,15
848974.0,199427.0,15
862925.0,243856.0,15
856561.0,128799.0,15
834171.0,189042.0,15
872516.0,217570.0,15
869167.0,105039.0,15
819670.0,183364.0,15
846575.0,159002.0,15
895127.0,152283.0,15
830445.0,186055.0,15
792179.0,148539.0,15
852410.0,187705.0,15
853554.0,158012.0,15
927170.0,250593.0,15
825605.0,167750.0,15
846731.0,162515.0,15
878170.0,123364.0,15
884966.0,133900.0,15
866987.0,154415.0,15
846284.0,113820.0,15
873474.0,159567.0,15
878258.0,158631.0,15
882306.0,173836.0,15
856924.0,169071.0,15
794260.0,129843.0,15
851568.0,114049.0,15
868736.0,179354.0,15
925732.0,210388.0,15
868375.0,161990.0,15
855593.0,161003.0,15
849599.0,157084.0,15
868317.0,177001.0,15
800304.0,163436.0,15
823740.0,169808.0,15
856202.0,158811.0,15
883929.0,121705.0,15
851010.0,157593.0,15
872506.0,157517.0,15
850533.0,160136.0,15
816393.0,185056.0,15
850911.0,157772.0,15
812942.0,85345.0,15
835581.0,147664.0,15
913298.0,162009.0,15
858172.0,137586.0,15
872781.0,154295.0,15
871597.0,185239.0,15
923120.0,170161.0,15
837019.0,215527.0,15
835702.0,176502.0,15
837017.0,154170.0,15
859177.0,143273.0,15
866851.0,129505.0,15
855852.0,134834.0,15
924289.0,171369.0,15
860154.0,113364.0,15
836739.0,162321.0,15
850775.0,162968.0,15
884750.0,154317.0,15
849681.0,125345.0,15
854463.0,159977.0,15
883567.0,161769.0,15
848137.0,87922.0,15
823676.0,152154.0,15
848135.0,130674.0,15
844400.0,150812.0,15
864494.0,136669.0,15
878157.0,210496.0,15
851088.0,157432.0,15
786528.0,104976.0,15
860106.0,157080.0,15
861412.0,205239.0,15
820675.0,168154.0,15
834036.0,144156.0,15
842907.0,133992.0,15
846720.0,162060.0,15
871544.0,144135.0,15
849003.0,115373.0,15
856753.0,145184.0,15
839757.0,190518.0,15
865113.0,161302.0,15
846936.0,161313.0,15
845802.0,172285.0,15
844357.0,163402.0,15
881106.0,105566.0,15
877835.0,151890.0,15
839445.0,149778.0,15
866560.0,165110.0,15
845798.0,213118.0,15
818668.0,120458.0,15
831446.0,182864.0,15
900955.0,185733.0,15
836647.0,167267.0,15
874672.0,104851.0,15
849126.0,121710.0,15
889142.0,154467.0,15
855993.0,225023.0,15
854095.0,158510.0,15
850787.0,125678.0,15
859298.0,139834.0,15
829216.0,121494.0,15
837676.0,148639.0,15
851919.0,127963.0,15
841353.0,125628.0,15
848563.0,117708.0,15
893106.0,176595.0,15
852503.0,146551.0,15
848290.0,143184.0,15
825541.0,151495.0,15
883231.0,232031.0,15
864667.0,143614.0,15
805089.0,96540.0,15
854909.0,149938.0,15
847097.0,176660.0,15
910435.0,171155.0,15
878749.0,188759.0,15
871780.0,158486.0,15
841292.0,155620.0,15
846011.0,160034.0,15
855969.0,193497.0,15
860927.0,110775.0,15
808069.0,134402.0,15
848413.0,183717.0,15
791722.0,162000.0,15
852104.0,134678.0,15
806567.0,140259.0,15
847423.0,161535.0,15
867993.0,162841.0,15
828038.0,149181.0,15
849404.0,170474.0,15
841967.0,150890.0,15
850945.0,161669.0,15
888697.0,143969.0,15
813868.0,188928.0,15
864991.0,159874.0,15
881109.0,152198.0,15
811694.0,188374.0,15
900360.0,153622.0,15
885764.0,151791.0,15
814553.0,164190.0,15
895623.0,178393.0,15
829790.0,205695.0,15
865170.0,173755.0,15
850453.0,155432.0,15
864211.0,216526.0,15
848770.0,111076.0,15
791199.0,164843.0,15
834165.0,140684.0,15
843905.0,219401.0,15
857328.0,143692.0,15
838146.0,159541.0,15
340693.0,569371.0,8
315965.0,587160.0,8
405652.0,622938.0,8
359831.0,554975.0,8
349296.0,557688.0,8
289423.0,559134.0,8
408689.0,562521.0,8
321312.0,574721.0,8
286568.0,547471.0,8
370259.0,585265.0,8
339967.0,562602.0,8
410917.0,614521.0,8
283570.0,563726.0,8
347197.0,591316.0,8
342064.0,570045.0,8
317331.0,556086.0,8
335792.0,541310.0,8
330455.0,568758.0,8
366794.0,582584.0,8
378697.0,591764.0,8
339981.0,565418.0,8
338342.0,566864.0,8
338474.0,563350.0,8
432052.0,613723.0,8
340064.0,550509.0,8
334354.0,578047.0,8
296775.0,572432.0,8
353335.0,592567.0,8
334016.0,562797.0,8
328438.0,532686.0,8
365927.0,571713.0,8
343247.0,564376.0,8
325057.0,581121.0,8
330881.0,579058.0,8
378648.0,579579.0,8
339115.0,566863.0,8
333299.0,597586.0,8
367048.0,553451.0,8
345840.0,561591.0,8
366934.0,581294.0,8
358793.0,547046.0,8
393570.0,579641.0,8
304399.0,545221.0,8
353979.0,578923.0,8
337002.0,589527.0,8
327738.0,559624.0,8
334549.0,579199.0,8
297110.0,556485.0,8
327403.0,548875.0,8
413894.0,564459.0,8
326043.0,553772.0,8
337811.0,558454.0,8
333901.0,532836.0,8
340545.0,568654.0,8
358483.0,554925.0,8
318065.0,581616.0,8
367751.0,578289.0,8
331667.0,570387.0,8
321519.0,561045.0,8
299578.0,572052.0,8
339632.0,536577.0,8
339993.0,544064.0,8
268672.0,508874.0,8
332960.0,569308.0,8
345042.0,566973.0,8
352220.0,602793.0,8
333934.0,537464.0,8
312386.0,555195.0,8
334182.0,558498.0,8
335328.0,562880.0,8
303637.0,570253.0,8
353145.0,580916.0,8
325118.0,556212.0,8
294310.0,563522.0,8
371563.0,547746.0,8
373894.0,555347.0,8
407040.0,577739.0,8
337298.0,565486.0,8
357813.0,572855.0,8
304157.0,566007.0,8
335918.0,566525.0,8
330127.0,570072.0,8
339212.0,538693.0,8
357275.0,551101.0,8
334785.0,565575.0,8
326152.0,563230.0,8
345903.0,561314.0,8
238748.0,551038.0,9
331175.0,551842.0,8
352520.0,566829.0,8
341675.0,560197.0,8
354746.0,552404.0,8
412500.0,594675.0,8
277027.0,536847.0,8
322209.0,547247.0,8
375619.0,553791.0,8
288068.0,561932.0,8
344770.0,570309.0,8
284031.0,566654.0,8
324442.0,567881.0,8
356246.0,549756.0,8
404420.0,577532.0,8
336829.0,562228.0,8
293591.0,522132.0,8
342963.0,582232.0,8
338860.0,561151.0,8
324055.0,590810.0,8
328331.0,541094.0,8
357737.0,597104.0,8
392534.0,593343.0,8
298665.0,568245.0,8
343376.0,575709.0,8
338863.0,563107.0,8
361370.0,552483.0,8
338935.0,567577.0,8
350117.0,535859.0,8
371000.0,555062.0,8
368617.0,554066.0,8
331795.0,529654.0,8
344237.0,573700.0,8
280110.0,522025.0,8
328657.0,564864.0,8
452457.0,597461.0,8
333669.0,542228.0,8
320628.0,564596.0,8
316642.0,553340.0,8
340712.0,563178.0,8
349955.0,535578.0,8
261538.0,525920.0,8
411008.0,607342.0,8
317136.0,563479.0,8
297022.0,574097.0,8
319927.0,524637.0,8
359772.0,568428.0,8
344791.0,578536.0,8
324093.0,580491.0,8
373907.0,558472.0,8
321905.0,573147.0,8
341721.0,576427.0,8
386373.0,606623.0,8
338588.0,543875.0,8
341578.0,558672.0,8
371616.0,570648.0,8
331987.0,568997.0,8
341329.0,565571.0,8
334350.0,556417.0,8
330482.0,571217.0,8
338950.0,572545.0,8
289087.0,565427.0,8
343997.0,585193.0,8
225786.0,529417.0,9
365567.0,547606.0,8
320827.0,519618.0,8
286990.0,529505.0,8
321517.0,569562.0,8
388806.0,569524.0,8
411796.0,581307.0,8
331440.0,551885.0,8
325958.0,573367.0,8
348171.0,569251.0,8
344239.0,530765.0,8
419151.0,577335.0,8
321309.0,569996.0,8
386377.0,574187.0,8
342099.0,561508.0,8
366688.0,562413.0,8
321657.0,538660.0,8
403004.0,613466.0,8
363304.0,553764.0,8
384922.0,582877.0,8
304556.0,568584.0,8
336787.0,538925.0,8
327998.0,571265.0,8
303269.0,531608.0,8
313649.0,523765.0,8
337889.0,558031.0,8
360165.0,553303.0,8
340032.0,561740.0,8
338634.0,564414.0,8
351459.0,565704.0,8
412767.0,593558.0,8
283474.0,532727.0,8
336928.0,557412.0,8
366478.0,594677.0,8
343450.0,570659.0,8
331616.0,565089.0,8
354529.0,573677.0,8
276703.0,559740.0,8
282066.0,560178.0,8
331366.0,567311.0,8
321910.0,573825.0,8
290107.0,571927.0,8
281979.0,518199.0,8
374695.0,563163.0,8
340191.0,567638.0,8
338583.0,563529.0,8
358312.0,559466.0,8
338474.0,564151.0,8
338205.0,566246.0,8
342901.0,582886.0,8
265815.0,540385.0,8
304086.0,576100.0,8
347966.0,563506.0,8
346144.0,560472.0,8
352381.0,592377.0,8
318681.0,528806.0,8
369366.0,561201.0,8
310041.0,533285.0,8
344314.0,595915.0,8
363956.0,609339.0,8
328586.0,551385.0,8
330767.0,552963.0,8
312558.0,520650.0,8
315850.0,573154.0,8
379388.0,567285.0,8
329905.0,579935.0,8
352209.0,548608.0,8
357679.0,548036.0,8
339288.0,588052.0,8
330335.0,568221.0,8
341559.0,562163.0,8
341925.0,569393.0,8
327695.0,543105.0,8
335999.0,549795.0,8
302359.0,534317.0,8
332586.0,566187.0,8
330750.0,561657.0,8
400380.0,588221.0,8
337910.0,562252.0,8
336549.0,565597.0,8
343717.0,559909.0,8
332116.0,594868.0,8
358384.0,556361.0,8
415881.0,584352.0,8
345164.0,579614.0,8
316395.0,537403.0,8
340970.0,564831.0,8
340383.0,600071.0,8
326627.0,534442.0,8
356865.0,545785.0,8
306298.0,551902.0,8
347575.0,565576.0,8
302583.0,571881.0,8
352424.0,557574.0,8
279008.0,521035.0,8
335029.0,553213.0,8
338753.0,563384.0,8
324032.0,577807.0,8
327973.0,534972.0,8
321924.0,527106.0,8
345495.0,579864.0,8
385091.0,599401.0,8
361138.0,551307.0,8
332884.0,533990.0,8
269126.0,508814.0,8
350056.0,566657.0,8
374117.0,546122.0,8
350283.0,559840.0,8
335525.0,567445.0,8
350798.0,535657.0,8
289002.0,520678.0,8
324516.0,522674.0,8
364844.0,571623.0,8
351815.0,574947.0,8
275524.0,559548.0,8
348773.0,583528.0,8
308966.0,523409.0,8
316313.0,570785.0,8
341966.0,561987.0,8
341446.0,538787.0,8
318823.0,565345.0,8
347522.0,587567.0,8
293746.0,544468.0,8
334983.0,572690.0,8
325125.0,564342.0,8
330145.0,546705.0,8
336336.0,563145.0,8
302353.0,545544.0,8
321216.0,569834.0,8
285152.0,520229.0,8
335027.0,565050.0,8
332270.0,556288.0,8
341016.0,545280.0,8
339640.0,561426.0,8
338633.0,563504.0,8
324952.0,565867.0,8
353583.0,554147.0,8
385183.0,556284.0,8
292636.0,520371.0,8
289176.0,571435.0,8
310944.0,559497.0,8
336974.0,563207.0,8
337173.0,557853.0,8
396903.0,591235.0,8
324440.0,567074.0,8
333378.0,557892.0,8
300366.0,565761.0,8
344437.0,529492.0,8
325871.0,558965.0,8
367641.0,556309.0,8
344552.0,554676.0,8
297212.0,546223.0,8
355924.0,577596.0,8
331340.0,524203.0,8
292932.0,558293.0,8
306346.0,523584.0,8
341966.0,586266.0,8
282430.0,560217.0,8
326251.0,549298.0,8
335201.0,585318.0,8
351564.0,573155.0,8
322365.0,530808.0,8
405542.0,600089.0,8
296763.0,528676.0,8
221233.0,538472.0,9
338084.0,560295.0,8
345674.0,579033.0,8
325883.0,560492.0,8
345123.0,577788.0,8
329859.0,564384.0,8
353939.0,570931.0,8
409616.0,575024.0,8
365058.0,545456.0,8
404782.0,557913.0,8
368724.0,598412.0,8
344096.0,575522.0,8
182618.0,346663.0,7
171461.0,348130.0,7
186536.0,330019.0,7
177241.0,292639.0,7
248683.0,383412.0,7
174057.0,358906.0,7
215243.0,289985.0,7
170688.0,343596.0,7
144502.0,345417.0,7
165689.0,356983.0,7
168443.0,367928.0,7
165158.0,343946.0,7
159711.0,352701.0,7
175781.0,341039.0,7
233480.0,289069.0,7
189029.0,366740.0,7
168573.0,350387.0,7
170271.0,306395.0,7
192775.0,350028.0,7
152054.0,345632.0,7
139299.0,314046.0,7
202053.0,322650.0,7
134695.0,350397.0,7
159654.0,381778.0,7
188093.0,324040.0,7
183236.0,338163.0,7
156984.0,430950.0,7
182751.0,435819.0,7
198908.0,372797.0,7
198517.0,325721.0,7
161345.0,350804.0,7
210497.0,359835.0,7
141993.0,331189.0,7
170788.0,373545.0,7
166449.0,363036.0,7
155479.0,345893.0,7
186380.0,363938.0,7
129332.0,360811.0,7
224225.0,376975.0,7
165945.0,345849.0,7
223550.0,323197.0,7
122755.0,434879.0,7
166584.0,349245.0,7
179472.0,342273.0,7
148749.0,359845.0,7
177505.0,332053.0,7
160222.0,380591.0,7
178869.0,370161.0,7
170002.0,370000.0,7
164354.0,347671.0,7
169966.0,368623.0,7
145867.0,363616.0,7
181561.0,358111.0,7
164915.0,350360.0,7
166973.0,341636.0,7
153181.0,277968.0,7
155505.0,355581.0,7
160835.0,347784.0,7
211265.0,396792.0,7
203254.0,362605.0,7
137967.0,354539.0,7
174064.0,357989.0,7
179609.0,408365.0,7
176506.0,351827.0,7
172879.0,347028.0,7
180506.0,330980.0,7
185966.0,359964.0,7
146070.0,316860.0,7
152957.0,321354.0,7
163190.0,331719.0,7
186914.0,361706.0,7
159191.0,309314.0,7
170425.0,316185.0,7
162040.0,316797.0,7
195000.0,354829.0,7
126702.0,351222.0,7
213064.0,290947.0,7
140612.0,350010.0,7
172885.0,352621.0,7
170155.0,343568.0,7
186806.0,334070.0,7
146072.0,288800.0,7
169293.0,348686.0,7
169502.0,411323.0,7
152861.0,407965.0,7
156041.0,295117.0,7
117585.0,391892.0,7
163981.0,344090.0,7
158664.0,310765.0,7
141113.0,351050.0,7
163796.0,345091.0,7
160369.0,353500.0,7
235671.0,300234.0,7
140876.0,334435.0,7
108232.0,373112.0,7
192460.0,388323.0,7
133744.0,308918.0,7
216209.0,437830.0,7
156945.0,339397.0,7
199859.0,316334.0,7
159496.0,331641.0,7
167845.0,341409.0,7
182435.0,365508.0,7
133795.0,360518.0,7
169628.0,365003.0,7
167262.0,399527.0,7
177728.0,352687.0,7
130683.0,363204.0,7
198197.0,344933.0,7
197735.0,355477.0,7
172020.0,342883.0,7
185561.0,342784.0,7
202011.0,337353.0,7
157394.0,346500.0,7
182050.0,424341.0,7
129664.0,341196.0,7
173188.0,355248.0,7
182499.0,310438.0,7
170384.0,339979.0,7
182093.0,321311.0,7
185696.0,338006.0,7
102221.0,374657.0,7
165881.0,364469.0,7
164875.0,349007.0,7
159120.0,340829.0,7
168274.0,374689.0,7
178495.0,373915.0,7
179811.0,342089.0,7
162606.0,363185.0,7
173908.0,362098.0,7
133580.0,348602.0,7
190634.0,357254.0,7
186503.0,369651.0,7
127134.0,379827.0,7
169145.0,355279.0,7
140121.0,346875.0,7
141860.0,324304.0,7
200251.0,306215.0,7
174373.0,363552.0,7
170022.0,347877.0,7
165657.0,338722.0,7
186361.0,351075.0,7
213022.0,338238.0,7
155228.0,357729.0,7
171614.0,435530.0,7
175586.0,365049.0,7
187858.0,381077.0,7
171074.0,293610.0,7
153738.0,360530.0,7
199043.0,270173.0,7
165886.0,348709.0,7
159428.0,300364.0,7
197463.0,378513.0,7
161273.0,347538.0,7
186082.0,330058.0,7
143732.0,279724.0,7
142899.0,329415.0,7
156977.0,424622.0,7
174878.0,362329.0,7
148915.0,338914.0,7
187324.0,377187.0,7
175766.0,380534.0,7
185071.0,362905.0,7
158550.0,354818.0,7
174672.0,345849.0,7
169880.0,343122.0,7
109090.0,326101.0,7
187854.0,354777.0,7
153903.0,353738.0,7
190241.0,329582.0,7
102983.0,382712.0,7
143117.0,345163.0,7
188985.0,367645.0,7
174834.0,411644.0,7
144228.0,344974.0,7
146821.0,297434.0,7
171848.0,343242.0,7
94382.0,320888.0,7
154412.0,310120.0,7
176490.0,344679.0,7
170616.0,352610.0,7
81648.0,360080.0,7
153607.0,379395.0,7
183970.0,355973.0,7
105444.0,361358.0,7
158571.0,361986.0,7
177579.0,319865.0,7
198892.0,322470.0,7
216975.0,296647.0,7
166654.0,350678.0,7
189102.0,333074.0,7
198712.0,303299.0,7
137098.0,384889.0,7
144306.0,328086.0,7
142455.0,343512.0,7
178319.0,318244.0,7
168886.0,353913.0,7
174656.0,393278.0,7
130805.0,354417.0,7
176433.0,350561.0,7
142252.0,329183.0,7
165476.0,377389.0,7
210102.0,333312.0,7
129453.0,321221.0,7
144705.0,375936.0,7
206687.0,342276.0,7
153583.0,352282.0,7
189499.0,356601.0,7
159392.0,351000.0,7
181463.0,326571.0,7
162975.0,335389.0,7
157691.0,334886.0,7
154167.0,353183.0,7
165607.0,362883.0,7
182239.0,333302.0,7
176982.0,354631.0,7
183516.0,374212.0,7
191099.0,341570.0,7
184570.0,360446.0,7
207804.0,354938.0,7
198493.0,326704.0,7
149269.0,341445.0,7
184798.0,392233.0,7
174438.0,319075.0,7
107124.0,325984.0,7
76052.0,360840.0,7
159302.0,337993.0,7
140101.0,341028.0,7
168149.0,345638.0,7
150009.0,371014.0,7
157193.0,350343.0,7
111476.0,358647.0,7
151164.0,332239.0,7
171318.0,373473.0,7
175169.0,349454.0,7
162600.0,392531.0,7
168067.0,340249.0,7
170102.0,323164.0,7
124132.0,340448.0,7
149863.0,284721.0,7
168079.0,347343.0,7
128953.0,304971.0,7
169228.0,364545.0,7
198415.0,321418.0,7
183610.0,321788.0,7
220869.0,407358.0,7
159472.0,372246.0,7
170837.0,353968.0,7
165653.0,317602.0,7
135537.0,356169.0,7
166973.0,334724.0,7
158750.0,312102.0,7
161448.0,373412.0,7
158713.0,379013.0,7
180493.0,319750.0,7
125715.0,302114.0,7
194033.0,340466.0,7
158871.0,312380.0,7
161387.0,343617.0,7
169121.0,347594.0,7
167653.0,352074.0,7
125902.0,318850.0,7
217686.0,296514.0,7
171986.0,429306.0,7
211360.0,279794.0,7
157181.0,346842.0,7
188909.0,341194.0,7
140128.0,325904.0,7
117874.0,389929.0,7
162921.0,317643.0,7
173978.0,346514.0,7
98533.0,355691.0,7
194956.0,372256.0,7
164847.0,287064.0,7
174410.0,372064.0,7
149092.0,352682.0,7
166323.0,352927.0,7
178515.0,370287.0,7
169337.0,348605.0,7
122986.0,334382.0,7
145282.0,391482.0,7
185919.0,363092.0,7
180519.0,368284.0,7
181227.0,347052.0,7
168284.0,309036.0,7
198251.0,335402.0,7
160922.0,347681.0,7
141861.0,328909.0,7
147911.0,328024.0,7
178264.0,330827.0,7
178235.0,346006.0,7
195151.0,317753.0,7
122360.0,357480.0,7
167330.0,332452.0,7
115363.0,305575.0,7
185661.0,404482.0,7
177858.0,352255.0,7
113068.0,317574.0,7
125675.0,360158.0,7
172842.0,351663.0,7
124724.0,459999.0,9
169069.0,348929.0,7
185253.0,344152.0,7
155625.0,262321.0,7
158201.0,315944.0,7
171277.0,361020.0,7
209553.0,326461.0,7
160165.0,362858.0,7
190725.0,336160.0,7
169994.0,409806.0,7
179463.0,361279.0,7
172662.0,388185.0,7
165284.0,425978.0,7
160206.0,303063.0,7
155690.0,368375.0,7
124516.0,325289.0,7
179326.0,321160.0,7
165573.0,305517.0,7
155508.0,372355.0,7
166799.0,318482.0,7
161780.0,324523.0,7
168805.0,351913.0,7
206519.0,355629.0,7
163091.0,350167.0,7
176144.0,353300.0,7
179483.0,359645.0,7
213951.0,352868.0,7
164046.0,346109.0,7
169569.0,346955.0,7
225356.0,318394.0,7
214966.0,358837.0,7
159253.0,343477.0,7
219779.0,330969.0,7
169876.0,340860.0,7
669118.0,354195.0,13
613682.0,399434.0,13
617399.0,392653.0,13
629247.0,364078.0,13
641070.0,424917.0,13
609532.0,388425.0,13
629369.0,398359.0,13
605911.0,379990.0,13
622550.0,391601.0,13
607905.0,403460.0,13
679465.0,399014.0,13
608711.0,373665.0,13
623498.0,399288.0,13
601212.0,370698.0,13
574989.0,407228.0,13
563517.0,391159.0,13
687717.0,379483.0,13
616040.0,330855.0,13
653024.0,367312.0,13
607440.0,403119.0,13
620240.0,397022.0,13
599215.0,391457.0,13
587478.0,424878.0,13
613921.0,394043.0,13
624328.0,392133.0,13
635887.0,384511.0,13
559877.0,401053.0,13
617755.0,401065.0,13
693265.0,388552.0,13
559933.0,403956.0,13
655859.0,416496.0,13
625919.0,402984.0,13
595215.0,391491.0,13
617493.0,379303.0,13
598657.0,442558.0,13
601644.0,436831.0,13
617118.0,374098.0,13
591051.0,389443.0,13
617296.0,386864.0,13
616468.0,394118.0,13
630384.0,376295.0,13
558770.0,390646.0,13
637332.0,414017.0,13
556780.0,409205.0,13
577473.0,384210.0,13
650054.0,432740.0,13
595063.0,328609.0,13
714443.0,433876.0,13
603566.0,410391.0,13
562356.0,383175.0,13
546166.0,423311.0,13
626539.0,394908.0,13
663820.0,416308.0,13
710910.0,464586.0,13
624310.0,398408.0,13
632546.0,389750.0,13
667249.0,417107.0,13
608309.0,377530.0,13
611357.0,393577.0,13
658219.0,436184.0,13
615032.0,402225.0,13
628131.0,406850.0,13
632390.0,363198.0,13
575116.0,345564.0,13
625784.0,441883.0,13
618893.0,419740.0,13
579564.0,428498.0,13
686324.0,364237.0,13
586242.0,396484.0,13
623248.0,399928.0,13
609219.0,362804.0,13
537211.0,376777.0,13
605310.0,394011.0,13
634973.0,450542.0,13
595411.0,413706.0,13
613037.0,419086.0,13
617444.0,397675.0,13
633304.0,366062.0,13
592620.0,446156.0,13
631633.0,337973.0,13
619637.0,395224.0,13
620330.0,398630.0,13
621964.0,409409.0,13
618446.0,372298.0,13
629789.0,426313.0,13
608687.0,374928.0,13
680646.0,375662.0,13
591831.0,438997.0,13
550086.0,442382.0,13
620330.0,396776.0,13
551236.0,401856.0,13
617178.0,414525.0,13
584938.0,344365.0,13
622636.0,456843.0,13
649987.0,408293.0,13
613719.0,382828.0,13
576987.0,392508.0,13
599766.0,409755.0,13
584077.0,398498.0,13
695514.0,321521.0,13
561388.0,355834.0,13
691600.0,397893.0,13
610676.0,449688.0,13
600114.0,413216.0,13
626811.0,372435.0,13
636291.0,360636.0,13
570384.0,393553.0,13
626700.0,401688.0,13
624106.0,398619.0,13
523329.0,404632.0,13
611954.0,360938.0,13
613536.0,395801.0,13
576488.0,397913.0,13
652745.0,406585.0,13
603193.0,371867.0,13
610026.0,444858.0,13
629725.0,405117.0,13
648429.0,422590.0,13
562772.0,382904.0,13
638823.0,411764.0,13
617764.0,397102.0,13
579922.0,379009.0,13
610698.0,396326.0,13
642607.0,374702.0,13
621629.0,397208.0,13
659321.0,404555.0,13
576235.0,462612.0,13
598628.0,425191.0,13
623502.0,420376.0,13
665965.0,449273.0,13
608923.0,385575.0,13
611579.0,407670.0,13
617573.0,386848.0,13
589862.0,418742.0,13
653325.0,396810.0,13
605073.0,372055.0,13
623137.0,408209.0,13
499262.0,398424.0,12
619339.0,401624.0,13
619295.0,397643.0,13
578878.0,420108.0,13
533577.0,355107.0,13
615934.0,426539.0,13
690195.0,475150.0,13
673514.0,390760.0,13
657120.0,386782.0,13
549945.0,372313.0,13
675858.0,410545.0,13
626586.0,415166.0,13
618449.0,404599.0,13
638783.0,399478.0,13
578113.0,395458.0,13
622557.0,361134.0,13
635324.0,420555.0,13
666135.0,430421.0,13
689725.0,465685.0,13
593601.0,445149.0,13
615094.0,383077.0,13
624599.0,417673.0,13
573969.0,434110.0,13
642881.0,403147.0,13
634383.0,422065.0,13
582389.0,428200.0,13
617330.0,369828.0,13
621989.0,394856.0,13
651828.0,468138.0,13
612936.0,398723.0,13
613351.0,407952.0,13
562777.0,437516.0,13
533212.0,366954.0,13
618113.0,400002.0,13
609344.0,448128.0,13
615342.0,413931.0,13
623989.0,368662.0,13
620187.0,400168.0,13
687638.0,439257.0,13
573925.0,449341.0,13
547270.0,403384.0,13
619318.0,361696.0,13
588432.0,446279.0,13
592924.0,393730.0,13
615325.0,383841.0,13
611875.0,422910.0,13
681941.0,370225.0,13
620818.0,396967.0,13
662154.0,385013.0,13
554938.0,347875.0,13
628655.0,321099.0,13
619628.0,399433.0,13
660333.0,393215.0,13
618391.0,348381.0,13
565829.0,395454.0,13
617689.0,397001.0,13
604554.0,393193.0,13
572417.0,403906.0,13
659193.0,384523.0,13
632060.0,367800.0,13
633894.0,404424.0,13
620610.0,395343.0,13
615175.0,397627.0,13
624293.0,333944.0,13
593689.0,316888.0,13
617319.0,374666.0,13
572244.0,349463.0,13
619239.0,400025.0,13
621622.0,390298.0,13
622101.0,371771.0,13
630205.0,338054.0,13
654147.0,407182.0,13
637594.0,420311.0,13
629004.0,392271.0,13
585207.0,336451.0,13
618349.0,397614.0,13
650157.0,402991.0,13
638966.0,486628.0,14
655201.0,411873.0,13
602335.0,433298.0,13
630706.0,388370.0,13
617428.0,397343.0,13
621537.0,390372.0,13
631365.0,450732.0,13
618406.0,404239.0,13
601390.0,453925.0,13
603945.0,361794.0,13
619427.0,396945.0,13
660103.0,414557.0,13
700340.0,335010.0,13
608182.0,416637.0,13
661166.0,404325.0,13
620939.0,385731.0,13
626371.0,349696.0,13
614683.0,367259.0,13
619332.0,401992.0,13
587453.0,401098.0,13
652349.0,321291.0,13
617954.0,395504.0,13
636807.0,424537.0,13
533889.0,333741.0,13
623386.0,387031.0,13
632952.0,385722.0,13
695010.0,449565.0,13
626843.0,367696.0,13
488205.0,420586.0,12
611547.0,411952.0,13
621241.0,368090.0,13
581240.0,362971.0,13
608747.0,363132.0,13
662177.0,420431.0,13
615665.0,405490.0,13
617447.0,366176.0,13
560393.0,389490.0,13
685785.0,409913.0,13
623433.0,400160.0,13
609091.0,437020.0,13
624343.0,403629.0,13
668879.0,388617.0,13
567105.0,437623.0,13
657450.0,365008.0,13
591955.0,378142.0,13
588952.0,437524.0,13
607665.0,447516.0,13
608601.0,418450.0,13
662830.0,420955.0,13
605891.0,428146.0,13
615239.0,317684.0,13
588933.0,366439.0,13
618053.0,357534.0,13
577773.0,434308.0,13
606558.0,458285.0,13
620955.0,389692.0,13
738190.0,371970.0,13
634661.0,446187.0,13
571692.0,449353.0,13
605650.0,447537.0,13
639502.0,371440.0,13
684568.0,380640.0,13
658735.0,399039.0,13
618157.0,384990.0,13
618869.0,398187.0,13
603388.0,403039.0,13
585255.0,415823.0,13
639507.0,438534.0,13
594112.0,399331.0,13
619475.0,400592.0,13
571570.0,375257.0,13
659297.0,373975.0,13
603285.0,449307.0,13
610199.0,394579.0,13
590500.0,418472.0,13
630904.0,404643.0,13
591725.0,406549.0,13
621560.0,397789.0,13
563182.0,367521.0,13
593879.0,444161.0,13
616054.0,371666.0,13
627679.0,395601.0,13
608195.0,447380.0,13
616737.0,397973.0,13
650120.0,324477.0,13
616926.0,398697.0,13
650859.0,460072.0,13
642157.0,345545.0,13
601369.0,411415.0,13
604703.0,392288.0,13
721905.0,378957.0,13
619415.0,397644.0,13
571043.0,347389.0,13
659191.0,459637.0,13
629978.0,400626.0,13
688487.0,383589.0,13
598053.0,438963.0,13
635242.0,356290.0,13
605899.0,412428.0,13
656642.0,407213.0,13
619924.0,406713.0,13
630067.0,372798.0,13
600007.0,377908.0,13
589164.0,386733.0,13
630491.0,379017.0,13
618676.0,377015.0,13
639613.0,388524.0,13
669409.0,415893.0,13
681608.0,436537.0,13
649968.0,371445.0,13
629000.0,417990.0,13
658884.0,390135.0,13
605407.0,427914.0,13
629891.0,414975.0,13
621778.0,364695.0,13
610722.0,418243.0,13
601599.0,388540.0,13
615649.0,429323.0,13
627206.0,402647.0,13
643965.0,366316.0,13
614877.0,402442.0,13
606486.0,420054.0,13
668482.0,358452.0,13
628715.0,434772.0,13
196462.0,887983.0,0
335970.0,846828.0,0
272394.0,837600.0,0
211165.0,825505.0,0
226433.0,837646.0,0
219686.0,859997.0,0
242087.0,844255.0,0
213558.0,873714.0,0
245144.0,845719.0,0
247997.0,878582.0,0
283576.0,809069.0,0
224676.0,859348.0,0
298214.0,820903.0,0
245468.0,859425.0,0
222704.0,824909.0,0
250921.0,819639.0,0
249209.0,801933.0,0
230591.0,874855.0,0
234091.0,876039.0,0
286769.0,862152.0,0
224789.0,866804.0,0
245023.0,849247.0,0
263626.0,851699.0,0
250198.0,825417.0,0
231263.0,863308.0,0
307605.0,840195.0,0
276635.0,875623.0,0
230399.0,842538.0,0
274070.0,798617.0,0
247939.0,831010.0,0
209922.0,850026.0,0
300726.0,862728.0,0
222727.0,770963.0,0
239505.0,885057.0,0
291561.0,845450.0,0
221122.0,832810.0,0
194258.0,914691.0,0
228928.0,855040.0,0
264627.0,808661.0,0
225916.0,843329.0,0
240221.0,843932.0,0
276600.0,823311.0,0
204854.0,810013.0,0
278326.0,829095.0,0
246047.0,846189.0,0
251032.0,876118.0,0
285258.0,848668.0,0
260943.0,889692.0,0
245928.0,867228.0,0
221002.0,870506.0,0
292872.0,896822.0,0
261896.0,897475.0,0
269723.0,849664.0,0
243941.0,843256.0,0
249370.0,872804.0,0
237120.0,856814.0,0
257472.0,849587.0,0
253703.0,834676.0,0
238080.0,830689.0,0
225991.0,869734.0,0
231781.0,799444.0,0
296892.0,901182.0,0
234604.0,822462.0,0
241473.0,860854.0,0
219864.0,842635.0,0
235901.0,856563.0,0
207217.0,878249.0,0
256949.0,820470.0,0
204597.0,863162.0,0
325275.0,866433.0,0
239344.0,814789.0,0
187894.0,871219.0,0
259973.0,862559.0,0
189752.0,865463.0,0
245707.0,844466.0,0
188480.0,870646.0,0
305915.0,854818.0,0
242443.0,851797.0,0
227138.0,824372.0,0
229970.0,878786.0,0
281971.0,825246.0,0
241166.0,845204.0,0
225846.0,874086.0,0
284077.0,905181.0,0
258975.0,837353.0,0
241972.0,840749.0,0
245602.0,857160.0,0
227363.0,836593.0,0
257926.0,936947.0,0
237689.0,839510.0,0
267799.0,847911.0,0
207552.0,772091.0,0
201567.0,881003.0,0
265125.0,820661.0,0
273165.0,852766.0,0
290210.0,818593.0,0
256487.0,868899.0,0
271395.0,862063.0,0
251958.0,886571.0,0
249253.0,856047.0,0
268289.0,878265.0,0
295375.0,861990.0,0
256316.0,818886.0,0
246470.0,842932.0,0
314666.0,849163.0,0
298979.0,774144.0,0
192885.0,887667.0,0
227819.0,900986.0,0
250003.0,859517.0,0
249857.0,844975.0,0
218369.0,851387.0,0
252586.0,806474.0,0
283017.0,850861.0,0
228095.0,843338.0,0
266980.0,825990.0,0
241381.0,844402.0,0
294053.0,878220.0,0
253339.0,750941.0,0
241413.0,845029.0,0
294326.0,807584.0,0
257856.0,846423.0,0
248210.0,886338.0,0
216846.0,737568.0,0
244743.0,830888.0,0
277055.0,860813.0,0
279923.0,805957.0,0
235041.0,823732.0,0
238524.0,842928.0,0
246453.0,883061.0,0
260590.0,827955.0,0
274554.0,845338.0,0
245254.0,907991.0,0
299821.0,866288.0,0
261120.0,804153.0,0
246729.0,879602.0,0
240308.0,844341.0,0
220620.0,819091.0,0
268926.0,813783.0,0
234219.0,845270.0,0
188424.0,849785.0,0
240171.0,843940.0,0
163766.0,859386.0,0
191257.0,802651.0,0
224852.0,934148.0,0
261093.0,844962.0,0
228711.0,848090.0,0
183478.0,851724.0,0
201080.0,823689.0,0
118590.0,857150.0,0
264798.0,767716.0,0
313609.0,860065.0,0
232246.0,844226.0,0
237757.0,834092.0,0
222682.0,806314.0,0
243331.0,844022.0,0
197362.0,870337.0,0
229857.0,843917.0,0
244257.0,776516.0,0
224083.0,852922.0,0
309399.0,838744.0,0
271205.0,817620.0,0
242247.0,864253.0,0
235337.0,831592.0,0
232357.0,810798.0,0
252199.0,828023.0,0
267049.0,950249.0,0
245678.0,821691.0,0
241138.0,844374.0,0
246182.0,840370.0,0
264175.0,824349.0,0
266977.0,887963.0,0
274685.0,883097.0,0
244260.0,847060.0,0
256073.0,917590.0,0
258416.0,835432.0,0
241687.0,844256.0,0
282603.0,846165.0,0
228032.0,893055.0,0
226949.0,889283.0,0
264274.0,813041.0,0
139601.0,914203.0,0
229131.0,842806.0,0
236274.0,861302.0,0
261819.0,863523.0,0
200005.0,825237.0,0
310591.0,841778.0,0
244049.0,817710.0,0
235022.0,911236.0,0
256512.0,837094.0,0
236495.0,861569.0,0
256889.0,880721.0,0
258528.0,856273.0,0
233284.0,860085.0,0
257480.0,849037.0,0
269354.0,811847.0,0
222445.0,891438.0,0
222356.0,840843.0,0
236207.0,886769.0,0
174371.0,864697.0,0
159683.0,838958.0,0
237750.0,837381.0,0
241545.0,838673.0,0
256082.0,831969.0,0
246468.0,838071.0,0
205919.0,844377.0,0
266111.0,813259.0,0
244246.0,847589.0,0
244899.0,812192.0,0
219316.0,898830.0,0
251866.0,864044.0,0
238308.0,790723.0,0
209264.0,836661.0,0
242519.0,845811.0,0
351883.0,890475.0,0
278929.0,842054.0,0
244027.0,850132.0,0
248570.0,843156.0,0
224681.0,858432.0,0
210276.0,824826.0,0
220884.0,916718.0,0
240846.0,841609.0,0
257093.0,801567.0,0
236749.0,813868.0,0
332237.0,911313.0,0
288954.0,846991.0,0
238084.0,844799.0,0
236431.0,844100.0,0
299772.0,828655.0,0
241633.0,846949.0,0
230107.0,858332.0,0
256216.0,903741.0,0
258135.0,880184.0,0
232331.0,872896.0,0
183583.0,787453.0,0
264293.0,890923.0,0
213208.0,833959.0,0
224288.0,931476.0,0
261622.0,869706.0,0
297183.0,841456.0,0
241849.0,815141.0,0
308616.0,811021.0,0
220931.0,876605.0,0
289719.0,866638.0,0
233931.0,838374.0,0
224694.0,848663.0,0
240645.0,855514.0,0
229567.0,855332.0,0
247613.0,815836.0,0
239387.0,847822.0,0
257788.0,880587.0,0
245532.0,846331.0,0
224241.0,828996.0,0
227582.0,856861.0,0
244618.0,845420.0,0
213775.0,841758.0,0
243511.0,830743.0,0
244799.0,841688.0,0
216061.0,848623.0,0
294492.0,832336.0,0
248931.0,844960.0,0
267478.0,894783.0,0
228715.0,837572.0,0
269483.0,848608.0,0
248276.0,844455.0,0
236987.0,881080.0,0
223732.0,826580.0,0
230149.0,886583.0,0
265299.0,859533.0,0
181803.0,799487.0,0
247210.0,859012.0,0
261633.0,773355.0,0
262370.0,891499.0,0
250394.0,847577.0,0
222019.0,839774.0,0
238328.0,770068.0,0
208941.0,860169.0,0
238714.0,858826.0,0
241005.0,825301.0,0
270742.0,847836.0,0
238492.0,833394.0,0
312726.0,853829.0,0
286663.0,846902.0,0
261796.0,819395.0,0
253890.0,807676.0,0
242415.0,844335.0,0
134134.0,842630.0,0
163956.0,833732.0,0
250359.0,849451.0,0
295357.0,842293.0,0
218942.0,848326.0,0
205466.0,778884.0,0
253899.0,862843.0,0
244966.0,841126.0,0
208494.0,850615.0,0
221126.0,837878.0,0
271093.0,861412.0,0
216543.0,842952.0,0
255185.0,825960.0,0
251070.0,877802.0,0
264565.0,859237.0,0
228611.0,836717.0,0
228517.0,837056.0,0
198583.0,860431.0,0
215976.0,836025.0,0
254411.0,848090.0,0
168036.0,807011.0,0
292194.0,906490.0,0
243951.0,865415.0,0
260922.0,785987.0,0
245740.0,814541.0,0
317675.0,838127.0,0
274505.0,864723.0,0
230049.0,865536.0,0
235084.0,818866.0,0
172690.0,847721.0,0
221706.0,875060.0,0
236740.0,853169.0,0
298800.0,824277.0,0
265246.0,889333.0,0
219498.0,851932.0,0
247484.0,846372.0,0
288675.0,919879.0,0
229280.0,813483.0,0
282622.0,879222.0,0
307300.0,836132.0,0
238260.0,882489.0,0
219049.0,825171.0,0
219321.0,848396.0,0
240028.0,849354.0,0
212009.0,864463.0,0
246857.0,852483.0,0
245118.0,811283.0,0
247855.0,794626.0,0
253655.0,821946.0,0
241475.0,850889.0,0
210162.0,860529.0,0
204922.0,854068.0,0
225567.0,853078.0,0
245258.0,799882.0,0
257134.0,845851.0,0
187314.0,815820.0,0
294275.0,174786.0,10
329428.0,66056.0,10
325938.0,164475.0,10
253129.0,154296.0,10
319838.0,170383.0,10
240213.0,118409.0,10
283784.0,178305.0,10
376613.0,159699.0,10
338717.0,181582.0,10
304283.0,183656.0,10
314519.0,146787.0,10
270899.0,175377.0,10
291228.0,101272.0,10
291170.0,110147.0,10
321522.0,163852.0,10
202627.0,179376.0,10
307913.0,91419.0,10
311093.0,169734.0,10
321775.0,164846.0,10
294832.0,124284.0,10
295537.0,91032.0,10
383044.0,180418.0,10
338570.0,192256.0,10
337894.0,175289.0,10
359796.0,165920.0,10
316263.0,163806.0,10
320972.0,170417.0,10
277860.0,111893.0,10
367806.0,123963.0,10
359352.0,162962.0,10
350964.0,149159.0,10
332201.0,181702.0,10
287966.0,150135.0,10
346942.0,152780.0,10
232784.0,169847.0,10
292492.0,99170.0,10
334529.0,166765.0,10
363208.0,238649.0,10
196795.0,190277.0,10
353333.0,113156.0,10
373191.0,88470.0,10
333789.0,151168.0,10
339732.0,163158.0,10
321822.0,187102.0,10
276792.0,141848.0,10
308044.0,164482.0,10
311328.0,159477.0,10
383052.0,89061.0,10
316820.0,150304.0,10
314479.0,178821.0,10
310824.0,144943.0,10
333143.0,200298.0,10
300672.0,175844.0,10
339482.0,151261.0,10
319208.0,164182.0,10
321645.0,164662.0,10
308763.0,166511.0,10
308066.0,145825.0,10
360143.0,155270.0,10
348893.0,195430.0,10
319283.0,158983.0,10
332284.0,194186.0,10
292402.0,184441.0,10
361227.0,221878.0,10
313573.0,109221.0,10
310693.0,154278.0,10
344482.0,215846.0,10
358652.0,186358.0,10
330533.0,214555.0,10
320400.0,160020.0,10
267662.0,127175.0,10
396583.0,82842.0,10
292193.0,125417.0,10
324121.0,96346.0,10
321344.0,166633.0,10
234837.0,151054.0,10
368060.0,76739.0,10
347050.0,150472.0,10
245783.0,109229.0,10
400550.0,144359.0,10
343217.0,153037.0,10
315488.0,160339.0,10
347406.0,155708.0,10
336845.0,172750.0,10
357014.0,148222.0,10
331862.0,169092.0,10
322140.0,165212.0,10
363075.0,162238.0,10
318738.0,182134.0,10
346717.0,178180.0,10
311819.0,140674.0,10
332032.0,157725.0,10
301983.0,173386.0,10
321556.0,163352.0,10
315830.0,102777.0,10
312275.0,145173.0,10
325861.0,203058.0,10
360703.0,150999.0,10
345976.0,148333.0,10
320210.0,191459.0,10
318665.0,158107.0,10
425563.0,163524.0,11
264427.0,188193.0,10
348177.0,179526.0,10
265430.0,151883.0,10
320718.0,158610.0,10
380609.0,185976.0,10
345234.0,188822.0,10
362027.0,150348.0,10
300294.0,178772.0,10
315963.0,174031.0,10
333787.0,189423.0,10
337343.0,156614.0,10
334812.0,137431.0,10
347492.0,153416.0,10
324518.0,167369.0,10
353991.0,193997.0,10
274321.0,219665.0,10
335476.0,183897.0,10
300021.0,159185.0,10
384591.0,137729.0,10
320000.0,162605.0,10
268938.0,211870.0,10
366146.0,196571.0,10
356280.0,185198.0,10
325767.0,164095.0,10
350480.0,214608.0,10
314791.0,163370.0,10
302582.0,134124.0,10
328851.0,147002.0,10
347039.0,195468.0,10
289531.0,145743.0,10
343018.0,239960.0,10
271123.0,163465.0,10
326959.0,154655.0,10
332307.0,189590.0,10
310529.0,141617.0,10
380281.0,205779.0,10
411606.0,137378.0,10
348191.0,165419.0,10
305292.0,175260.0,10
256120.0,141529.0,10
390934.0,170420.0,10
358562.0,112650.0,10
355499.0,224982.0,10
363437.0,77975.0,10
232043.0,177614.0,10
246753.0,175170.0,10
327204.0,152871.0,10
371685.0,151612.0,10
291723.0,177824.0,10
324488.0,180658.0,10
340048.0,131847.0,10
295311.0,115000.0,10
239495.0,163949.0,10
267521.0,119941.0,10
401396.0,125431.0,10
260930.0,184935.0,10
336085.0,170252.0,10
291782.0,159562.0,10
307616.0,207295.0,10
352910.0,145624.0,10
294653.0,104623.0,10
372883.0,222824.0,10
318707.0,160697.0,10
281548.0,175184.0,10
307747.0,128551.0,10
336991.0,217448.0,10
270233.0,195701.0,10
433100.0,134861.0,11
311433.0,134432.0,10
375353.0,117180.0,10
315146.0,160782.0,10
311163.0,166821.0,10
305333.0,171939.0,10
332470.0,161202.0,10
302266.0,185591.0,10
370775.0,144355.0,10
314552.0,151035.0,10
343145.0,172236.0,10
331212.0,162496.0,10
299460.0,144708.0,10
383894.0,179880.0,10
345703.0,149860.0,10
331381.0,162660.0,10
344364.0,213940.0,10
378347.0,144880.0,10
322789.0,137560.0,10
328425.0,159826.0,10
341137.0,78287.0,10
317771.0,154916.0,10
336266.0,148507.0,10
342066.0,119083.0,10
297177.0,88467.0,10
280519.0,165350.0,10
305901.0,166656.0,10
284757.0,175853.0,10
281732.0,221764.0,10
321262.0,178491.0,10
344710.0,213543.0,10
311408.0,168232.0,10
356500.0,188114.0,10
304376.0,222294.0,10
278116.0,185472.0,10
247892.0,115217.0,10
306326.0,164202.0,10
311516.0,94189.0,10
323747.0,163351.0,10
321778.0,158339.0,10
309501.0,166931.0,10
333942.0,143344.0,10
339393.0,209145.0,10
340996.0,170964.0,10
315333.0,144566.0,10
332853.0,168141.0,10
290958.0,168323.0,10
297731.0,175876.0,10
280274.0,182662.0,10
306540.0,193434.0,10
319155.0,159850.0,10
355168.0,114823.0,10
270201.0,236813.0,10
347630.0,120931.0,10
446538.0,180119.0,11
371052.0,147369.0,10
403365.0,215735.0,10
258201.0,153095.0,10
335121.0,214149.0,10
283786.0,133973.0,10
353951.0,133923.0,10
345069.0,165968.0,10
367079.0,183232.0,10
363932.0,228249.0,10
354132.0,161850.0,10
390538.0,157197.0,10
333805.0,148479.0,10
325344.0,159885.0,10
323905.0,168029.0,10
274495.0,182470.0,10
309684.0,150076.0,10
381617.0,157000.0,10
322841.0,169735.0,10
359520.0,132845.0,10
249616.0,144446.0,10
321620.0,169950.0,10
415495.0,116126.0,10
335742.0,196692.0,10
271424.0,251323.0,10
340066.0,152500.0,10
327734.0,163585.0,10
346079.0,220993.0,10
276741.0,143844.0,10
377042.0,182019.0,10
400055.0,131725.0,10
332246.0,172932.0,10
310386.0,171068.0,10
327569.0,158418.0,10
324444.0,166910.0,10
317217.0,163896.0,10
270404.0,183080.0,10
277512.0,151920.0,10
324966.0,170021.0,10
306421.0,145493.0,10
382419.0,110392.0,10
318266.0,171979.0,10
321245.0,141627.0,10
318192.0,170844.0,10
320821.0,165455.0,10
329029.0,204399.0,10
311065.0,219934.0,10
287127.0,192494.0,10
283571.0,109326.0,10
308748.0,199866.0,10
369490.0,164369.0,10
270729.0,67329.0,10
279010.0,181792.0,10
282957.0,179982.0,10
308489.0,145929.0,10
327927.0,168535.0,10
327022.0,161396.0,10
376557.0,109320.0,10
248427.0,146381.0,10
321182.0,150806.0,10
312337.0,154488.0,10
268518.0,181514.0,10
285865.0,167818.0,10
279251.0,184486.0,10
306042.0,66361.0,10
318215.0,180010.0,10
293978.0,197091.0,10
338044.0,233264.0,10
293864.0,241877.0,10
344380.0,183817.0,10
316257.0,171544.0,10
248779.0,111087.0,10
313783.0,177198.0,10
415534.0,110526.0,10
296164.0,96874.0,10
315670.0,160928.0,10
339137.0,160334.0,10
290700.0,196285.0,10
307107.0,174760.0,10
325279.0,168587.0,10
328423.0,161604.0,10
333883.0,276840.0,10
308555.0,134011.0,10
322101.0,164171.0,10
306357.0,123397.0,10
292931.0,125341.0,10
284378.0,223470.0,10
335003.0,151395.0,10
267212.0,158699.0,10
343851.0,193618.0,10
296627.0,93375.0,10
334980.0,143900.0,10
335235.0,180161.0,10
360945.0,167952.0,10
257841.0,193878.0,10
275288.0,212315.0,10
331798.0,143962.0,10
333937.0,51121.0,10
328775.0,156501.0,10
319829.0,168353.0,10
293201.0,192871.0,10
327608.0,177638.0,10
332562.0,174699.0,10
348815.0,150017.0,10
312227.0,159417.0,10
324337.0,162686.0,10
323329.0,93955.0,10
291973.0,127483.0,10
258779.0,106777.0,10
325360.0,134919.0,10
267264.0,153034.0,10
312421.0,170617.0,10
293350.0,175997.0,10
275459.0,190178.0,10
244344.0,185382.0,10
252845.0,181181.0,10
357923.0,192120.0,10
382720.0,263255.0,10
326040.0,156509.0,10
165104.0,543395.0,9
139903.0,592241.0,9
132705.0,557674.0,9
114311.0,601605.0,9
102460.0,533057.0,9
185935.0,554627.0,9
123441.0,594415.0,9
97216.0,564094.0,9
110948.0,539533.0,9
101553.0,517715.0,9
133023.0,574572.0,9
144570.0,573936.0,9
136531.0,557658.0,9
173576.0,507309.0,9
179189.0,536028.0,9
141238.0,557986.0,9
156430.0,615117.0,9
69169.0,631490.0,9
31842.0,565765.0,9
144620.0,575370.0,9
155899.0,561715.0,9
137451.0,555339.0,9
68388.0,574380.0,9
166290.0,521714.0,9
210499.0,526471.0,9
148319.0,584541.0,9
135336.0,552552.0,9
136606.0,562296.0,9
136301.0,569544.0,9
119386.0,620549.0,9
139212.0,557713.0,9
84501.0,604671.0,9
176602.0,574552.0,9
105945.0,528795.0,9
140989.0,537156.0,9
138449.0,555969.0,9
128669.0,574591.0,9
95749.0,602775.0,9
139460.0,557260.0,9
130440.0,562623.0,9
147885.0,527813.0,9
58190.0,608304.0,9
151701.0,574146.0,9
148385.0,538149.0,9
142668.0,544976.0,9
145251.0,553486.0,9
142265.0,546135.0,9
202913.0,525966.0,9
192564.0,547558.0,9
179240.0,577011.0,9
182840.0,584262.0,9
144242.0,557674.0,9
135849.0,550612.0,9
101427.0,615890.0,9
143265.0,548387.0,9
112379.0,547679.0,9
137347.0,622867.0,9
139146.0,569516.0,9
125756.0,579609.0,9
77109.0,512651.0,9
138073.0,556272.0,9
202547.0,601240.0,9
19835.0,570290.0,9
118137.0,579149.0,9
201324.0,574588.0,9
113259.0,568684.0,9
139052.0,610320.0,9
177246.0,571940.0,9
107972.0,601191.0,9
138199.0,541774.0,9
220156.0,589643.0,9
137787.0,566189.0,9
138122.0,557230.0,9
93136.0,613387.0,9
154620.0,576738.0,9
131523.0,568207.0,9
136967.0,565662.0,9
142886.0,556511.0,9
125005.0,540340.0,9
138965.0,558201.0,9
139474.0,569485.0,9
171471.0,576787.0,9
157138.0,565993.0,9
198294.0,552987.0,9
136078.0,547418.0,9
140164.0,531093.0,9
156003.0,502860.0,9
67867.0,545392.0,9
133780.0,533275.0,9
116142.0,550919.0,9
148510.0,579793.0,9
179224.0,584189.0,9
149906.0,546909.0,9
29760.0,557873.0,9
115287.0,602637.0,9
206561.0,574789.0,9
108651.0,521409.0,9
115905.0,560139.0,9
127373.0,549518.0,9
141699.0,530028.0,9
141641.0,557376.0,9
111855.0,531542.0,9
136580.0,553967.0,9
137516.0,556818.0,9
179793.0,483378.0,9
105568.0,584102.0,9
125840.0,520239.0,9
117212.0,565692.0,9
140717.0,554093.0,9
139033.0,593287.0,9
172882.0,571145.0,9
104862.0,608089.0,9
158366.0,537970.0,9
138525.0,534268.0,9
69565.0,599820.0,9
94571.0,528963.0,9
153185.0,562792.0,9
147599.0,513563.0,9
223226.0,546198.0,9
143879.0,536650.0,9
151099.0,518008.0,9
109317.0,531425.0,9
127202.0,525948.0,9
137740.0,618961.0,9
138734.0,557713.0,9
109912.0,565003.0,9
158060.0,525533.0,9
260096.0,530550.0,8
134449.0,557556.0,9
121650.0,599844.0,9
188992.0,563992.0,9
207218.0,589497.0,9
179830.0,547383.0,9
152331.0,529689.0,9
112184.0,530469.0,9
166904.0,503121.0,9
183026.0,545377.0,9
152658.0,569277.0,9
172116.0,513383.0,9
95899.0,530846.0,9
144325.0,538991.0,9
166597.0,572855.0,9
105193.0,596017.0,9
166840.0,562425.0,9
117089.0,561455.0,9
141718.0,549576.0,9
117959.0,536156.0,9
125351.0,529954.0,9
140706.0,547367.0,9
138012.0,558263.0,9
135284.0,555664.0,9
142663.0,556520.0,9
112673.0,516997.0,9
137657.0,611252.0,9
119237.0,591137.0,9
186380.0,487625.0,9
113099.0,546732.0,9
176278.0,550602.0,9
158773.0,588778.0,9
140962.0,518168.0,9
144856.0,551807.0,9
155943.0,576199.0,9
152584.0,564548.0,9
165356.0,570715.0,9
127858.0,609681.0,9
257984.0,555707.0,8
149758.0,564768.0,9
129550.0,560106.0,9
159271.0,568953.0,9
151374.0,538387.0,9
129030.0,588495.0,9
155430.0,549012.0,9
172756.0,612694.0,9
93459.0,514032.0,9
99569.0,523174.0,9
148080.0,544056.0,9
169322.0,577938.0,9
173847.0,514665.0,9
136418.0,562926.0,9
140047.0,540043.0,9
74519.0,527539.0,9
146681.0,562995.0,9
110877.0,562558.0,9
95071.0,503965.0,9
136232.0,576477.0,9
146120.0,576624.0,9
259173.0,548593.0,8
107965.0,535173.0,9
103153.0,601002.0,9
158283.0,570550.0,9
206219.0,604081.0,9
116492.0,509399.0,9
97430.0,598804.0,9
193275.0,538687.0,9
116860.0,550810.0,9
179834.0,541387.0,9
143226.0,522375.0,9
111174.0,539350.0,9
94933.0,588147.0,9
139434.0,557410.0,9
131532.0,536658.0,9
38179.0,532155.0,9
113321.0,606322.0,9
123905.0,508486.0,9
201753.0,569116.0,9
206660.0,575002.0,9
138884.0,529162.0,9
139636.0,565984.0,9
110479.0,594587.0,9
76253.0,529394.0,9
160280.0,493497.0,9
161475.0,587507.0,9
149054.0,556986.0,9
223408.0,520131.0,9
158596.0,562634.0,9
141991.0,539522.0,9
137172.0,523482.0,9
161154.0,538355.0,9
130157.0,581286.0,9
142000.0,554964.0,9
86243.0,559135.0,9
130883.0,557824.0,9
127964.0,538935.0,9
122130.0,568886.0,9
210597.0,589836.0,9
139847.0,552015.0,9
147664.0,559632.0,9
167716.0,562762.0,9
130657.0,555545.0,9
165382.0,518876.0,9
221994.0,575146.0,9
87236.0,607429.0,9
154268.0,542573.0,9
197367.0,535989.0,9
133094.0,557996.0,9
155274.0,552551.0,9
139911.0,561185.0,9
146924.0,565933.0,9
146183.0,622188.0,9
176092.0,531921.0,9
184402.0,546549.0,9
139641.0,557503.0,9
170816.0,547069.0,9
156474.0,570384.0,9
146678.0,558942.0,9
136569.0,532130.0,9
162124.0,500069.0,9
127503.0,623654.0,9
182854.0,534283.0,9
141222.0,572556.0,9
177665.0,557923.0,9
201907.0,516078.0,9
130249.0,579307.0,9
166560.0,559687.0,9
138220.0,560056.0,9
134725.0,537440.0,9
127940.0,590467.0,9
143592.0,606949.0,9
64979.0,538104.0,9
94061.0,576129.0,9
79285.0,554817.0,9
103192.0,550021.0,9
198823.0,538624.0,9
145758.0,562414.0,9
211584.0,591058.0,9
139451.0,549580.0,9
220224.0,592068.0,9
150404.0,539885.0,9
142767.0,538646.0,9
143912.0,595772.0,9
184412.0,537050.0,9
55359.0,623076.0,9
137197.0,556522.0,9
138461.0,558771.0,9
80673.0,566893.0,9
203667.0,577260.0,9
94249.0,600812.0,9
145244.0,561042.0,9
94458.0,533883.0,9
130201.0,562734.0,9
98675.0,580608.0,9
134422.0,544300.0,9
141771.0,551655.0,9
141924.0,558399.0,9
137316.0,562151.0,9
143363.0,551365.0,9
197849.0,575524.0,9
134261.0,519275.0,9
129997.0,568115.0,9
170494.0,560892.0,9
164453.0,544085.0,9
128788.0,546091.0,9
88703.0,577217.0,9
131109.0,568012.0,9
95074.0,570002.0,9
232047.0,522748.0,9
125548.0,552424.0,9
142563.0,560090.0,9
118391.0,553625.0,9
132497.0,593125.0,9
162713.0,586568.0,9
122296.0,566921.0,9
56907.0,586609.0,9
48143.0,523870.0,9
104489.0,542515.0,9
130582.0,565948.0,9
180633.0,570302.0,9
241485.0,523039.0,9
178005.0,594957.0,9
211266.0,534538.0,9
137680.0,555667.0,9
144401.0,565666.0,9
101722.0,535117.0,9
115781.0,594942.0,9
107160.0,530936.0,9
139978.0,587153.0,9
179931.0,536588.0,9
105353.0,589239.0,9
119496.0,543146.0,9
142575.0,517378.0,9
76661.0,531044.0,9
137699.0,556769.0,9
154964.0,554923.0,9
142516.0,559062.0,9
98525.0,579761.0,9
163925.0,576666.0,9
153326.0,558408.0,9
262846.0,559590.0,8
102898.0,592834.0,9
138349.0,554238.0,9
193502.0,557364.0,9
202175.0,495076.0,9
154162.0,536416.0,9
77875.0,516899.0,9
163477.0,561507.0,9
131874.0,552586.0,9
120357.0,560737.0,9
119535.0,551857.0,9
114108.0,540537.0,9
136059.0,558213.0,9
121786.0,564225.0,9
233105.0,591646.0,9
149760.0,577278.0,9
139237.0,548782.0,9
34795.0,606311.0,9
147663.0,555514.0,9
108512.0,525986.0,9
509279.0,180548.0,11
495165.0,163113.0,11
474652.0,193224.0,11
507786.0,199578.0,11
566666.0,195255.0,11
506012.0,178372.0,11
541435.0,206637.0,11
507311.0,177795.0,11
461701.0,188438.0,11
506069.0,169723.0,11
477155.0,161019.0,11
505044.0,180646.0,11
467697.0,168082.0,11
511868.0,170991.0,11
500225.0,177559.0,11
460616.0,174810.0,11
573755.0,218049.0,11
543616.0,144788.0,11
509804.0,175714.0,11
542311.0,161882.0,11
475944.0,189337.0,11
517750.0,199025.0,11
520952.0,169184.0,11
559076.0,190770.0,11
416763.0,96639.0,10
549384.0,208450.0,11
549147.0,171454.0,11
520880.0,200955.0,11
484348.0,202813.0,11
487472.0,166999.0,11
504728.0,174541.0,11
465734.0,114090.0,11
570266.0,188032.0,11
505584.0,176135.0,11
514739.0,189624.0,11
500589.0,181466.0,11
504096.0,192493.0,11
553118.0,165472.0,11
502511.0,205121.0,11
551798.0,221411.0,11
502737.0,174437.0,11
513181.0,177419.0,11
513010.0,141058.0,11
535564.0,177248.0,11
514551.0,176526.0,11
450030.0,169537.0,11
521152.0,152439.0,11
505512.0,203857.0,11
509928.0,184873.0,11
490271.0,216725.0,11
529931.0,180136.0,11
508228.0,220433.0,11
534461.0,178056.0,11
508551.0,174306.0,11
545365.0,210090.0,11
510414.0,142485.0,11
507895.0,177445.0,11
506288.0,177990.0,11
512560.0,224816.0,11
454160.0,102568.0,11
437116.0,148591.0,11
519003.0,160741.0,11
509173.0,178051.0,11
507674.0,147431.0,11
513562.0,229098.0,11
473361.0,185066.0,11
517580.0,170329.0,11
509285.0,174620.0,11
510496.0,174308.0,11
438860.0,114065.0,11
534702.0,244239.0,11
504755.0,150226.0,11
474135.0,199330.0,11
508907.0,173787.0,11
509651.0,149290.0,11
490956.0,124550.0,11
499637.0,194036.0,11
515856.0,148316.0,11
482568.0,159010.0,11
522728.0,184704.0,11
493070.0,189771.0,11
483807.0,198925.0,11
469656.0,153741.0,11
470041.0,102049.0,11
576965.0,187481.0,11
515230.0,149567.0,11
469470.0,189347.0,11
484678.0,123623.0,11
509554.0,186931.0,11
480787.0,147737.0,11
523036.0,123677.0,11
508144.0,167733.0,11
515450.0,173220.0,11
489378.0,172861.0,11
497406.0,125809.0,11
514173.0,139330.0,11
520809.0,176718.0,11
499009.0,205823.0,11
509773.0,148899.0,11
505117.0,141877.0,11
494698.0,176711.0,11
501334.0,163002.0,11
502500.0,176532.0,11
546565.0,151576.0,11
520878.0,209094.0,11
486891.0,175164.0,11
512241.0,169240.0,11
569692.0,182365.0,11
491382.0,122267.0,11
457033.0,127590.0,11
535453.0,200970.0,11
436641.0,55171.0,11
520845.0,176291.0,11
496130.0,178113.0,11
507473.0,227670.0,11
507092.0,175151.0,11
493714.0,173387.0,11
528826.0,154067.0,11
509951.0,176794.0,11
505400.0,160422.0,11
473316.0,162622.0,11
489504.0,146224.0,11
516888.0,152451.0,11
505128.0,175022.0,11
520819.0,181586.0,11
506812.0,167031.0,11
495032.0,203929.0,11
508789.0,174256.0,11
502302.0,210265.0,11
511106.0,122518.0,11
486997.0,164803.0,11
535787.0,199735.0,11
540364.0,163758.0,11
496006.0,191010.0,11
497454.0,172375.0,11
488330.0,173837.0,11
547076.0,156969.0,11
509956.0,141698.0,11
516084.0,178725.0,11
535034.0,172649.0,11
529858.0,213114.0,11
496384.0,166644.0,11
519201.0,139101.0,11
575022.0,213534.0,11
469404.0,181944.0,11
464672.0,193513.0,11
491609.0,180519.0,11
470314.0,178936.0,11
481067.0,172555.0,11
508797.0,176269.0,11
491145.0,206344.0,11
485361.0,169229.0,11
474527.0,107596.0,11
499643.0,171625.0,11
514722.0,190943.0,11
474641.0,81263.0,11
495995.0,160032.0,11
516963.0,174184.0,11
521006.0,163218.0,11
434581.0,123267.0,11
514876.0,182995.0,11
501416.0,172980.0,11
513991.0,165072.0,11
512900.0,219252.0,11
510739.0,121103.0,11
509522.0,174376.0,11
507862.0,172954.0,11
512716.0,181316.0,11
483423.0,111797.0,11
505603.0,147517.0,11
482628.0,171320.0,11
541790.0,172616.0,11
473649.0,178184.0,11
520763.0,250478.0,11
494000.0,189403.0,11
533931.0,173291.0,11
485110.0,150541.0,11
528679.0,199079.0,11
545699.0,174099.0,11
517495.0,186329.0,11
520461.0,173217.0,11
456821.0,105575.0,11
495388.0,156422.0,11
514934.0,219169.0,11
508989.0,168995.0,11
500474.0,169746.0,11
504511.0,149234.0,11
467144.0,147308.0,11
512733.0,174091.0,11
452791.0,156805.0,11
488800.0,197919.0,11
593887.0,210595.0,11
561725.0,168220.0,11
528011.0,201615.0,11
514428.0,222503.0,11
512051.0,145220.0,11
472054.0,129986.0,11
507959.0,166780.0,11
518068.0,137556.0,11
561110.0,219865.0,11
563315.0,215613.0,11
523779.0,163905.0,11
490037.0,164357.0,11
519421.0,154681.0,11
504373.0,165819.0,11
513957.0,139202.0,11
486422.0,178919.0,11
533351.0,147778.0,11
511030.0,221292.0,11
496069.0,102201.0,11
496491.0,171534.0,11
497254.0,171940.0,11
518892.0,165875.0,11
481083.0,151953.0,11
507741.0,178883.0,11
486224.0,196184.0,11
509039.0,173839.0,11
462658.0,75167.0,11
519680.0,208378.0,11
499676.0,196897.0,11
480876.0,160465.0,11
521032.0,177084.0,11
486034.0,196122.0,11
505225.0,158750.0,11
485968.0,123023.0,11
478442.0,125560.0,11
502355.0,164119.0,11
502915.0,183059.0,11
497211.0,174121.0,11
537174.0,194410.0,11
505293.0,194591.0,11
491442.0,116195.0,11
473202.0,140745.0,11
505079.0,219082.0,11
541455.0,223176.0,11
528938.0,174372.0,11
528094.0,178343.0,11
520759.0,184923.0,11
538994.0,179328.0,11
507832.0,181513.0,11
493897.0,189627.0,11
514408.0,173338.0,11
514391.0,156592.0,11
513362.0,223841.0,11
443618.0,157237.0,11
477746.0,163845.0,11
480571.0,204150.0,11
486326.0,174643.0,11
505384.0,174535.0,11
541220.0,177840.0,11
522869.0,176818.0,11
497255.0,188783.0,11
545240.0,221008.0,11
512291.0,169608.0,11
576158.0,210283.0,11
508099.0,175066.0,11
545455.0,182232.0,11
537836.0,177165.0,11
508597.0,173640.0,11
507445.0,193647.0,11
524397.0,239456.0,11
527753.0,213985.0,11
528766.0,239937.0,11
481356.0,162127.0,11
541498.0,192746.0,11
506142.0,170513.0,11
494574.0,198972.0,11
477901.0,173288.0,11
509223.0,177592.0,11
532010.0,176195.0,11
541006.0,175198.0,11
490355.0,153464.0,11
566524.0,179796.0,11
550216.0,184337.0,11
516085.0,195504.0,11
523796.0,219588.0,11
511118.0,175212.0,11
445043.0,152257.0,11
497582.0,190801.0,11
514622.0,206979.0,11
479631.0,178459.0,11
496476.0,140337.0,11
498389.0,178069.0,11
506361.0,225237.0,11
519396.0,177174.0,11
529409.0,157228.0,11
501111.0,157402.0,11
514219.0,167014.0,11
558307.0,203307.0,11
545957.0,196749.0,11
508080.0,175697.0,11
589900.0,217899.0,11
509383.0,167302.0,11
474516.0,189144.0,11
498730.0,183037.0,11
473093.0,188791.0,11
471328.0,176297.0,11
510901.0,175806.0,11
559809.0,162597.0,11
499272.0,168413.0,11
514981.0,172608.0,11
511289.0,183247.0,11
455460.0,153040.0,11
519962.0,175210.0,11
532549.0,172894.0,11
516128.0,178722.0,11
519113.0,194925.0,11
502877.0,192740.0,11
520782.0,134308.0,11
574369.0,207483.0,11
515494.0,188459.0,11
486870.0,146947.0,11
484395.0,107291.0,11
504243.0,237336.0,11
483776.0,213021.0,11
536623.0,213083.0,11
544539.0,180391.0,11
502002.0,166223.0,11
500801.0,185558.0,11
525670.0,184762.0,11
481841.0,209337.0,11
494317.0,170837.0,11
521153.0,242968.0,11
473933.0,191767.0,11
524502.0,248106.0,11
487952.0,168607.0,11
460133.0,173081.0,11
453261.0,148432.0,11
491792.0,214464.0,11
497815.0,165151.0,11
515348.0,184750.0,11
598908.0,254032.0,11
558478.0,213316.0,11
538228.0,168834.0,11
464634.0,173867.0,11
518675.0,175686.0,11
533566.0,214619.0,11
540196.0,180312.0,11
500360.0,213540.0,11
517882.0,175467.0,11
549642.0,157506.0,11
511368.0,173673.0,11
506815.0,173387.0,11
500192.0,192292.0,11
533775.0,163036.0,11
511484.0,160453.0,11
489062.0,200329.0,11
537514.0,179689.0,11
551867.0,170425.0,11
406184.0,394506.0,12
417996.0,408162.0,12
428667.0,451233.0,12
425386.0,379634.0,12
388202.0,405780.0,12
370955.0,325071.0,12
353749.0,391896.0,12
398629.0,401626.0,12
393402.0,401018.0,12
465599.0,383470.0,12
400257.0,404936.0,12
444610.0,400247.0,12
392046.0,391220.0,12
410506.0,395586.0,12
508805.0,429073.0,12
424782.0,449687.0,12
384467.0,407359.0,12
397399.0,403186.0,12
449683.0,388863.0,12
387756.0,435588.0,12
451753.0,383440.0,12
414448.0,429790.0,12
394599.0,409068.0,12
387330.0,409037.0,12
402109.0,430185.0,12
393825.0,453067.0,12
412111.0,418466.0,12
397213.0,377839.0,12
361159.0,436327.0,12
383075.0,447616.0,12
383832.0,340506.0,12
402227.0,414685.0,12
411957.0,397796.0,12
377408.0,395123.0,12
417231.0,381303.0,12
442915.0,420573.0,12
415394.0,396940.0,12
420792.0,430901.0,12
397791.0,444527.0,12
385804.0,393101.0,12
384572.0,388523.0,12
402069.0,434868.0,12
414997.0,432597.0,12
392919.0,405103.0,12
396150.0,430329.0,12
439145.0,407748.0,12
401240.0,354085.0,12
424228.0,442208.0,12
336023.0,416862.0,12
351352.0,362910.0,12
413549.0,408517.0,12
341775.0,425156.0,12
424811.0,413300.0,12
402677.0,412615.0,12
419118.0,421655.0,12
354601.0,358760.0,12
410682.0,349328.0,12
399020.0,404042.0,12
392258.0,405478.0,12
353941.0,399967.0,12
411471.0,446824.0,12
397158.0,370362.0,12
431117.0,394746.0,12
395246.0,392783.0,12
400468.0,405703.0,12
354094.0,424230.0,12
352429.0,359750.0,12
358830.0,431609.0,12
387039.0,365758.0,12
395013.0,387179.0,12
400545.0,404212.0,12
435067.0,403802.0,12
394436.0,450912.0,12
376444.0,371957.0,12
433463.0,410297.0,12
426625.0,475977.0,12
424241.0,452915.0,12
396520.0,405610.0,12
347638.0,429109.0,12
374797.0,407601.0,12
451670.0,388347.0,12
373684.0,418326.0,12
412615.0,403902.0,12
392428.0,402295.0,12
389868.0,410718.0,12
383022.0,409854.0,12
468350.0,395883.0,12
373339.0,398565.0,12
383878.0,376230.0,12
434815.0,388041.0,12
368864.0,411216.0,12
321050.0,422497.0,12
276794.0,384458.0,12
383197.0,409869.0,12
381340.0,409680.0,12
415521.0,368125.0,12
404288.0,349685.0,12
367922.0,415238.0,12
422056.0,382193.0,12
407334.0,445969.0,12
373917.0,427628.0,12
422261.0,374345.0,12
392728.0,411341.0,12
336585.0,420977.0,12
401730.0,354294.0,12
401854.0,402867.0,12
401330.0,395538.0,12
393903.0,387227.0,12
399188.0,402459.0,12
389510.0,370897.0,12
399858.0,403574.0,12
343651.0,438750.0,12
388743.0,388465.0,12
398246.0,406245.0,12
400727.0,396769.0,12
399077.0,403968.0,12
402071.0,427626.0,12
405902.0,394724.0,12
383992.0,398245.0,12
372230.0,412488.0,12
418086.0,394699.0,12
400305.0,466024.0,12
380458.0,329635.0,12
338347.0,337961.0,12
411171.0,426853.0,12
392447.0,408539.0,12
375233.0,399429.0,12
385338.0,440797.0,12
397764.0,406712.0,12
414290.0,393146.0,12
394846.0,407130.0,12
402252.0,414628.0,12
332913.0,385678.0,12
396517.0,394572.0,12
388479.0,343648.0,12
405569.0,411933.0,12
444816.0,409502.0,12
393659.0,439272.0,12
415222.0,434798.0,12
396621.0,381952.0,12
416893.0,347737.0,12
398980.0,407667.0,12
404273.0,429948.0,12
397851.0,400846.0,12
447192.0,359893.0,12
350428.0,392503.0,12
451853.0,420873.0,12
402566.0,415798.0,12
362753.0,369848.0,12
407188.0,396576.0,12
380139.0,434251.0,12
409167.0,449719.0,12
405470.0,392374.0,12
359036.0,365793.0,12
398617.0,402424.0,12
408714.0,391412.0,12
382222.0,441817.0,12
403751.0,385539.0,12
400548.0,402390.0,12
393828.0,404314.0,12
373943.0,376926.0,12
390182.0,360941.0,12
391776.0,409868.0,12
342501.0,336320.0,12
401923.0,417123.0,12
342400.0,426090.0,12
405325.0,407466.0,12
369480.0,377148.0,12
416110.0,374296.0,12
401162.0,412142.0,12
442931.0,430123.0,12
478361.0,409520.0,12
401313.0,405195.0,12
357466.0,354094.0,12
417617.0,414816.0,12
417281.0,410881.0,12
470047.0,410943.0,12
378824.0,430343.0,12
369227.0,397853.0,12
397918.0,462032.0,12
397331.0,407010.0,12
427810.0,400178.0,12
401708.0,409924.0,12
426172.0,363147.0,12
401411.0,414503.0,12
409823.0,412908.0,12
386068.0,422517.0,12
398255.0,405428.0,12
346358.0,384801.0,12
397735.0,448097.0,12
348847.0,425164.0,12
394761.0,461494.0,12
357253.0,388075.0,12
402267.0,410395.0,12
415233.0,441297.0,12
404445.0,402466.0,12
408224.0,399767.0,12
369044.0,371649.0,12
343119.0,382074.0,12
416750.0,435776.0,12
387103.0,401925.0,12
368200.0,364844.0,12
393758.0,439738.0,12
418623.0,404382.0,12
402660.0,406054.0,12
427749.0,457110.0,12
405356.0,391545.0,12
432184.0,395118.0,12
322936.0,368566.0,12
402710.0,424367.0,12
456025.0,389630.0,12
395450.0,407822.0,12
377941.0,403683.0,12
407576.0,434017.0,12
348263.0,386451.0,12
391966.0,387214.0,12
392576.0,406460.0,12
380236.0,460907.0,12
398153.0,407081.0,12
399942.0,385908.0,12
398795.0,472619.0,12
467535.0,416014.0,12
377758.0,413906.0,12
398724.0,404179.0,12
396710.0,378698.0,12
377588.0,402670.0,12
408992.0,462226.0,12
423371.0,373970.0,12
412861.0,434751.0,12
403160.0,392159.0,12
395168.0,404189.0,12
399346.0,423786.0,12
409244.0,460295.0,12
429515.0,393708.0,12
370602.0,353215.0,12
356739.0,417826.0,12
408147.0,436628.0,12
436362.0,379677.0,12
399291.0,386712.0,12
396081.0,396707.0,12
419200.0,403389.0,12
383855.0,390984.0,12
370252.0,347498.0,12
460198.0,402706.0,12
418046.0,358461.0,12
408500.0,393942.0,12
397618.0,404912.0,12
392660.0,395873.0,12
406739.0,400918.0,12
439048.0,403700.0,12
392424.0,375266.0,12
379128.0,418858.0,12
391970.0,419802.0,12
389279.0,424006.0,12
468907.0,448885.0,12
410601.0,410653.0,12
396556.0,406492.0,12
417795.0,405673.0,12
375971.0,427549.0,12
414374.0,439977.0,12
368363.0,409733.0,12
402081.0,407514.0,12
399753.0,404222.0,12
424840.0,479570.0,12
381863.0,416651.0,12
394606.0,401451.0,12
389520.0,406500.0,12
418140.0,415328.0,12
387461.0,394402.0,12
391384.0,413273.0,12
391439.0,399819.0,12
365223.0,422531.0,12
407189.0,437102.0,12
415933.0,422485.0,12
402031.0,426039.0,12
428513.0,456990.0,12
382396.0,405483.0,12
368166.0,429422.0,12
418636.0,440000.0,12
411674.0,402377.0,12
424365.0,371987.0,12
394159.0,411676.0,12
371548.0,414612.0,12
460628.0,434107.0,12
399717.0,403257.0,12
361875.0,448199.0,12
393424.0,399084.0,12
333682.0,401078.0,12
377870.0,377550.0,12
405117.0,364266.0,12
452398.0,378671.0,12
397889.0,398501.0,12
424089.0,352640.0,12
417494.0,392231.0,12
402851.0,395579.0,12
376493.0,366302.0,12
460684.0,371809.0,12
449571.0,387141.0,12
405322.0,414743.0,12
420489.0,448078.0,12
377975.0,461076.0,12
492551.0,434649.0,12
477323.0,384551.0,12
359740.0,379592.0,12
429653.0,438596.0,12
361814.0,411086.0,12
400660.0,404811.0,12
410418.0,409414.0,12
392162.0,355397.0,12
376145.0,414107.0,12
371464.0,328177.0,12
379593.0,380662.0,12
448231.0,384406.0,12
404157.0,406324.0,12
420616.0,401526.0,12
398985.0,413427.0,12
400538.0,407020.0,12
391530.0,398743.0,12
408182.0,408351.0,12
435926.0,408504.0,12
411090.0,435389.0,12
398311.0,383357.0,12
409291.0,431539.0,12
399460.0,399410.0,12
400473.0,408327.0,12
399693.0,395271.0,12
431437.0,436764.0,12
312123.0,419763.0,12
411053.0,432808.0,12
401616.0,404245.0,12
364455.0,436191.0,12
407833.0,402474.0,12
367864.0,442851.0,12
437870.0,375761.0,12
308726.0,371721.0,12
418996.0,389541.0,12
371498.0,393120.0,12
407429.0,399155.0,12
380749.0,403860.0,12
399997.0,411205.0,12
377215.0,418429.0,12
380202.0,396671.0,12
436034.0,385951.0,12
395210.0,357805.0,12
415136.0,446807.0,12
315013.0,318367.0,12
372424.0,413163.0,12
340966.0,420480.0,12
354767.0,417464.0,12
387165.0,404027.0,12
854654.0,549222.0,6
889260.0,541423.0,6
870826.0,524625.0,6
868025.0,569296.0,6
861364.0,598127.0,6
825141.0,535997.0,6
906640.0,587296.0,6
906396.0,535803.0,6
887896.0,558572.0,6
817776.0,525600.0,6
844066.0,546029.0,6
848976.0,558706.0,6
897131.0,604582.0,6
845120.0,549830.0,6
872241.0,556733.0,6
857818.0,534525.0,6
794265.0,594573.0,6
857126.0,548151.0,6
842786.0,614034.0,6
889975.0,589361.0,6
841130.0,529745.0,6
813561.0,583156.0,6
855603.0,542414.0,6
887556.0,531797.0,6
837032.0,509264.0,6
852037.0,649724.0,5
793791.0,581315.0,6
904450.0,533427.0,6
879722.0,550246.0,6
839088.0,528460.0,6
869369.0,536868.0,6
871873.0,509094.0,6
900991.0,497149.0,6
919544.0,541255.0,6
860520.0,544023.0,6
863042.0,557890.0,6
889805.0,522039.0,6
879031.0,489881.0,6
852918.0,552112.0,6
903840.0,586229.0,6
932662.0,537069.0,6
908506.0,549611.0,6
867533.0,542290.0,6
844617.0,499013.0,6
862002.0,534006.0,6
883981.0,607581.0,6
864265.0,549505.0,6
883062.0,547498.0,6
860870.0,546516.0,6
895565.0,501871.0,6
843344.0,604964.0,6
864059.0,545894.0,6
830888.0,544022.0,6
858819.0,544904.0,6
853562.0,531620.0,6
873791.0,606643.0,6
859213.0,538040.0,6
841843.0,503359.0,6
842719.0,506205.0,6
847904.0,574114.0,6
863859.0,546898.0,6
888919.0,535805.0,6
855045.0,547633.0,6
863102.0,545783.0,6
829747.0,530312.0,6
850607.0,565348.0,6
810277.0,520837.0,6
879104.0,595389.0,6
840406.0,538772.0,6
838367.0,536676.0,6
837457.0,518745.0,6
861519.0,531277.0,6
816056.0,542574.0,6
862534.0,547936.0,6
808555.0,600511.0,6
862571.0,550965.0,6
832010.0,554080.0,6
882403.0,534710.0,6
840020.0,568424.0,6
865030.0,541150.0,6
846594.0,522941.0,6
890223.0,556874.0,6
873136.0,513309.0,6
846622.0,553552.0,6
905770.0,531812.0,6
803216.0,577565.0,6
914183.0,620056.0,6
877510.0,555087.0,6
861193.0,573954.0,6
840994.0,578455.0,6
875640.0,561262.0,6
876782.0,536438.0,6
849501.0,552769.0,6
866973.0,550375.0,6
906382.0,534459.0,6
822722.0,553764.0,6
861656.0,545794.0,6
856991.0,546255.0,6
865081.0,551012.0,6
895680.0,609341.0,6
867801.0,548156.0,6
846324.0,557586.0,6
840059.0,534876.0,6
860473.0,545982.0,6
937823.0,540173.0,6
886532.0,500026.0,6
904133.0,534915.0,6
880545.0,506175.0,6
833499.0,504718.0,6
854979.0,539793.0,6
818739.0,545052.0,6
825596.0,639465.0,5
838036.0,542999.0,6
846216.0,528037.0,6
850790.0,525740.0,6
864040.0,538985.0,6
869818.0,543227.0,6
879309.0,561548.0,6
860563.0,534839.0,6
846082.0,541026.0,6
860151.0,543055.0,6
847292.0,559228.0,6
838239.0,576018.0,6
837307.0,548214.0,6
853093.0,586321.0,6
884326.0,479802.0,6
838978.0,622330.0,6
842903.0,567914.0,6
832490.0,450680.0,6
844948.0,563321.0,6
859577.0,544625.0,6
908409.0,591215.0,6
819843.0,570485.0,6
875625.0,599766.0,6
887780.0,536497.0,6
884613.0,507507.0,6
849858.0,524682.0,6
846741.0,539481.0,6
870630.0,516603.0,6
843111.0,581603.0,6
869718.0,536928.0,6
857648.0,511640.0,6
881571.0,532467.0,6
860230.0,528054.0,6
861966.0,562885.0,6
898736.0,548642.0,6
961951.0,577029.0,6
866878.0,501919.0,6
874791.0,559085.0,6
874239.0,594786.0,6
915504.0,527751.0,6
758172.0,608152.0,6
859515.0,505303.0,6
861033.0,507671.0,6
837880.0,444574.0,6
827672.0,545966.0,6
869960.0,562146.0,6
858151.0,548242.0,6
877440.0,529137.0,6
821835.0,564313.0,6
846371.0,578530.0,6
829482.0,475863.0,6
860507.0,526669.0,6
878696.0,530134.0,6
837312.0,572687.0,6
886773.0,509047.0,6
879805.0,547494.0,6
859679.0,541785.0,6
910683.0,612854.0,6
860863.0,546060.0,6
874110.0,541972.0,6
790149.0,565157.0,6
873984.0,580836.0,6
873742.0,448214.0,6
894785.0,440382.0,6
874312.0,574582.0,6
879902.0,518647.0,6
851462.0,526597.0,6
839012.0,550376.0,6
808097.0,502521.0,6
879500.0,590791.0,6
864721.0,548447.0,6
866044.0,584750.0,6
876151.0,582116.0,6
819250.0,517655.0,6
807278.0,566392.0,6
872033.0,558924.0,6
894268.0,532597.0,6
846752.0,613846.0,6
850478.0,566230.0,6
849777.0,531138.0,6
823584.0,568458.0,6
823202.0,619103.0,6
843715.0,568319.0,6
861023.0,545953.0,6
889743.0,560547.0,6
862785.0,579661.0,6
880833.0,537039.0,6
854374.0,550707.0,6
842616.0,515205.0,6
860721.0,547751.0,6
831024.0,505776.0,6
829240.0,591428.0,6
860384.0,555572.0,6
853348.0,525893.0,6
849118.0,571562.0,6
856167.0,542249.0,6
853874.0,560250.0,6
838808.0,524944.0,6
814561.0,575281.0,6
773292.0,531652.0,6
891515.0,520429.0,6
805884.0,549492.0,6
833437.0,556713.0,6
843432.0,490725.0,6
791355.0,529644.0,6
865157.0,562132.0,6
920249.0,551171.0,6
808777.0,512952.0,6
898916.0,630099.0,6
863706.0,491401.0,6
857231.0,551055.0,6
877413.0,541536.0,6
838821.0,563842.0,6
854037.0,528890.0,6
824475.0,518063.0,6
857360.0,562228.0,6
856437.0,532452.0,6
862117.0,600081.0,6
839280.0,539324.0,6
885379.0,603686.0,6
859185.0,553000.0,6
865020.0,550686.0,6
882007.0,593653.0,6
861027.0,546004.0,6
846772.0,509036.0,6
844627.0,556401.0,6
861069.0,548327.0,6
854342.0,541611.0,6
844086.0,531300.0,6
923996.0,594948.0,6
832154.0,559403.0,6
803202.0,549001.0,6
856619.0,544008.0,6
897286.0,558004.0,6
851216.0,516166.0,6
858537.0,548576.0,6
828001.0,524067.0,6
849327.0,548496.0,6
858278.0,544519.0,6
793875.0,473793.0,6
766617.0,548142.0,6
847728.0,576707.0,6
858418.0,546935.0,6
816840.0,539239.0,6
852687.0,540692.0,6
861927.0,562723.0,6
892382.0,513991.0,6
811745.0,461099.0,6
800152.0,576217.0,6
870139.0,566584.0,6
859492.0,572224.0,6
830226.0,483466.0,6
850674.0,533083.0,6
834632.0,541216.0,6
899052.0,526048.0,6
925485.0,596955.0,6
893145.0,551400.0,6
839493.0,521150.0,6
883434.0,581923.0,6
873380.0,495161.0,6
765011.0,607246.0,6
898578.0,612291.0,6
906772.0,510158.0,6
858325.0,586062.0,6
795909.0,462864.0,6
812634.0,554848.0,6
846841.0,533595.0,6
810817.0,509540.0,6
826192.0,577784.0,6
856552.0,545271.0,6
848628.0,503773.0,6
784291.0,581417.0,6
890613.0,567037.0,6
870936.0,575427.0,6
851166.0,532382.0,6
871544.0,592403.0,6
870337.0,534303.0,6
819580.0,545900.0,6
912649.0,498734.0,6
870214.0,556246.0,6
853279.0,560714.0,6
857319.0,543529.0,6
878519.0,606677.0,6
858057.0,542933.0,6
860375.0,544804.0,6
888111.0,525539.0,6
892877.0,544283.0,6
788776.0,582930.0,6
875311.0,542259.0,6
875248.0,462524.0,6
835831.0,555864.0,6
885828.0,586348.0,6
916704.0,609548.0,6
893991.0,544467.0,6
866352.0,535933.0,6
872247.0,494281.0,6
832372.0,600070.0,6
902302.0,514170.0,6
877298.0,565604.0,6
873636.0,510687.0,6
854818.0,554389.0,6
915666.0,506439.0,6
905849.0,524809.0,6
862062.0,545736.0,6
871981.0,649086.0,5
900829.0,590250.0,6
859419.0,546754.0,6
855983.0,534803.0,6
916082.0,517633.0,6
874877.0,535143.0,6
885963.0,528959.0,6
853439.0,518568.0,6
855177.0,626376.0,6
814249.0,488483.0,6
857283.0,548467.0,6
865489.0,548920.0,6
913686.0,576892.0,6
880275.0,564545.0,6
861381.0,546835.0,6
880958.0,588608.0,6
868979.0,528502.0,6
872237.0,555834.0,6
861868.0,549168.0,6
836951.0,546244.0,6
836874.0,555991.0,6
769546.0,554722.0,6
855074.0,542557.0,6
868128.0,568529.0,6
878859.0,540155.0,6
852267.0,498220.0,6
874125.0,543036.0,6
846646.0,548548.0,6
851517.0,541076.0,6
806888.0,492319.0,6
831779.0,577161.0,6
860259.0,545444.0,6
892423.0,530999.0,6
838442.0,506780.0,6
853291.0,557685.0,6
730887.0,825972.0,4
645233.0,927201.0,4
645563.0,819480.0,4
573324.0,878802.0,4
693699.0,822637.0,4
649412.0,856993.0,4
684558.0,878573.0,4
656658.0,854609.0,4
651506.0,856097.0,4
618405.0,842859.0,4
667280.0,765619.0,4
600996.0,890782.0,4
674463.0,862486.0,4
676694.0,851597.0,4
620685.0,880813.0,4
716590.0,874975.0,4
721401.0,951880.0,4
657094.0,857640.0,4
644693.0,867371.0,4
633601.0,861482.0,4
673646.0,857346.0,4
657184.0,855509.0,4
708244.0,871834.0,4
659081.0,860560.0,4
657811.0,885904.0,4
642843.0,909185.0,4
709303.0,897538.0,4
610505.0,851476.0,4
612648.0,772280.0,4
673196.0,812941.0,4
673267.0,883727.0,4
694451.0,863981.0,4
678869.0,902492.0,4
690018.0,819280.0,4
677983.0,890077.0,4
639997.0,865965.0,4
709254.0,847444.0,4
679799.0,858680.0,4
664355.0,831472.0,4
659166.0,861665.0,4
692386.0,863386.0,4
688076.0,833905.0,4
680506.0,840053.0,4
641840.0,886745.0,4
673292.0,857105.0,4
622212.0,860015.0,4
736557.0,897961.0,4
668234.0,886696.0,4
675013.0,876331.0,4
675902.0,853961.0,4
695092.0,820091.0,4
730287.0,897070.0,4
691716.0,873426.0,4
722404.0,952207.0,4
678606.0,878004.0,4
664156.0,846474.0,4
688118.0,911574.0,4
674492.0,861230.0,4
675382.0,836312.0,4
664639.0,925795.0,4
626334.0,852383.0,4
669690.0,842110.0,4
653177.0,819935.0,4
674265.0,861435.0,4
677404.0,883871.0,4
672212.0,860894.0,4
715956.0,900724.0,4
674236.0,877152.0,4
679818.0,860064.0,4
578509.0,923881.0,4
680381.0,823757.0,4
720629.0,816515.0,4
640210.0,902029.0,4
671772.0,860357.0,4
618009.0,867493.0,4
672628.0,882986.0,4
680261.0,863401.0,4
682100.0,853708.0,4
681049.0,846975.0,4
668567.0,856554.0,4
656957.0,871011.0,4
662293.0,872590.0,4
673953.0,835926.0,4
718544.0,888401.0,4
648590.0,917839.0,4
699124.0,824089.0,4
668367.0,855686.0,4
675866.0,868784.0,4
700098.0,894685.0,4
683156.0,877209.0,4
661357.0,801817.0,4
789095.0,895634.0,4
673332.0,857517.0,4
627452.0,835854.0,4
672059.0,860501.0,4
665048.0,854911.0,4
683580.0,872374.0,4
668839.0,882598.0,4
682510.0,905902.0,4
671235.0,856653.0,4
680228.0,843225.0,4
662249.0,918050.0,4
674961.0,865594.0,4
679211.0,886670.0,4
676220.0,858214.0,4
567861.0,856582.0,4
668228.0,858122.0,4
682241.0,850176.0,4
712262.0,856869.0,4
671325.0,856592.0,4
680592.0,858526.0,4
683098.0,847245.0,4
646816.0,829778.0,4
675633.0,863416.0,4
677551.0,860007.0,4
683656.0,842406.0,4
678195.0,860537.0,4
700747.0,821557.0,4
679054.0,898217.0,4
676360.0,866112.0,4
665308.0,868551.0,4
665775.0,845662.0,4
660459.0,860658.0,4
670119.0,859029.0,4
672068.0,874862.0,4
644301.0,925034.0,4
704574.0,866263.0,4
665660.0,828338.0,4
680579.0,836946.0,4
682053.0,853410.0,4
672544.0,859978.0,4
713088.0,880834.0,4
680969.0,838864.0,4
674151.0,853540.0,4
664903.0,839150.0,4
641902.0,755287.0,4
683610.0,910782.0,4
676962.0,897430.0,4
594720.0,848457.0,4
658748.0,777680.0,4
700341.0,869730.0,4
640432.0,837036.0,4
668519.0,860048.0,4
672668.0,861428.0,4
638004.0,837697.0,4
673071.0,856021.0,4
668772.0,858570.0,4
636871.0,929410.0,4
599612.0,859210.0,4
739318.0,810528.0,4
654937.0,781879.0,4
663095.0,949045.0,4
658734.0,883214.0,4
647968.0,877873.0,4
682699.0,851115.0,4
684739.0,884934.0,4
667620.0,855461.0,4
616281.0,845406.0,4
697161.0,840318.0,4
679237.0,862389.0,4
664260.0,859029.0,4
674121.0,858589.0,4
700962.0,927047.0,4
723397.0,859761.0,4
648230.0,857418.0,4
677896.0,867053.0,4
680828.0,839011.0,4
680757.0,826912.0,4
673826.0,880137.0,4
694897.0,834591.0,4
683585.0,905984.0,4
671609.0,815681.0,4
669748.0,863109.0,4
674659.0,860894.0,4
659734.0,844127.0,4
656156.0,868643.0,4
653258.0,860766.0,4
684131.0,898331.0,4
666516.0,859837.0,4
655237.0,871554.0,4
653592.0,959882.0,4
659280.0,845579.0,4
646422.0,900526.0,4
627580.0,840406.0,4
604546.0,847329.0,4
631316.0,837043.0,4
705518.0,925936.0,4
662898.0,884734.0,4
718088.0,852181.0,4
666315.0,872311.0,4
710082.0,881600.0,4
674433.0,860605.0,4
685821.0,816579.0,4
678856.0,850293.0,4
669427.0,846395.0,4
687043.0,806921.0,4
657981.0,866945.0,4
676480.0,860779.0,4
704065.0,832648.0,4
669178.0,866011.0,4
693783.0,860206.0,4
683027.0,876064.0,4
670957.0,810768.0,4
674134.0,908464.0,4
637665.0,858741.0,4
664325.0,877878.0,4
664025.0,801796.0,4
672800.0,874378.0,4
685124.0,873363.0,4
674496.0,870664.0,4
684602.0,823981.0,4
677769.0,841641.0,4
651576.0,865180.0,4
650720.0,817440.0,4
674472.0,860330.0,4
673028.0,860593.0,4
683413.0,861559.0,4
669638.0,842011.0,4
674365.0,860819.0,4
687360.0,862438.0,4
662684.0,813662.0,4
673242.0,886958.0,4
674311.0,859933.0,4
595580.0,798572.0,4
636769.0,854047.0,4
685551.0,798742.0,4
663945.0,828490.0,4
717187.0,883275.0,4
706906.0,885612.0,4
675988.0,841317.0,4
671976.0,864630.0,4
669366.0,869654.0,4
651216.0,815098.0,4
676767.0,832479.0,4
657879.0,857942.0,4
665828.0,868934.0,4
674905.0,876286.0,4
640000.0,863831.0,4
672818.0,881943.0,4
634800.0,823669.0,4
658590.0,886551.0,4
639463.0,847461.0,4
625416.0,868848.0,4
672266.0,862875.0,4
723173.0,872525.0,4
688098.0,908495.0,4
673410.0,863889.0,4
679788.0,889464.0,4
615826.0,863168.0,4
598671.0,854219.0,4
753254.0,840123.0,4
669430.0,905798.0,4
726847.0,875488.0,4
648443.0,845340.0,4
674114.0,970756.0,4
675551.0,956849.0,4
677459.0,896720.0,4
634004.0,895206.0,4
653745.0,809028.0,4
682420.0,846881.0,4
666636.0,858686.0,4
682481.0,879399.0,4
656834.0,885944.0,4
718257.0,857481.0,4
672598.0,856928.0,4
693997.0,774817.0,4
612392.0,916853.0,4
671444.0,860206.0,4
671404.0,857240.0,4
689628.0,868984.0,4
704570.0,845515.0,4
652233.0,886054.0,4
705929.0,851313.0,4
682315.0,843851.0,4
667131.0,849230.0,4
707894.0,904500.0,4
649345.0,864613.0,4
629998.0,877765.0,4
659542.0,840985.0,4
621278.0,888326.0,4
682304.0,876841.0,4
679747.0,860939.0,4
663049.0,843538.0,4
672876.0,844123.0,4
689948.0,850548.0,4
696257.0,894819.0,4
629377.0,853726.0,4
658034.0,897179.0,4
696808.0,866039.0,4
699668.0,863774.0,4
669697.0,859388.0,4
606506.0,766728.0,4
672474.0,868765.0,4
697011.0,856357.0,4
708262.0,945359.0,4
653181.0,886417.0,4
674521.0,861127.0,4
729194.0,856897.0,4
663168.0,859429.0,4
644843.0,838626.0,4
672797.0,843231.0,4
680677.0,868629.0,4
614497.0,830844.0,4
646671.0,857872.0,4
679242.0,853941.0,4
665697.0,929571.0,4
749941.0,916922.0,4
602365.0,916673.0,4
684537.0,843821.0,4
694308.0,904062.0,4
731712.0,878998.0,4
667713.0,847241.0,4
730383.0,879136.0,4
689363.0,817577.0,4
728743.0,850843.0,4
683941.0,915579.0,4
722463.0,882675.0,4
670519.0,870010.0,4
683252.0,858182.0,4
669288.0,860515.0,4
675357.0,870850.0,4
673449.0,860643.0,4
687325.0,883004.0,4
603472.0,894573.0,4
669713.0,851017.0,4
693362.0,837786.0,4
605535.0,925341.0,4
687078.0,791051.0,4
672387.0,800443.0,4
683048.0,937776.0,4
713579.0,879202.0,4
571669.0,873544.0,4
695260.0,824882.0,4
702654.0,841548.0,4
601619.0,825604.0,4
564416.0,878492.0,4
703773.0,870419.0,4
674931.0,870112.0,4
673140.0,864863.0,4
681053.0,892766.0,4
683321.0,873932.0,4
701017.0,931252.0,4
645677.0,814395.0,4
677362.0,853471.0,4
659046.0,930099.0,4
665426.0,853940.0,4
691827.0,863963.0,4
650661.0,861267.0,4
599647.0,858702.0,4
684091.0,842566.0,4'''
s1='''-1.98666,-0.00228882,0
-1.95108,0.0324707,0
-1.92679,-0.046936,0
-1.86688,-0.0678101,0
-1.84698,0.00881958,0
-1.8064,-0.108795,0
-1.784,-0.0264893,0
-1.75729,-0.185059,0
-1.7399,-0.0408325,0
-1.73489,-0.0914001,0
-1.71658,-0.0308838,0
-1.70807,-0.2211,0
-1.70673,-0.13623,0
-1.68668,0.270081,0
-1.66971,-0.298492,0
-1.66492,0.0975952,0
-1.64252,-0.310272,0
-1.63068,-0.00253296,0
-1.62112,0.0897827,0
-1.61554,0.166321,0
-1.61407,-0.382355,0
-1.6066,0.347351,0
-1.60306,-0.199371,0
-1.595,0.07901,0
-1.59195,0.360779,0
-1.58936,0.0223999,0
-1.58804,0.362732,0
-1.58438,-0.113831,0
-1.57388,0.262939,0
-1.57239,-0.361328,0
-1.56573,0.375122,0
-1.5614,-0.254486,0
-1.55902,-0.278778,0
-1.5528,0.357086,0
-1.54498,0.242676,0
-1.54111,0.337036,0
-1.53464,0.228302,0
-1.52863,0.0557861,0
-1.51288,-0.059082,0
-1.51059,-0.361603,0
-1.50552,0.48938,0
-1.49927,0.184265,0
-1.49515,-0.0307922,0
-1.48807,0.451355,0
-1.48346,0.173767,0
-1.47598,-0.3526,0
-1.46555,0.50119,0
-1.46384,0.35965,0
-1.45856,0.42926,0
-1.45395,-0.180267,0
-1.44376,-0.00686646,0
-1.44016,0.107635,0
-1.43466,0.224274,0
-1.42972,-0.295868,0
-1.42075,-0.220917,0
-1.4133,0.225006,0
-1.40979,0.18042,0
-1.40314,0.413483,0
-1.40115,-0.119781,0
-1.39865,0.140106,0
-1.38828,-0.548767,0
-1.38297,0.533112,0
-1.37769,-0.435028,0
-1.37555,0.324554,0
-1.37039,-0.0731506,0
-1.36783,-0.517639,0
-1.34933,-0.0012207,0
-1.34558,0.603424,0
-1.33566,-0.0579529,0
-1.33243,-0.315948,0
-1.33029,-0.0121155,0
-1.32767,0.430023,0
-1.32068,-0.307617,0
-1.31607,-0.201385,0
-1.31107,-0.000244141,0
-1.30746,-0.573303,0
-1.29886,-0.345551,0
-1.29474,-0.344635,0
-1.29062,0.605804,0
-1.27979,0.546448,0
-1.27356,0.584534,0
-1.27148,-0.499481,0
-1.26971,0.453003,0
-1.2674,0.291962,0
-1.26077,-0.0504761,0
-1.25916,-0.0043335,0
-1.25595,0.410278,0
-1.25195,-0.203003,0
-1.25006,0.443298,0
-1.24643,-0.546661,0
-1.23843,-0.454926,0
-1.23621,0.260803,0
-1.22723,0.599945,0
-1.22549,0.632782,0
-1.22211,-0.634735,0
-1.22003,-0.143372,0
-1.21259,-0.475494,0
-1.20874,0.383606,0
-1.20563,-0.649017,0
-1.20206,-0.109741,0
-1.19711,0.629974,0
-1.1936,0.618256,0
-1.19183,-0.107513,0
-1.19003,-0.0169373,0
-1.18143,0.624878,0
-1.18051,-0.701538,0
-1.17413,0.072113,0
-1.16898,-0.657288,0
-1.16473,0.209076,0
-1.15799,-0.423492,0
-1.15488,-0.326904,0
-1.15408,-0.142853,0
-1.15207,-0.651978,0
-1.14667,-0.37439,0
-1.14481,0.630157,0
-1.13977,-0.423279,0
-1.13577,0.51651,0
-1.13055,0.545013,0
-1.12375,0.577179,0
-1.12106,-0.0520935,0
-1.11835,0.618256,0
-1.11377,0.454498,0
-1.11166,0.0415955,0
-1.10962,-0.70105,0
-1.10846,0.593353,0
-1.10416,-0.105408,0
-1.10339,0.57724,0
-1.10254,-0.0229187,0
-1.0983,-0.545349,0
-1.08801,0.614258,0
-1.08545,-0.551971,0
-1.08353,-0.284119,0
-1.08078,-0.181244,0
-1.07422,0.556244,0
-1.07123,-0.917023,0
-1.06671,0.491211,0
-1.06152,-0.0592957,0
-1.0538,0.0814209,0
-1.05173,-0.187347,0
-1.04953,0.211029,0
-1.04865,-0.655731,0
-1.0477,-0.788391,0
-1.04697,-0.877838,0
-1.04449,-0.377136,0
-1.03946,-0.881714,0
-1.03857,-0.252075,0
-1.03311,-0.966339,0
-1.03137,0.125153,0
-1.02905,0.259277,0
-1.02667,0.041626,0
-1.02185,0.929871,0
-1.02042,0.94223,0
-1.01749,-0.813538,0
-1.01392,-0.205414,0
-1.0123,0.658447,0
-1.00879,-0.918762,0
-1.00308,0.639313,0
-1.00085,-0.768555,0
-0.998718,-0.864166,0
-0.995728,-0.417908,0
-0.99411,-0.707855,0
-0.992645,0.576233,0
-0.986542,-0.103149,0
-0.98468,-0.821777,0
-0.983765,0.3284,0
-0.980164,-0.204102,0
-0.979126,0.73291,0
-0.974945,-0.215393,0
-0.972198,-0.355072,0
-0.969818,0.465332,0
-0.966797,0.880554,0
-0.962799,0.741272,0
-0.959106,-0.173523,0
-0.956635,0.699219,0
-0.953094,0.818298,0
-0.94986,-0.881592,0
-0.947357,0.168427,0
-0.946045,-0.908447,0
-0.944305,-0.571228,0
-0.941193,0.00820923,0
-0.936005,-0.361206,0
-0.929291,0.607727,0
-0.928223,-0.294739,0
-0.925385,0.00518799,0
-0.92334,0.186066,0
-0.919403,-0.846313,0
-0.918518,-0.806519,0
-0.915344,0.309265,0
-0.906464,0.825958,0
-0.90329,-0.668182,0
-0.899872,0.434753,0
-0.898071,0.411591,0
-0.894867,0.359741,0
-0.893799,-0.368439,0
-0.888428,-0.10202,0
-0.885254,-0.304688,0
-0.88092,0.247192,0
-0.878265,0.284424,0
-0.872772,0.0657349,0
-0.868378,-0.847046,0
-0.867401,-0.515228,0
-0.862793,0.706848,0
-0.859894,0.738953,0
-0.858368,0.172302,0
-0.857239,-0.689484,0
-0.852692,-0.165894,0
-0.846558,-0.721924,0
-0.842773,-0.727783,0
-0.840393,0.52359,0
-0.83847,-0.66745,0
-0.834717,0.17984,0
-0.83197,0.206512,0
-0.827545,-0.0689087,0
-0.824677,-0.549866,0
-0.823425,0.374237,0
-0.822388,0.452728,0
-0.82074,-0.481903,0
-0.817078,0.0176697,0
-0.813049,0.237823,0
-0.811646,-0.343109,0
-0.808594,0.787079,0
-0.806091,0.715576,0
-0.803162,0.451691,0
-0.79834,-0.282349,0
-0.797028,0.576294,0
-0.795044,0.772949,0
-0.792145,0.474091,0
-0.788574,0.128326,0
-0.785889,0.245972,0
-0.781128,-0.077179,0
-0.778259,0.568634,0
-0.775452,-0.643127,0
-0.769623,0.423828,0
-0.764374,0.108795,0
-0.757263,0.129242,0
-0.750793,-0.570679,0
-0.748169,0.462189,0
-0.746399,0.0569458,0
-0.742706,-0.109619,0
-0.738098,-0.31134,0
-0.730591,0.348969,0
-0.722961,-0.560974,0
-0.720428,0.568726,0
-0.717102,0.227417,0
-0.71225,-0.168945,0
-0.700378,-0.322174,0
-0.693604,-0.28421,0
-0.689697,0.154266,0
-0.686646,0.143219,0
-0.680115,0.0863342,0
-0.673859,0.2547,0
-0.669128,-0.562744,0
-0.665955,0.582977,0
-0.662598,0.393372,0
-0.658447,0.0443726,0
-0.651062,0.462158,0
-0.648407,-0.0144958,0
-0.642487,-0.618073,0
-0.639008,-0.379883,0
-0.636566,0.454041,0
-0.628998,-0.0436401,0
-0.625916,0.0791321,0
-0.618042,0.441925,0
-0.615662,0.243927,0
-0.611298,0.550171,0
-0.606445,-0.3974,0
-0.602142,-0.119781,0
-0.60141,-0.43277,0
-0.598938,0.349792,0
-0.589081,0.49411,0
-0.588318,0.232056,0
-0.583527,0.476624,0
-0.575287,0.122253,0
-0.57251,-0.221619,0
-0.570862,0.318176,0
-0.568298,0.0210571,0
-0.555969,-0.466614,0
-0.552521,0.359222,0
-0.546753,-0.476837,0
-0.543915,-0.271881,0
-0.538666,0.494843,0
-0.534363,-0.126862,0
-0.519043,0.109436,0
-0.509277,0.453735,0
-0.503113,-0.439026,0
-0.499329,-0.274139,0
-0.49646,0.33432,0
-0.490753,0.382355,0
-0.483124,-0.14035,0
-0.4758,-0.452026,0
-0.468842,-0.209991,0
-0.463226,0.300842,0
-0.455109,0.36499,0
-0.453064,0.089325,0
-0.451263,-0.351044,0
-0.447388,-0.394501,0
-0.44046,0.158386,0
-0.433319,0.0203247,0
-0.429871,0.39621,0
-0.424805,0.207031,0
-0.419037,-0.195618,0
-0.415894,0.306641,0
-0.409271,-0.0709229,0
-0.399261,-0.27536,0
-0.393799,-0.195526,0
-0.374939,0.0413513,0
-0.371338,-0.160187,0
-0.367035,-0.311249,0
-0.356689,-0.0560303,0
-0.349274,-0.199585,0
-0.343719,-0.00466919,0
-0.337585,-0.307465,0
-0.330872,0.180237,0
-0.312378,0.0904236,0
-0.307129,-0.30304,0
-0.302338,0.135315,0
-0.295898,0.109589,0
-0.29126,-0.0582275,0
-0.272339,-0.0432129,0
-0.260773,0.148071,0
-0.255249,0.215088,0
-0.232117,-0.0505066,0
-0.226471,0.0637207,0
-0.214508,0.17041,0
-0.204773,-0.196503,0
-0.182526,0.099823,0
-0.16333,0.0557556,0
-0.148804,-0.0747986,0
-0.142639,0.109314,0
-0.131531,-0.0812073,0
-0.113983,0.0637207,0
-0.104889,-0.0725098,0
-0.0978394,0.0479126,0
0.035675,-0.0065918,1
0.0690918,-0.0161438,1
0.0965576,-0.0513916,1
0.150604,0.0811768,1
0.169495,0.144989,1
0.187866,0.168488,1
0.198273,0.118134,1
0.205658,-0.125641,1
0.212982,-0.0686646,1
0.226959,-0.0592957,1
0.240356,0.0065918,1
0.24588,0.18985,1
0.252075,0.0549927,1
0.272705,0.000762939,1
0.278442,0.129303,1
0.285431,-0.26709,1
0.291443,0.156982,1
0.302063,-0.0382996,1
0.325256,0.307251,1
0.34198,-0.285461,1
0.345459,-0.19519,1
0.35025,-0.275543,1
0.354004,0.351593,1
0.36496,-0.231567,1
0.370178,-0.0288086,1
0.376434,-0.330902,1
0.384857,-0.28064,1
0.386108,0.300507,1
0.392761,-0.343964,1
0.40329,0.0374756,1
0.409119,-0.378601,1
0.41391,0.138062,1
0.418762,0.0652771,1
0.422821,-0.0962524,1
0.433563,0.405975,1
0.443909,0.0197449,1
0.448425,0.376099,1
0.458923,0.256226,1
0.463837,0.0225525,1
0.467255,0.292999,1
0.477783,-0.229614,1
0.47995,-0.300446,1
0.484344,0.293121,1
0.48645,0.424622,1
0.493042,0.189331,1
0.503357,0.165802,1
0.508698,-0.503265,1
0.512909,-0.255737,1
0.519989,0.41684,1
0.527832,0.484283,1
0.528687,-0.0919189,1
0.532654,0.430267,1
0.541504,-0.319427,1
0.543976,0.375732,1
0.549744,-0.0967712,1
0.556427,0.314606,1
0.559723,0.412079,1
0.574188,0.0428772,1
0.578125,0.220947,1
0.582672,0.00747681,1
0.585968,-0.109161,1
0.603302,-0.22821,1
0.606873,0.428314,1
0.615906,0.238251,1
0.618073,0.573334,1
0.624756,-0.61853,1
0.62973,0.137665,1
0.635284,-0.607971,1
0.638519,0.0758667,1
0.641327,0.143555,1
0.644836,0.643311,1
0.650421,-0.637604,1
0.657562,-0.548553,1
0.660614,-0.456909,1
0.667877,-0.199371,1
0.673492,0.141663,1
0.675293,-0.0345154,1
0.682617,-0.274445,1
0.68515,0.424286,1
0.685883,0.0248108,1
0.693634,-0.493256,1
0.696442,0.488464,1
0.706848,-0.361694,1
0.719116,-0.166199,1
0.723022,0.0182495,1
0.727478,-0.0773315,1
0.733337,-0.444244,1
0.739014,-0.494904,1
0.742798,-0.119415,1
0.74588,0.686401,1
0.750336,0.541992,1
0.752045,0.226013,1
0.753448,-0.265778,1
0.759064,-0.197601,1
0.763275,-0.653473,1
0.766296,0.164307,1
0.773376,-0.591644,1
0.778992,-0.071167,1
0.780762,0.0630188,1
0.78418,0.672394,1
0.790192,-0.611023,1
0.793488,0.743713,1
0.796356,-0.233734,1
0.799774,0.0809021,1
0.801941,0.0674133,1
0.805634,0.258423,1
0.806915,0.348816,1
0.812103,0.161407,1
0.816803,-0.297668,1
0.825256,0.801666,1
0.82901,-0.522614,1
0.833496,0.74472,1
0.835419,-0.190948,1
0.837311,0.0430298,1
0.84314,0.820526,1
0.846069,0.195984,1
0.849609,-0.749878,1
0.854004,-0.692841,1
0.857086,-0.410767,1
0.862274,-0.470215,1
0.863373,-0.726074,1
0.866608,-0.798828,1
0.8685,-0.139404,1
0.871674,-0.210571,1
0.872833,0.168457,1
0.875244,0.0609436,1
0.878296,-0.393402,1
0.88266,0.42572,1
0.886505,0.472076,1
0.887665,-0.449615,1
0.891083,0.0343018,1
0.895172,-0.792725,1
0.898102,-0.723419,1
0.901733,-0.693207,1
0.907776,-0.511597,1
0.917511,0.845673,1
0.921112,0.595978,1
0.9245,-0.326782,1
0.926453,-0.362305,1
0.93042,-0.660706,1
0.935364,0.397583,1
0.937653,0.601746,1
0.941254,0.647034,1
0.944946,-0.940155,1
0.946991,0.84671,1
0.952148,-0.457825,1
0.953522,-0.883942,1
0.955292,-0.464844,1
0.958374,-0.503479,1
0.962036,0.0777893,1
0.962341,-0.139221,1
0.964203,-0.117279,1
0.965302,-0.812775,1
0.976563,0.300507,1
0.978271,-0.874207,1
0.978851,-0.0107117,1
0.980774,0.104523,1
0.98584,-0.974518,1
0.989471,0.122925,1
0.994873,0.857056,1
0.99823,-0.374603,1
0.999146,-0.211456,1
0.999969,0.572601,1
1.00275,-0.389008,1
1.00443,-0.652161,1
1.00693,0.417664,1
1.01181,-0.952789,1
1.01343,-0.644318,1
1.0148,-0.599182,1
1.01849,-0.0119324,1
1.01941,-0.569977,1
1.02359,-0.668152,1
1.02405,-0.716522,1
1.0274,0.604004,1
1.03067,0.150757,1
1.03397,-0.630066,1
1.03726,0.353271,1
1.0386,0.542786,1
1.04385,-0.462494,1
1.04559,-0.733551,1
1.04672,0.25592,1
1.05087,0.908569,1
1.05402,0.291595,1
1.05695,0.584961,1
1.06055,-0.710632,1
1.0632,0.620728,1
1.06711,-0.0430298,1
1.07596,0.0642395,1
1.08087,-0.510742,1
1.08316,-0.696594,1
1.08514,0.719055,1
1.08917,-0.357269,1
1.09253,0.710022,1
1.09436,-0.0574341,1
1.09811,0.375153,1
1.09927,-0.0587158,1
1.10187,0.20517,1
1.10394,0.543335,1
1.11038,-0.502319,1
1.11203,-0.841614,1
1.1138,-0.494568,1
1.11676,0.0870361,1
1.12009,0.788788,1
1.1235,-0.853912,1
1.12732,0.644196,1
1.1293,0.377167,1
1.13141,-0.260376,1
1.13437,-0.578461,1
1.13605,0.581024,1
1.1409,-0.66803,1
1.14276,-0.681641,1
1.14688,-0.0361023,1
1.15015,-0.802521,1
1.15164,0.163086,1
1.15506,-0.170288,1
1.1597,-0.694397,1
1.16235,0.827057,1
1.16534,-0.659851,1
1.16757,0.750641,1
1.16928,-0.135864,1
1.17108,0.111877,1
1.17548,-0.359741,1
1.17792,-0.456451,1
1.18182,-0.511383,1
1.18524,-0.346893,1
1.18948,0.67099,1
1.19122,-0.73996,1
1.19647,-0.630371,1
1.19907,-0.0548401,1
1.2037,0.160095,1
1.20541,0.391846,1
1.20804,0.329437,1
1.20959,0.120209,1
1.2164,0.600922,1
1.22638,-0.518158,1
1.22836,-0.586578,1
1.22995,0.134338,1
1.23401,-0.319458,1
1.23758,0.559509,1
1.24042,-0.257202,1
1.24161,0.667419,1
1.24417,-0.671844,1
1.24692,0.7276,1
1.24908,0.663605,1
1.25552,-0.630829,1
1.26093,0.231812,1
1.26624,-0.66922,1
1.26962,0.62796,1
1.27469,-0.279022,1
1.28384,0.0115356,1
1.28952,0.0409241,1
1.29514,0.401855,1
1.29694,-0.0458679,1
1.30084,-0.310791,1
1.30286,-0.303986,1
1.30823,-0.489136,1
1.3125,0.429596,1
1.32062,-0.0213013,1
1.32462,0.643921,1
1.32849,-0.0484314,1
1.33179,-0.559845,1
1.33591,0.274841,1
1.33987,0.454407,1
1.34106,-0.658569,1
1.35321,0.234436,1
1.35831,-0.45108,1
1.36191,0.429443,1
1.36401,0.430756,1
1.36618,-0.370819,1
1.36768,-0.481598,1
1.36905,0.255554,1
1.36966,0.478912,1
1.3765,-0.296661,1
1.39063,-0.526245,1
1.39209,-0.153656,1
1.39481,-0.321381,1
1.40402,0.237061,1
1.41052,0.202789,1
1.41373,0.151978,1
1.41763,0.285614,1
1.4252,-0.119904,1
1.42776,0.310669,1
1.43051,0.179535,1
1.4346,-0.367401,1
1.43893,0.479767,1
1.44672,-0.263397,1
1.45709,-0.418915,1
1.45856,0.240662,1
1.46823,-0.179321,1
1.47595,0.290741,1
1.47754,0.415436,1
1.48785,-0.407867,1
1.49118,0.220367,1
1.49228,0.0247192,1
1.49612,-0.448669,1
1.50476,0.100159,1
1.5097,-0.337921,1
1.52151,0.474121,1
1.52701,-0.157928,1
1.53482,-0.246582,1
1.54272,0.246094,1
1.54611,-0.012146,1
1.55042,0.30661,1
1.55835,-0.329041,1
1.56339,0.00402832,1
1.56668,-0.0856628,1
1.56873,0.342102,1
1.58154,-0.328766,1
1.58865,-0.115112,1
1.599,0.378235,1
1.6033,-0.342926,1
1.61719,0.152161,1
1.61942,0.101929,1
1.63718,0.137329,1
1.63901,0.29599,1
1.64832,0.154114,1
1.65167,-0.303406,1
1.66376,0.264069,1
1.67181,0.0612488,1
1.67459,0.268402,1
1.68314,0.122101,1
1.68491,-0.218536,1
1.69772,0.279724,1
1.70377,-0.188202,1
1.71082,0.173248,1
1.7168,0.0922241,1
1.72382,0.056488,1
1.76147,-0.205048,1
1.78043,-0.119263,1
1.80167,0.158081,1
1.82196,-0.0116882,1
1.89746,0.0846863,1
1.92902,0.0603943,1
1.94073,-0.0214539,1
2.05722,0.0110779,2
2.10861,0.102142,2
2.14987,-0.0936584,2
2.17911,0.173309,2
2.18698,-0.109192,2
2.20157,-0.146881,2
2.2197,0.091217,2
2.24872,-0.247925,2
2.2532,0.22406,2
2.26382,0.241638,2
2.27188,-0.208344,2
2.27835,-0.070343,2
2.28525,0.0941467,2
2.29672,-0.277924,2
2.29907,0.0544434,2
2.31921,0.259399,2
2.33023,0.0839844,2
2.3428,-0.175171,2
2.3526,0.0591736,2
2.35352,0.133026,2
2.36484,0.0157471,2
2.36829,-0.269623,2
2.37363,-0.0921631,2
2.38733,-0.0741882,2
2.38959,0.0939941,2
2.40002,0.0426025,2
2.40585,0.209473,2
2.41144,-0.0728149,2
2.41879,0.224518,2
2.42297,0.175842,2
2.42682,0.204285,2
2.42789,0.0256042,2
2.435,-0.246185,2
2.43747,-0.0715332,2
2.44388,-0.20163,2
2.44864,-0.142212,2
2.46478,0.193542,2
2.46765,0.286072,2
2.47937,-0.167877,2
2.48972,-0.124512,2
2.49762,0.313904,2
2.5011,-0.0581055,2
2.50766,0.294159,2
2.51285,0.356049,2
2.5202,0.342072,2
2.52289,0.118591,2
2.52631,0.204468,2
2.53787,-0.0915527,2
2.54175,-0.391205,2
2.55161,0.539917,2
2.55545,0.0363159,2
2.55902,0.0640564,2
2.5607,0.528381,2
2.56357,0.509491,2
2.57114,0.551941,2
2.58441,-0.549316,2
2.58743,0.267731,2
2.59094,0.539001,2
2.59372,-0.304382,2
2.60315,-0.515411,2
2.61188,-0.605804,2
2.62854,0.112122,2
2.63144,0.536224,2
2.63452,-0.472046,2
2.63599,-0.468658,2
2.63885,-0.121674,2
2.64261,0.392975,2
2.65051,0.365112,2
2.65549,-0.158417,2
2.6629,-0.329742,2
2.66971,0.461731,2
2.68051,0.252258,2
2.68524,0.365448,2
2.69016,-0.564606,2
2.69623,-0.551727,2
2.7002,-0.698151,2
2.70883,-0.458435,2
2.7139,-0.170288,2
2.7164,0.0361633,2
2.71814,0.65448,2
2.72324,-0.718109,2
2.72723,0.615723,2
2.73398,0.0748596,2
2.74237,-0.732697,2
2.74606,-0.572052,2
2.75839,-0.272949,2
2.75995,0.586792,2
2.76364,-0.508545,2
2.77298,-0.560638,2
2.77682,0.0619812,2
2.77802,0.173706,2
2.78073,-0.602753,2
2.78256,-0.639435,2
2.78326,-0.525208,2
2.78488,-0.452972,2
2.78641,0.405029,2
2.78781,0.369049,2
2.78964,-0.64621,2
2.79434,0.0440369,2
2.79495,0.280609,2
2.79855,-0.260925,2
2.80307,-0.243042,2
2.80582,0.581055,2
2.81036,0.382233,2
2.81281,0.038269,2
2.81903,0.139404,2
2.82272,0.183868,2
2.82599,-0.334991,2
2.828,0.694977,2
2.82977,0.492859,2
2.83295,-0.157745,2
2.84146,-0.138336,2
2.84363,-0.123779,2
2.84824,0.0020752,2
2.85178,0.429321,2
2.85681,-0.432983,2
2.85843,-0.26062,2
2.86032,-0.768524,2
2.86804,0.759979,2
2.87018,-0.525452,2
2.87125,0.193695,2
2.87335,-0.804504,2
2.87732,0.139679,2
2.88031,0.642242,2
2.88464,-0.69339,2
2.88751,-0.237579,2
2.88989,0.449158,2
2.89114,0.182465,2
2.89581,0.210724,2
2.89938,-0.165558,2
2.90128,-0.682861,2
2.90222,0.520782,2
2.90607,0.610657,2
2.90976,0.646149,2
2.91849,0.258667,2
2.92023,0.351654,2
2.92188,0.726013,2
2.92603,0.410858,2
2.92819,-0.481171,2
2.93301,-0.6521,2
2.93375,-0.455658,2
2.93649,-0.414703,2
2.93857,0.28595,2
2.94196,-0.550354,2
2.94498,-0.538269,2
2.94739,0.101196,2
2.94943,0.315247,2
2.95187,-0.270203,2
2.95551,0.809296,2
2.95801,0.663727,2
2.96033,0.339386,2
2.96317,0.366333,2
2.9675,0.89502,2
2.96887,-0.514252,2
2.97232,-0.0813599,2
2.97369,0.45752,2
2.97565,-0.284943,2
2.97696,0.463379,2
2.97906,0.0137634,2
2.9808,-0.946472,2
2.98367,-0.539825,2
2.98596,0.220612,2
2.98901,0.431091,2
2.99033,0.747498,2
2.99329,-0.382599,2
2.99576,0.738739,2
2.99887,-0.678223,2
3.00092,0.371033,2
3.00455,-0.155701,2
3.00589,0.779358,2
3.00998,-0.895355,2
3.01294,-0.559235,2
3.01599,0.933167,2
3.01868,-0.436096,2
3.02548,-0.586945,2
3.02826,-0.673523,2
3.03024,-0.356506,2
3.03531,0.822601,2
3.03925,-0.544922,2
3.04364,-0.490143,2
3.04602,0.493958,2
3.04761,-0.068573,2
3.04971,-0.0145264,2
3.05396,-0.120453,2
3.05743,-0.457092,2
3.05875,0.610321,2
3.06168,0.814545,2
3.06763,0.712799,2
3.07077,-0.0324097,2
3.07281,-0.235535,2
3.07443,-0.292145,2
3.07974,0.165344,2
3.08524,0.163055,2
3.08661,-0.311707,2
3.08749,-0.458557,2
3.09381,0.358978,2
3.09811,-0.598083,2
3.10007,0.384277,2
3.10239,0.822235,2
3.10626,-0.548706,2
3.11032,0.0556335,2
3.11185,0.350525,2
3.11728,0.798615,2
3.11844,0.456482,2
3.12057,0.491852,2
3.12723,-0.500946,2
3.13058,0.763672,2
3.13449,-0.206512,2
3.13641,0.434021,2
3.14218,-0.763763,2
3.14899,-0.65036,2
3.15115,0.270172,2
3.15399,-0.278412,2
3.15457,0.612427,2
3.15753,-0.426178,2
3.15942,0.138214,2
3.16125,-0.675934,2
3.1716,0.67337,2
3.17664,0.0755005,2
3.18048,-0.604767,2
3.18756,0.544708,2
3.19,0.777618,2
3.19876,-0.680298,2
3.20206,-0.737396,2
3.21487,-0.369385,2
3.21985,0.347504,2
3.22232,0.7276,2
3.22562,-0.718597,2
3.23056,0.748138,2
3.2327,0.122192,2
3.23462,0.524323,2
3.23734,0.717255,2
3.24307,-0.411346,2
3.24435,-0.324493,2
3.24692,0.334259,2
3.25131,0.726898,2
3.25693,0.360413,2
3.25861,-0.693146,2
3.2619,0.351166,2
3.26447,-0.603943,2
3.26501,-0.147186,2
3.26624,0.0339355,2
3.27014,0.499084,2
3.2717,-0.621552,2
3.27347,0.528503,2
3.27777,0.152313,2
3.28333,-0.501434,2
3.28769,-0.280609,2
3.28922,-0.339417,2
3.2959,0.134979,2
3.29633,-0.030365,2
3.30093,0.0595398,2
3.30658,-0.479645,2
3.31137,0.389252,2
3.31741,0.48642,2
3.32153,0.654602,2
3.32693,0.153931,2
3.32919,0.0820007,2
3.332,0.408051,2
3.33798,0.151215,2
3.34775,-0.640961,2
3.34933,0.544098,2
3.3533,-0.479828,2
3.35666,-0.602295,2
3.3605,0.598236,2
3.36368,-0.32901,2
3.36963,-0.54657,2
3.37216,-0.165497,2
3.37738,0.409119,2
3.38049,-0.346313,2
3.38693,0.423767,2
3.38907,0.450378,2
3.39676,-0.439636,2
3.40265,-0.297852,2
3.40353,0.191589,2
3.40561,-0.276794,2
3.40683,-0.572144,2
3.41605,-0.0525513,2
3.42078,0.350586,2
3.42383,0.229065,2
3.42749,0.00424194,2
3.43936,0.127319,2
3.4451,-0.496277,2
3.45013,-0.0566406,2
3.45911,0.484161,2
3.474,0.38681,2
3.47787,-0.129852,2
3.47946,-0.00546265,2
3.48279,0.48996,2
3.4863,0.486206,2
3.49957,0.101379,2
3.50613,-0.394775,2
3.51175,-0.28717,2
3.52103,0.287811,2
3.52444,0.0290527,2
3.5271,0.0439453,2
3.52866,0.427521,2
3.53918,-0.0939026,2
3.54218,-0.0062561,2
3.5448,-0.347443,2
3.55627,-0.144684,2
3.56598,0.38916,2
3.5708,0.136505,2
3.58566,-0.0426331,2
3.58865,-0.224548,2
3.59561,-0.289886,2
3.60492,0.215271,2
3.61404,0.306366,2
3.62933,-0.057312,2
3.63083,-0.065918,2
3.64154,0.18985,2
3.65033,0.052124,2
3.65396,-0.204346,2
3.66513,-0.0629883,2
3.67136,0.0726318,2
3.68698,0.00180054,2
3.69879,-0.231598,2
3.70825,0.162201,2
3.72067,0.00872803,2
3.72763,0.0240173,2
3.73965,0.0938721,2
3.7464,-0.0191345,2
3.7493,0.0215149,2
3.75403,-0.0637207,2
3.77179,0.0154419,2
3.78943,-0.0479126,2
3.81622,-0.0239563,2
3.82578,-0.14856,2
3.84033,0.0152283,2
3.85938,-0.122925,2
3.88324,-0.0414734,2
3.89261,-0.063324,2
3.94199,0.0240784,2
-1.93231,2.05688,3
-1.90518,2.009,3
-1.88153,1.94046,3
-1.86761,1.92023,3
-1.85272,2.10648,3
-1.83911,2.00101,3
-1.7999,2.19858,3
-1.78265,2.21329,3
-1.77502,2.00275,3
-1.75934,2.04181,3
-1.73984,1.80817,3
-1.72427,2.09909,3
-1.72137,1.82779,3
-1.71008,1.88257,3
-1.69571,1.8363,3
-1.68637,1.87103,3
-1.67737,2.28848,3
-1.67133,2.30392,3
-1.66821,1.94101,3
-1.65323,2.02115,3
-1.638,2.2608,3
-1.62967,1.65765,3
-1.62424,2.1701,3
-1.60574,1.8114,3
-1.59955,2.06055,3
-1.59415,2.29919,3
-1.58881,2.39233,3
-1.58514,1.84055,3
-1.58054,1.93057,3
-1.5737,1.99783,3
-1.57159,1.6824,3
-1.56454,1.57214,3
-1.55038,1.5929,3
-1.54828,2.05545,3
-1.54092,1.64651,3
-1.53262,1.71887,3
-1.53027,1.68292,3
-1.51788,1.99048,3
-1.51538,1.8457,3
-1.50912,1.6058,3
-1.50653,1.58188,3
-1.49503,2.39099,3
-1.48654,1.6199,3
-1.48071,2.11398,3
-1.47455,2.37515,3
-1.47293,1.68576,3
-1.47055,1.51404,3
-1.46542,1.51028,3
-1.45731,1.86404,3
-1.45291,1.83685,3
-1.44852,1.50958,3
-1.44302,1.47629,3
-1.43936,1.48929,3
-1.43219,1.54187,3
-1.431,2.18814,3
-1.42657,1.60938,3
-1.42151,2.48254,3
-1.41852,1.8111,3
-1.41235,1.84836,3
-1.40887,1.6337,3
-1.40222,2.34082,3
-1.39685,1.6467,3
-1.38885,2.04858,3
-1.38577,1.52826,3
-1.38177,1.73004,3
-1.37192,1.67947,3
-1.36615,1.8345,3
-1.36203,2.18109,3
-1.35809,1.85056,3
-1.35205,1.50137,3
-1.3457,1.97635,3
-1.33948,2.22568,3
-1.33527,2.34802,3
-1.33145,2.48105,3
-1.32724,2.62027,3
-1.32339,1.80786,3
-1.31711,1.57184,3
-1.30859,1.57455,3
-1.30188,2.21515,3
-1.29965,2.52585,3
-1.29474,2.12704,3
-1.29004,1.56985,3
-1.28903,2.49606,3
-1.28204,2.44705,3
-1.27554,1.77289,3
-1.2626,2.72552,3
-1.25943,2.12665,3
-1.25473,1.56488,3
-1.24942,2.65216,3
-1.24545,2.39194,3
-1.24493,1.95157,3
-1.23962,2.02783,3
-1.23746,1.35693,3
-1.23288,2.37238,3
-1.22629,2.49457,3
-1.22336,1.48874,3
-1.22122,1.69821,3
-1.21933,1.62869,3
-1.21609,2.38919,3
-1.21219,1.27737,3
-1.20898,1.79559,3
-1.20444,1.50153,3
-1.20148,2.20203,3
-1.19656,2.6908,3
-1.19067,2.34515,3
-1.18658,2.45279,3
-1.18286,1.93405,3
-1.18091,2.42419,3
-1.18042,2.20667,3
-1.17651,2.57343,3
-1.16931,1.79968,3
-1.16806,1.23145,3
-1.16318,1.55911,3
-1.15622,2.01114,3
-1.15216,2.28735,3
-1.15063,2.65219,3
-1.14862,2.43289,3
-1.14557,2.06973,3
-1.14322,2.32407,3
-1.14059,1.82507,3
-1.13736,1.88669,3
-1.13263,2.582,3
-1.12817,2.54031,3
-1.1214,2.66574,3
-1.11954,1.35843,3
-1.11313,1.79922,3
-1.11172,1.22986,3
-1.10895,1.7196,3
-1.10507,1.8999,3
-1.09399,1.33704,3
-1.08908,1.43921,3
-1.08633,1.2366,3
-1.08289,2.83868,3
-1.0795,2.08481,3
-1.07889,2.79538,3
-1.07648,2.68683,3
-1.06384,2.70099,3
-1.06293,1.75659,3
-1.05896,1.61542,3
-1.05725,2.20807,3
-1.05338,2.54083,3
-1.04941,2.60687,3
-1.04828,2.48746,3
-1.04376,1.26062,3
-1.04172,2.42773,3
-1.04001,1.99774,3
-1.03702,1.22577,3
-1.03372,2.92981,3
-1.02942,2.16196,3
-1.02716,1.06186,3
-1.02344,1.09222,3
-1.02271,1.33392,3
-1.02032,2.05518,3
-1.01859,2.41794,3
-1.01749,2.53476,3
-1.01566,1.50214,3
-1.00848,2.47766,3
-1.00531,2.09509,3
-0.99942,1.67645,3
-0.996887,2.58102,3
-0.995117,1.31046,3
-0.988586,1.69122,3
-0.983612,2.97116,3
-0.982452,2.61359,3
-0.981293,2.28003,3
-0.979431,2.31299,3
-0.977814,2.17334,3
-0.970306,1.10837,3
-0.964844,2.12823,3
-0.963837,1.79843,3
-0.962738,1.10635,3
-0.95639,1.81512,3
-0.953705,1.99649,3
-0.951569,2.4512,3
-0.945313,1.16666,3
-0.939941,2.41327,3
-0.934998,1.70398,3
-0.933167,2.62225,3
-0.924866,2.58084,3
-0.922394,1.23865,3
-0.921722,2.65118,3
-0.918884,2.34668,3
-0.916687,2.4686,3
-0.91449,2.54422,3
-0.912994,1.87717,3
-0.908875,2.117,3
-0.905273,2.87112,3
-0.902527,1.35944,3
-0.902008,2.30267,3
-0.899597,1.31812,3
-0.896606,1.5083,3
-0.893097,1.47214,3
-0.889618,1.94409,3
-0.886475,2.31436,3
-0.88031,1.1763,3
-0.874756,1.22842,3
-0.871979,1.14838,3
-0.863892,1.87018,3
-0.860809,2.08353,3
-0.855835,2.57315,3
-0.85376,2.58228,3
-0.851044,1.54004,3
-0.849518,2.31415,3
-0.848175,2.39185,3
-0.845795,1.66077,3
-0.845062,1.59973,3
-0.842255,1.42807,3
-0.840027,1.90369,3
-0.838593,2.35593,3
-0.833282,2.63586,3
-0.831909,2.8194,3
-0.83017,2.3656,3
-0.825653,1.61288,3
-0.823639,1.24976,3
-0.822937,1.6076,3
-0.817963,2.23648,3
-0.814758,1.98148,3
-0.811951,2.37616,3
-0.811157,2.20868,3
-0.808594,1.84961,3
-0.802246,2.70557,3
-0.800537,2.0368,3
-0.797058,2.07294,3
-0.794128,1.6991,3
-0.790283,1.82886,3
-0.787506,1.67371,3
-0.78537,1.77838,3
-0.780334,2.34338,3
-0.779388,2.40213,3
-0.777771,1.77911,3
-0.776672,1.74136,3
-0.772827,2.48706,3
-0.769958,1.44272,3
-0.764771,2.54755,3
-0.760284,1.40082,3
-0.753021,2.53934,3
-0.750122,2.36432,3
-0.745422,1.94559,3
-0.742828,1.97717,3
-0.740631,2.25046,3
-0.736664,2.21231,3
-0.729828,2.34045,3
-0.727234,1.59097,3
-0.721588,2.02725,3
-0.719604,1.86606,3
-0.717773,1.69919,3
-0.715973,1.93118,3
-0.713013,1.29312,3
-0.708099,2.63278,3
-0.702972,1.69394,3
-0.700897,2.17633,3
-0.698975,1.36737,3
-0.693634,2.64679,3
-0.685394,2.29868,3
-0.680023,2.41211,3
-0.676575,2.60156,3
-0.671356,1.75348,3
-0.669617,1.51645,3
-0.668488,1.6795,3
-0.665771,1.89801,3
-0.663269,2.34152,3
-0.660919,1.44513,3
-0.65271,2.57986,3
-0.651123,1.48828,3
-0.647064,1.68301,3
-0.641846,2.45169,3
-0.637207,1.71317,3
-0.627136,1.75308,3
-0.621948,1.86032,3
-0.617889,1.52066,3
-0.616089,1.78156,3
-0.608063,2.53748,3
-0.604218,1.44385,3
-0.598663,1.48621,3
-0.593079,2.17416,3
-0.585846,2.41162,3
-0.579285,2.01501,3
-0.575409,1.62912,3
-0.571045,2.0018,3
-0.567841,2.29053,3
-0.566376,1.59747,3
-0.558533,1.69229,3
-0.555634,2.37976,3
-0.549835,2.2962,3
-0.542786,2.3046,3
-0.537781,1.88229,3
-0.531921,2.17551,3
-0.529541,1.77872,3
-0.52182,1.72003,3
-0.510406,2.40939,3
-0.507813,2.10477,3
-0.495209,1.57202,3
-0.484802,2.06616,3
-0.482056,2.08286,3
-0.475342,1.77625,3
-0.473175,2.15536,3
-0.463287,2.19543,3
-0.459564,1.5499,3
-0.455139,1.92926,3
-0.442139,1.85336,3
-0.440826,2.06454,3
-0.438293,2.3674,3
-0.433197,2.04492,3
-0.425476,2.28766,3
-0.41806,2.19403,3
-0.409271,1.60754,3
-0.402039,2.13235,3
-0.393921,1.83652,3
-0.392365,2.00961,3
-0.391357,1.68546,3
-0.380096,1.91913,3
-0.374023,1.72702,3
-0.36145,2.34616,3
-0.355469,2.21082,3
-0.343201,1.86374,3
-0.329559,2.18097,3
-0.31723,1.96301,3
-0.310486,1.84198,3
-0.302399,1.77579,3
-0.284698,1.87708,3
-0.280304,2.23764,3
-0.267639,2.0585,3
-0.252197,2.09048,3
-0.2435,2.09958,3
-0.220337,2.07513,3
-0.213654,2.20673,3
-0.194946,2.04202,3
-0.187531,1.86646,3
-0.176483,1.99268,3
-0.15741,1.97293,3
-0.150238,2.11682,3
-0.109222,1.97632,3
-0.0785522,2.01627,3
0.0498047,2.01563,4
0.07547,1.96558,4
0.0967102,2.06485,4
0.119232,1.97302,4
0.149628,1.99344,4
0.17981,2.11484,4
0.196259,1.85559,4
0.213837,2.15826,4
0.218811,1.83517,4
0.226868,2.1799,4
0.234589,2.21979,4
0.241577,2.22522,4
0.242554,1.90375,4
0.248047,2.19672,4
0.2612,2.10107,4
0.270782,1.9407,4
0.276642,2.1347,4
0.285065,1.85049,4
0.290863,2.13766,4
0.307037,2.21341,4
0.312317,2.25632,4
0.319977,1.94849,4
0.337524,2.24146,4
0.34433,1.93442,4
0.352966,1.81665,4
0.361908,2.26648,4
0.366821,2.2916,4
0.379395,2.03543,4
0.387726,1.82043,4
0.390594,1.78113,4
0.397675,1.73721,4
0.406433,1.6358,4
0.413727,2.17776,4
0.424194,1.92386,4
0.432343,1.90762,4
0.436035,1.64154,4
0.444122,2.10648,4
0.457611,1.88199,4
0.465698,1.97809,4
0.468506,1.71127,4
0.473846,1.76093,4
0.480408,1.90369,4
0.486755,2.13434,4
0.488098,1.94785,4
0.500946,1.89081,4
0.505005,2.47287,4
0.513245,2.47263,4
0.516327,1.69006,4
0.519989,1.51355,4
0.524292,2.27463,4
0.531616,2.1297,4
0.532532,2.4993,4
0.540009,2.18967,4
0.542816,1.63846,4
0.544922,1.63574,4
0.547302,2.3905,4
0.551758,2.24619,4
0.555664,1.87564,4
0.56311,2.38608,4
0.567932,1.87479,4
0.57959,1.96906,4
0.582703,2.36267,4
0.583313,2.32056,4
0.584229,2.47784,4
0.590302,1.67664,4
0.594025,2.11856,4
0.599731,2.21393,4
0.604675,2.39914,4
0.612091,2.01276,4
0.614624,1.54434,4
0.616302,1.83231,4
0.621307,1.51501,4
0.623962,1.8428,4
0.627686,1.4433,4
0.629608,2.30557,4
0.632263,2.40936,4
0.636444,1.69153,4
0.641815,2.25549,4
0.644257,2.49106,4
0.647827,2.58881,4
0.648773,2.40009,4
0.652649,2.24237,4
0.65332,1.67035,4
0.656311,1.87369,4
0.659698,1.69968,4
0.664032,1.86621,4
0.667694,1.44403,4
0.674042,2.1062,4
0.677429,2.37064,4
0.68396,2.13257,4
0.690063,2.11905,4
0.699677,1.78638,4
0.703491,2.66208,4
0.715515,2.65948,4
0.722198,1.79132,4
0.725708,1.91199,4
0.729767,2.08673,4
0.732513,1.50052,4
0.73468,1.41068,4
0.742401,2.06387,4
0.756866,1.50586,4
0.760895,1.30511,4
0.76297,1.84229,4
0.764435,2.314,4
0.766785,1.88754,4
0.771454,1.9472,4
0.777252,1.83698,4
0.779633,1.60602,4
0.786896,2.36261,4
0.790497,1.53693,4
0.794891,2.00064,4
0.796356,2.60468,4
0.806824,2.72598,4
0.810333,2.11389,4
0.811218,2.13998,4
0.813995,2.7764,4
0.816376,1.42538,4
0.820892,2.7233,4
0.826233,1.25299,4
0.82785,1.76834,4
0.83252,2.78571,4
0.841614,2.1275,4
0.84436,1.29156,4
0.850555,1.54596,4
0.857971,2.68817,4
0.86261,2.85135,4
0.864746,1.21802,4
0.867828,2.10562,4
0.873871,2.79004,4
0.87674,1.95322,4
0.879303,1.46719,4
0.882935,2.3389,4
0.884796,1.20697,4
0.890533,2.85565,4
0.893677,1.12601,4
0.897369,1.57867,4
0.899078,2.73566,4
0.902588,1.40256,4
0.905548,2.55405,4
0.910004,2.04425,4
0.914612,1.89005,4
0.918549,1.75778,4
0.920166,2.34967,4
0.924133,2.64761,4
0.928101,2.20978,4
0.930176,1.38589,4
0.935944,1.50919,4
0.938629,2.80194,4
0.944366,1.56985,4
0.947327,2.84259,4
0.952087,1.25964,4
0.953735,2.73212,4
0.959137,1.62881,4
0.961304,1.22244,4
0.964661,2.94904,4
0.97229,2.63315,4
0.974701,1.16119,4
0.976837,2.42847,4
0.978241,1.64038,4
0.978607,2.64182,4
0.983276,1.86719,4
0.987335,2.74646,4
0.991211,2.12164,4
0.995575,2.1405,4
0.99942,2.45764,4
1.0011,2.7247,4
1.00665,1.72437,4
1.01181,1.31787,4
1.01422,2.68039,4
1.01505,1.74924,4
1.0188,1.54608,4
1.01944,1.87228,4
1.02274,2.70798,4
1.02524,2.11072,4
1.02728,1.77829,4
1.03152,1.43063,4
1.03497,1.83707,4
1.04089,2.1532,4
1.04205,1.66162,4
1.04483,1.77875,4
1.04794,2.89334,4
1.05045,2.21967,4
1.05075,1.73294,4
1.05359,2.17255,4
1.05807,2.4856,4
1.06305,2.42496,4
1.06558,1.34528,4
1.06821,2.52658,4
1.06976,1.57889,4
1.07382,2.2666,4
1.07736,1.38477,4
1.07965,2.68549,4
1.08179,2.78842,4
1.08701,1.12036,4
1.08902,1.17276,4
1.09409,1.19925,4
1.09552,2.88324,4
1.10345,1.60687,4
1.10501,1.94421,4
1.10876,1.88174,4
1.11917,2.22131,4
1.12164,1.5784,4
1.12555,1.46365,4
1.12885,1.3085,4
1.13547,2.32892,4
1.14401,2.51874,4
1.14584,1.1799,4
1.14868,2.73602,4
1.15149,1.40915,4
1.1564,2.09967,4
1.16293,1.41537,4
1.16708,1.84311,4
1.17145,2.39813,4
1.17422,1.65451,4
1.17798,2.67172,4
1.18024,2.48166,4
1.18237,1.89236,4
1.18536,1.28271,4
1.1871,1.93979,4
1.19022,2.72491,4
1.19196,2.11636,4
1.19998,2.24448,4
1.203,1.95493,4
1.20691,2.41669,4
1.21017,1.79059,4
1.21292,2.25168,4
1.21637,1.32956,4
1.21964,1.26236,4
1.22681,2.06091,4
1.23032,2.36292,4
1.23398,1.48395,4
1.23428,1.31516,4
1.23746,1.48041,4
1.24292,1.88968,4
1.24515,1.44006,4
1.24576,1.69604,4
1.2475,2.7507,4
1.24884,1.34125,4
1.25128,2.65216,4
1.25912,1.79425,4
1.26474,2.23618,4
1.2688,1.55417,4
1.27496,1.28854,4
1.27838,1.7753,4
1.28265,1.78351,4
1.28549,1.39865,4
1.28662,1.81525,4
1.28952,1.6825,4
1.298,1.84341,4
1.30142,2.42227,4
1.30307,1.74374,4
1.30768,2.12805,4
1.31146,2.56543,4
1.31613,1.39706,4
1.3237,1.57587,4
1.33221,1.67941,4
1.34573,1.84424,4
1.34924,2.03104,4
1.35547,2.63284,4
1.36118,1.63284,4
1.37033,1.76379,4
1.37244,1.38937,4
1.37601,2.32922,4
1.37665,2.2789,4
1.3786,2.09503,4
1.38113,2.4678,4
1.38672,2.03894,4
1.38885,1.97885,4
1.39249,2.04135,4
1.39444,1.97327,4
1.40128,2.5007,4
1.40558,1.57877,4
1.4071,2.47723,4
1.41196,1.43253,4
1.42569,2.05725,4
1.42944,2.43845,4
1.43365,1.82431,4
1.43588,1.79889,4
1.43881,2.38235,4
1.4465,2.51273,4
1.4502,1.82932,4
1.45224,1.70197,4
1.45831,1.61929,4
1.45941,2.37091,4
1.46548,2.27289,4
1.4696,2.28732,4
1.47058,1.91858,4
1.47321,1.99161,4
1.47687,2.08157,4
1.47818,1.62796,4
1.4812,1.53873,4
1.48953,2.03027,4
1.49637,2.36279,4
1.49673,2.09805,4
1.4978,1.9917,4
1.49854,1.81308,4
1.51105,2.38922,4
1.52209,1.83853,4
1.52643,1.53937,4
1.52954,2.05023,4
1.53922,2.4418,4
1.53979,2.02747,4
1.55099,1.67908,4
1.56708,1.75351,4
1.56857,2.34061,4
1.57703,1.6449,4
1.58847,2.28378,4
1.59637,1.95422,4
1.60422,2.22235,4
1.61621,2.25272,4
1.62051,1.68796,4
1.62802,1.85583,4
1.6311,2.2012,4
1.64667,2.16736,4
1.65735,2.33206,4
1.66574,1.85709,4
1.66766,2.03305,4
1.67233,1.69717,4
1.68106,1.72302,4
1.69522,1.95154,4
1.69638,2.18243,4
1.71417,2.23691,4
1.72604,1.85678,4
1.73157,2.10828,4
1.74692,1.78223,4
1.75659,2.08804,4
1.76129,1.7829,4
1.78708,1.85797,4
1.79169,1.93298,4
1.80368,2.19223,4
1.83136,2.13458,4
1.85022,2.10358,4
1.88742,1.98291,4
1.97363,2.00342,4
2.08051,2.07288,5
2.12631,2.03818,5
2.13678,2.12567,5
2.14609,2.08231,5
2.16718,1.88608,5
2.20209,1.97421,5
2.21243,2.05444,5
2.21686,1.95367,5
2.23111,2.03116,5
2.24045,1.76459,5
2.24786,1.86081,5
2.26047,2.09396,5
2.27234,1.90063,5
2.29266,2.0405,5
2.29971,2.29544,5
2.30536,1.73196,5
2.30966,2.20422,5
2.31253,2.18457,5
2.34552,1.97803,5
2.34833,2.11533,5
2.35211,1.91727,5
2.35977,1.89926,5
2.37704,1.86072,5
2.39496,1.63565,5
2.39789,1.96722,5
2.40634,1.75452,5
2.41229,2.332,5
2.42398,1.67471,5
2.42566,2.06308,5
2.42856,1.89316,5
2.43216,2.40808,5
2.43634,1.64755,5
2.44736,2.19891,5
2.44931,1.81674,5
2.45566,2.25739,5
2.46338,1.83505,5
2.47003,2.13745,5
2.49081,1.63867,5
2.49597,2.45261,5
2.50433,2.20355,5
2.50674,2.27658,5
2.517,1.83606,5
2.53079,2.05582,5
2.53748,2.46936,5
2.54633,1.95776,5
2.54922,2.54803,5
2.55371,2.15991,5
2.56439,2.09027,5
2.57095,2.05215,5
2.57687,1.96835,5
2.58994,2.21454,5
2.59195,1.54279,5
2.59607,1.42657,5
2.5994,1.81088,5
2.608,2.09012,5
2.61414,2.56586,5
2.62115,1.5423,5
2.62866,1.81262,5
2.63065,2.56,5
2.63406,1.60068,5
2.63727,1.41281,5
2.64209,1.94485,5
2.64447,1.66971,5
2.6485,1.84897,5
2.65063,1.54749,5
2.65259,1.59027,5
2.65894,1.44745,5
2.66196,2.08856,5
2.66367,1.45935,5
2.66879,2.06796,5
2.67599,2.57193,5
2.67819,2.62231,5
2.67969,2.33859,5
2.68063,1.55148,5
2.68491,2.31653,5
2.68872,2.46362,5
2.69083,1.51233,5
2.69455,2.55502,5
2.6991,1.70511,5
2.70084,2.54617,5
2.70398,1.62527,5
2.70721,2.082,5
2.71283,2.39813,5
2.71442,2.37228,5
2.71851,2.69086,5
2.72104,1.97528,5
2.72272,1.87076,5
2.72525,1.35876,5
2.73077,2.68164,5
2.73373,2.44119,5
2.73807,1.32022,5
2.74542,2.10632,5
2.75104,2.14603,5
2.75513,1.92566,5
2.75638,1.92261,5
2.75839,2.48141,5
2.7644,1.6434,5
2.76685,1.76196,5
2.77206,2.06345,5
2.77481,1.86658,5
2.7775,1.77911,5
2.77924,2.34863,5
2.78134,2.7623,5
2.78436,2.59637,5
2.79172,2.46579,5
2.79282,2.25174,5
2.79926,1.4415,5
2.79971,2.76895,5
2.80173,1.34152,5
2.80847,1.44696,5
2.80887,2.37592,5
2.81119,2.39737,5
2.81555,1.33749,5
2.81763,1.69507,5
2.82346,1.91269,5
2.82623,2.63986,5
2.82709,1.28668,5
2.82855,1.75256,5
2.83426,2.52164,5
2.84146,1.62427,5
2.84875,2.45218,5
2.85107,2.63165,5
2.85413,1.64307,5
2.85626,2.85052,5
2.85983,2.37909,5
2.86145,1.53748,5
2.86493,2.41849,5
2.8671,2.12442,5
2.87418,2.46793,5
2.87735,2.42197,5
2.87964,1.73154,5
2.88052,1.2052,5
2.88205,1.65347,5
2.88531,1.19986,5
2.88782,2.38373,5
2.89001,1.2749,5
2.89502,1.93448,5
2.89795,2.33572,5
2.90332,2.88205,5
2.90741,2.17593,5
2.90942,2.724,5
2.91183,1.42346,5
2.91565,2.25543,5
2.91861,1.80734,5
2.92108,2.65094,5
2.92419,2.75955,5
2.93088,2.09436,5
2.93704,2.71921,5
2.94171,1.31122,5
2.94427,1.36105,5
2.94745,2.70377,5
2.94946,2.16382,5
2.95361,2.17462,5
2.95758,1.63968,5
2.95944,2.25626,5
2.961,2.03885,5
2.96613,2.32782,5
2.9678,1.57446,5
2.97046,2.18643,5
2.97379,2.84265,5
2.97696,1.51141,5
2.98004,2.7735,5
2.98083,1.30136,5
2.98282,2.58005,5
2.98523,1.92081,5
2.98697,2.49792,5
2.98883,1.24069,5
2.99283,2.40121,5
2.99484,2.42133,5
2.99863,1.8591,5
3.00385,1.47449,5
3.00696,1.39435,5
3.0094,2.68738,5
3.01105,1.84149,5
3.01205,2.198,5
3.01743,1.47104,5
3.02087,1.09155,5
3.0224,2.32632,5
3.02328,2.32111,5
3.02411,2.88953,5
3.0264,1.26935,5
3.03076,2.07434,5
3.03238,2.86264,5
3.03647,2.70499,5
3.03912,2.19061,5
3.04593,2.13562,5
3.04877,1.33902,5
3.05072,1.17465,5
3.05655,2.07593,5
3.06744,1.96967,5
3.07144,1.65613,5
3.07581,1.20181,5
3.07742,1.19644,5
3.08032,2.88486,5
3.08218,2.06015,5
3.08316,2.54797,5
3.08521,1.75708,5
3.08835,2.40158,5
3.0932,2.3255,5
3.09586,1.23459,5
3.09723,2.66397,5
3.10355,1.57327,5
3.10684,1.77908,5
3.11163,1.19577,5
3.11441,1.94153,5
3.12079,2.04791,5
3.12311,2.45999,5
3.12735,2.63947,5
3.13083,1.76965,5
3.13138,1.26459,5
3.1333,1.47659,5
3.13419,2.43005,5
3.13696,2.1459,5
3.14136,1.65723,5
3.14392,1.29086,5
3.14929,2.064,5
3.15491,2.62646,5
3.15811,2.58032,5
3.16385,2.1109,5
3.16599,1.29443,5
3.16776,1.22086,5
3.16846,1.83691,5
3.17105,2.81357,5
3.17389,2.00592,5
3.17877,1.957,5
3.18137,1.79501,5
3.18256,2.54654,5
3.1875,1.6377,5
3.18976,1.45212,5
3.19608,2.04047,5
3.20227,2.27914,5
3.20422,1.96429,5
3.20563,1.54437,5
3.21078,1.71762,5
3.2131,1.26306,5
3.21835,1.35565,5
3.22195,1.80038,5
3.22311,2.73428,5
3.22617,1.4519,5
3.22861,1.95343,5
3.23581,1.49716,5
3.23874,2.2663,5
3.24231,2.62048,5
3.24594,1.61954,5
3.24969,1.35989,5
3.2536,2.44592,5
3.25418,2.6011,5
3.25754,2.5784,5
3.26321,2.194,5
3.26666,1.34439,5
3.26773,2.60007,5
3.27057,2.29419,5
3.27527,1.83249,5
3.27954,1.36417,5
3.28699,2.27759,5
3.29712,2.63467,5
3.30301,1.60641,5
3.30719,2.39944,5
3.31085,2.26004,5
3.31464,1.41522,5
3.31793,2.4794,5
3.3222,2.12479,5
3.3259,2.40936,5
3.32886,1.89935,5
3.33429,1.48233,5
3.33588,1.58374,5
3.34055,1.5762,5
3.34399,2.08392,5
3.34711,2.03406,5
3.35147,2.26361,5
3.35477,1.52029,5
3.35745,2.25208,5
3.3663,1.78552,5
3.36682,1.99854,5
3.37466,1.43594,5
3.37692,2.43002,5
3.38132,1.44839,5
3.38739,2.28635,5
3.39316,1.8114,5
3.39801,1.84586,5
3.40479,2.0069,5
3.4119,1.42624,5
3.41449,2.42435,5
3.41516,1.98715,5
3.4191,1.92853,5
3.43216,1.80518,5
3.44815,1.78436,5
3.45438,2.33969,5
3.45657,1.47473,5
3.46252,1.72556,5
3.46732,1.59048,5
3.46982,1.6813,5
3.47385,1.92932,5
3.49091,1.66876,5
3.49554,2.31009,5
3.49622,1.85815,5
3.50247,1.89487,5
3.51593,1.90283,5
3.52621,2.33618,5
3.52762,1.59692,5
3.54263,1.74579,5
3.55145,1.67621,5
3.56015,2.40253,5
3.56989,2.10098,5
3.57733,1.86804,5
3.58649,1.98285,5
3.58981,2.13232,5
3.59628,1.67398,5
3.59805,1.95706,5
3.60922,1.8978,5
3.62021,1.92978,5
3.62399,1.80493,5
3.62561,1.75723,5
3.63898,1.94696,5
3.65802,2.29364,5
3.6752,2.06989,5
3.6806,1.68863,5
3.6864,2.00787,5
3.69318,1.96558,5
3.70163,1.82159,5
3.71313,1.8793,5
3.71497,2.26001,5
3.72678,2.10449,5
3.73892,2.10092,5
3.74979,1.90497,5
3.76236,2.20676,5
3.7858,1.93524,5
3.81601,2.03711,5
3.82367,2.05148,5
3.83133,2.0611,5
3.8692,2.05157,5
3.89252,1.90277,5
3.91681,2.05954,5
-1.93716,4.00568,6
-1.91208,3.92288,6
-1.8916,4.03107,6
-1.86368,4.03214,6
-1.8309,4.12003,6
-1.80826,3.94485,6
-1.79886,4.03201,6
-1.78427,4.19528,6
-1.76816,4.12051,6
-1.75714,3.84512,6
-1.75476,4.12378,6
-1.74869,3.83783,6
-1.74258,3.82544,6
-1.73029,4.1568,6
-1.72473,4.13089,6
-1.70978,4.02588,6
-1.66925,3.7688,6
-1.65714,4.03146,6
-1.65585,4.25748,6
-1.65021,3.75052,6
-1.64029,3.8992,6
-1.63577,3.66113,6
-1.62808,3.66614,6
-1.62021,3.77026,6
-1.61264,4.0434,6
-1.60876,4.18246,6
-1.60507,3.63626,6
-1.59464,3.85828,6
-1.58121,4.04358,6
-1.57144,4.23856,6
-1.55994,4.43195,6
-1.54971,3.99045,6
-1.54459,3.97098,6
-1.53189,4.17886,6
-1.52747,3.96231,6
-1.51993,4.36838,6
-1.51755,4.15887,6
-1.51346,4.07718,6
-1.50281,4.35059,6
-1.49753,3.74194,6
-1.49118,4.39777,6
-1.48499,3.98297,6
-1.48264,4.13516,6
-1.48044,3.85791,6
-1.47592,4.27701,6
-1.47345,4.4585,6
-1.46512,3.76663,6
-1.45819,4.24606,6
-1.45422,3.93137,6
-1.44986,3.88171,6
-1.4411,3.49606,6
-1.43759,3.97925,6
-1.43518,3.45157,6
-1.4252,4.24118,6
-1.42209,3.4938,6
-1.41507,3.9664,6
-1.41391,3.85895,6
-1.40424,4.05252,6
-1.40009,3.43027,6
-1.38983,3.60193,6
-1.38812,4.09888,6
-1.38052,3.67798,6
-1.37323,4.11621,6
-1.37051,4.29428,6
-1.36404,3.75333,6
-1.35995,4.30975,6
-1.35199,3.89795,6
-1.34375,3.82172,6
-1.34067,3.85062,6
-1.33646,3.59869,6
-1.33511,4.22617,6
-1.33133,4.51883,6
-1.327,4.62265,6
-1.32007,3.6824,6
-1.31705,4.00336,6
-1.31229,4.43109,6
-1.30774,4.59918,6
-1.30417,4.58472,6
-1.30048,4.37076,6
-1.29706,4.01746,6
-1.28867,4.35828,6
-1.28415,4.3327,6
-1.27817,4.59531,6
-1.27536,3.70526,6
-1.2739,3.30762,6
-1.26816,4.71918,6
-1.26648,3.8154,6
-1.26367,3.84341,6
-1.26169,4.24149,6
-1.25974,4.56906,6
-1.25803,3.34717,6
-1.25488,4.62271,6
-1.25424,3.41391,6
-1.24847,3.33289,6
-1.24161,3.34216,6
-1.23608,4.25186,6
-1.23492,3.34488,6
-1.2316,3.4856,6
-1.22635,4.32755,6
-1.22583,4.5387,6
-1.22537,4.56091,6
-1.22052,3.39728,6
-1.21762,3.62656,6
-1.21524,4.51346,6
-1.21158,3.88824,6
-1.21008,3.2262,6
-1.20651,4.33279,6
-1.2041,3.75058,6
-1.20029,4.03595,6
-1.19907,4.52066,6
-1.19699,3.36902,6
-1.19263,3.2439,6
-1.18744,3.86005,6
-1.1842,3.65869,6
-1.18356,4.44711,6
-1.18213,4.21411,6
-1.1796,4.69614,6
-1.16998,4.7326,6
-1.16425,3.54642,6
-1.15988,4.35052,6
-1.15863,3.70688,6
-1.15375,4.43063,6
-1.15128,4.54565,6
-1.1506,3.91678,6
-1.1489,3.35446,6
-1.14474,3.21637,6
-1.14322,4.09033,6
-1.1405,4.69623,6
-1.13666,4.08719,6
-1.1297,4.86465,6
-1.12717,3.55411,6
-1.12103,3.33636,6
-1.11597,3.33255,6
-1.11496,3.28769,6
-1.11276,3.8978,6
-1.10941,3.95364,6
-1.10443,3.25259,6
-1.10355,4.61301,6
-1.10251,4.56088,6
-1.09607,4.16669,6
-1.09323,4.84079,6
-1.08844,3.68112,6
-1.08206,3.97418,6
-1.07767,4.33752,6
-1.07245,4.87308,6
-1.06894,4.51733,6
-1.06628,4.68063,6
-1.06155,3.6676,6
-1.05707,3.11813,6
-1.05673,3.15582,6
-1.05212,3.42032,6
-1.04645,3.23642,6
-1.03918,4.0997,6
-1.03693,4.57831,6
-1.0365,3.67932,6
-1.03165,3.23599,6
-1.02933,3.48038,6
-1.02597,4.37521,6
-1.02463,4.82019,6
-1.01904,3.57507,6
-1.01346,4.84543,6
-1.01224,3.46912,6
-1.01126,3.70456,6
-1.00882,4.71552,6
-1.00153,4.59964,6
-1.00018,3.65262,6
-0.99707,4.40604,6
-0.994812,3.68506,6
-0.991272,4.56195,6
-0.990723,3.39612,6
-0.986694,4.67816,6
-0.984619,3.60764,6
-0.980774,4.44907,6
-0.978699,4.85193,6
-0.977875,4.37137,6
-0.974945,3.80917,6
-0.969055,4.81473,6
-0.966553,4.83807,6
-0.964386,3.43582,6
-0.962036,4.10193,6
-0.959961,3.31979,6
-0.955811,3.61572,6
-0.951935,3.92624,6
-0.950256,4.69232,6
-0.943695,3.11362,6
-0.934662,4.79431,6
-0.929871,3.26837,6
-0.928986,4.27811,6
-0.927002,4.16693,6
-0.924377,3.93552,6
-0.920532,3.9353,6
-0.916779,4.61771,6
-0.910797,4.08926,6
-0.907379,4.16061,6
-0.89978,3.35471,6
-0.897705,4.70502,6
-0.896423,4.35092,6
-0.894043,3.23837,6
-0.892944,3.51477,6
-0.8909,4.07986,6
-0.886658,4.24921,6
-0.879486,3.97018,6
-0.874329,4.53241,6
-0.870636,4.7189,6
-0.86676,4.26657,6
-0.85733,3.83264,6
-0.852081,4.70062,6
-0.847168,3.60559,6
-0.841736,4.37518,6
-0.835327,3.28564,6
-0.829712,4.5578,6
-0.82132,4.1442,6
-0.818726,3.41922,6
-0.81485,3.46609,6
-0.81131,4.49881,6
-0.805725,3.74658,6
-0.804321,4.36053,6
-0.802032,4.72482,6
-0.798248,3.60373,6
-0.792969,3.94534,6
-0.790985,4.31223,6
-0.787445,3.92978,6
-0.783783,3.56094,6
-0.781799,3.92175,6
-0.778076,3.24286,6
-0.772278,4.26865,6
-0.767517,4.03027,6
-0.762939,3.50674,6
-0.760193,4.68491,6
-0.755646,3.34592,6
-0.750946,3.28937,6
-0.746887,3.51874,6
-0.74649,3.4097,6
-0.744324,4.0191,6
-0.740631,3.65689,6
-0.730286,3.94162,6
-0.72287,3.6041,6
-0.714996,3.45151,6
-0.710693,3.5285,6
-0.709076,3.40454,6
-0.707855,4.07953,6
-0.706787,4.37881,6
-0.704132,4.24292,6
-0.701965,3.90842,6
-0.70047,3.51724,6
-0.698242,4.23773,6
-0.695618,4.37582,6
-0.69397,3.45782,6
-0.690125,4.33707,6
-0.688202,3.3468,6
-0.684784,4.12488,6
-0.681641,4.42679,6
-0.676697,4.13739,6
-0.668518,3.35706,6
-0.665405,3.6525,6
-0.663025,3.82498,6
-0.654388,3.34863,6
-0.650848,3.95288,6
-0.6492,3.88586,6
-0.647064,4.63589,6
-0.64389,4.16901,6
-0.640778,4.01926,6
-0.637115,3.57504,6
-0.634521,3.85696,6
-0.630524,4.39166,6
-0.626526,3.95816,6
-0.623688,3.73834,6
-0.618652,3.71042,6
-0.615784,4.09485,6
-0.6138,4.2196,6
-0.610962,4.29758,6
-0.606323,3.9473,6
-0.602051,3.48679,6
-0.591644,3.87137,6
-0.582275,4.43628,6
-0.578003,3.77768,6
-0.570648,4.20959,6
-0.563019,4.13147,6
-0.55896,3.61252,6
-0.55777,4.24698,6
-0.554962,4.36093,6
-0.549042,4.06668,6
-0.53833,4.16998,6
-0.533813,4.07227,6
-0.527435,4.47617,6
-0.525696,4.24094,6
-0.523285,4.24829,6
-0.519806,4.2843,6
-0.518097,4.41098,6
-0.515381,3.52289,6
-0.504333,4.4733,6
-0.500031,4.04248,6
-0.496185,4.38489,6
-0.490082,4.27466,6
-0.486542,4.14966,6
-0.484406,3.62491,6
-0.473602,3.70819,6
-0.467773,3.59668,6
-0.450928,4.10559,6
-0.44635,4.27695,6
-0.438324,4.28928,6
-0.431061,3.76746,6
-0.422516,3.78616,6
-0.415436,4.18512,6
-0.410767,4.39377,6
-0.407806,4.34552,6
-0.405701,3.6969,6
-0.396301,3.9317,6
-0.387177,4.35425,6
-0.376068,4.05475,6
-0.369781,4.36899,6
-0.359924,3.79892,6
-0.354828,3.82147,6
-0.350586,4.15848,6
-0.340973,3.87634,6
-0.323364,4.25729,6
-0.316864,4.13535,6
-0.305695,4.01968,6
-0.294281,4.15765,6
-0.272766,3.7814,6
-0.266541,3.93216,6
-0.253204,4.00793,6
-0.244873,4.13492,6
-0.231628,3.79352,6
-0.214081,3.85052,6
-0.200195,3.90726,6
-0.18924,4.0137,6
-0.171387,3.96051,6
-0.143188,3.94156,6
-0.134491,4.04556,6
-0.124359,3.90234,6
-0.103607,4.05844,6
-0.0974426,3.92606,6
0.0699463,3.96103,7
0.0842896,3.95071,7
0.097229,4.05142,7
0.133209,3.97961,7
0.152008,4.13089,7
0.158997,3.88974,7
0.16391,3.92572,7
0.174957,4.16818,7
0.182678,4.17368,7
0.191925,3.95502,7
0.204529,4.16885,7
0.214691,3.83188,7
0.217743,4.17441,7
0.237701,3.81274,7
0.24173,4.21759,7
0.257965,3.93951,7
0.276459,3.80591,7
0.28299,4.05844,7
0.295959,4.15607,7
0.304016,3.89716,7
0.310974,4.03534,7
0.317993,3.85281,7
0.32254,4.18903,7
0.33078,4.15012,7
0.348419,4.31232,7
0.352081,3.77771,7
0.361176,4.33829,7
0.368378,3.91931,7
0.37677,3.84897,7
0.38501,3.85229,7
0.389099,3.78857,7
0.390778,3.97021,7
0.411499,3.5946,7
0.416077,4.12045,7
0.421844,3.80212,7
0.427826,3.77063,7
0.436279,3.72046,7
0.445404,4.42212,7
0.448975,4.10416,7
0.452057,3.59415,7
0.458527,3.89658,7
0.465149,3.8317,7
0.470795,4.39383,7
0.475128,4.08832,7
0.484039,3.84259,7
0.487671,4.32434,7
0.490692,4.44788,7
0.49707,3.74902,7
0.507629,3.82458,7
0.512604,3.91183,7
0.515564,4.03012,7
0.521576,4.11664,7
0.526642,4.22415,7
0.532684,4.4826,7
0.535339,3.77408,7
0.541046,3.56155,7
0.550507,3.4808,7
0.554382,3.6246,7
0.55719,4.40378,7
0.565186,3.5498,7
0.568848,4.17639,7
0.576752,4.45053,7
0.580414,3.63895,7
0.591248,3.43042,7
0.598572,4.10342,7
0.609528,4.53558,7
0.614563,4.18158,7
0.615601,4.33267,7
0.61969,3.57541,7
0.624359,3.53674,7
0.627136,3.95932,7
0.628754,4.00287,7
0.631134,3.58316,7
0.636169,3.46759,7
0.636658,4.58798,7
0.641113,4.46606,7
0.646698,4.14413,7
0.651886,3.61816,7
0.655762,3.88837,7
0.663574,3.75967,7
0.670227,4.0813,7
0.677673,4.42862,7
0.679077,3.48047,7
0.68396,3.42603,7
0.689392,4.53543,7
0.694672,3.45004,7
0.700531,3.96552,7
0.703735,4.41422,7
0.704437,3.53354,7
0.706543,4.14926,7
0.708221,3.50708,7
0.712585,3.39868,7
0.719696,4.16425,7
0.722229,4.64084,7
0.7258,3.58621,7
0.731964,3.82022,7
0.733795,4.36942,7
0.738007,3.82019,7
0.745422,4.577,7
0.748962,3.89218,7
0.752594,3.77304,7
0.75528,4.72433,7
0.757416,3.34933,7
0.764862,4.07834,7
0.771606,3.44815,7
0.776001,3.64163,7
0.784729,4.64453,7
0.785797,3.58783,7
0.791107,4.64563,7
0.792419,3.38144,7
0.799377,4.3269,7
0.802795,3.77039,7
0.805542,3.73499,7
0.809387,4.15317,7
0.81192,4.55093,7
0.815704,3.95392,7
0.819336,3.61752,7
0.820435,3.93759,7
0.825745,3.61932,7
0.828705,3.67844,7
0.830322,4.04004,7
0.831482,4.82062,7
0.838043,3.99487,7
0.840546,3.5032,7
0.845184,4.59506,7
0.850525,3.99109,7
0.853699,4.19952,7
0.855042,3.164,7
0.858582,4.41919,7
0.861389,4.79349,7
0.861908,4.27606,7
0.862823,4.57336,7
0.863861,4.03156,7
0.871307,3.26294,7
0.874542,3.38635,7
0.877228,3.62494,7
0.879517,3.78497,7
0.885437,3.16162,7
0.887024,3.33151,7
0.88913,4.09912,7
0.890961,3.58923,7
0.895569,4.28372,7
0.898895,3.35129,7
0.907959,4.33481,7
0.911316,3.48312,7
0.912659,3.16589,7
0.916504,3.38168,7
0.92218,4.6001,7
0.924957,3.1293,7
0.926025,3.96365,7
0.928833,4.86517,7
0.932831,4.01578,7
0.937286,4.50848,7
0.94043,4.68011,7
0.943848,3.8186,7
0.946716,3.63217,7
0.952057,3.14615,7
0.953583,3.65805,7
0.9552,3.80569,7
0.959106,4.87698,7
0.961548,4.65311,7
0.964325,4.63211,7
0.965698,4.17017,7
0.967346,3.98978,7
0.970337,4.12433,7
0.972168,4.31296,7
0.975891,4.01099,7
0.980011,3.85413,7
0.984863,3.59445,7
0.988312,4.73459,7
0.99295,3.68121,7
0.998749,3.28589,7
1.00439,3.11453,7
1.00693,3.8801,7
1.00882,4.17465,7
1.01126,3.28241,7
1.01376,3.13205,7
1.01669,4.57007,7
1.02228,3.27173,7
1.02795,3.84067,7
1.02896,4.18372,7
1.03293,3.66116,7
1.0361,4.06717,7
1.03674,4.05231,7
1.03836,3.93793,7
1.03979,4.63669,7
1.04605,4.29846,7
1.04776,4.35269,7
1.04926,4.19342,7
1.0509,3.05722,7
1.05533,4.20367,7
1.05838,3.14542,7
1.06146,4.59708,7
1.06436,3.77728,7
1.07053,4.16779,7
1.07538,3.9227,7
1.07846,3.10388,7
1.08301,4.72879,7
1.08664,3.36081,7
1.08847,3.47971,7
1.09351,4.25931,7
1.09772,3.91141,7
1.09927,4.02655,7
1.10696,3.15912,7
1.11038,4.78918,7
1.11783,4.71777,7
1.12006,4.45444,7
1.12247,3.71918,7
1.12613,3.81482,7
1.12982,4.38556,7
1.13229,3.80038,7
1.13434,3.66605,7
1.13864,3.79861,7
1.1413,4.64835,7
1.14322,4.74179,7
1.14572,4.21008,7
1.15042,4.09592,7
1.1543,3.2262,7
1.16144,4.83313,7
1.16638,3.77261,7
1.16962,3.69693,7
1.1731,4.75662,7
1.17456,4.56192,7
1.17908,3.9707,7
1.18317,3.23975,7
1.1875,4.68878,7
1.19153,4.42087,7
1.19559,3.27811,7
1.2002,3.74158,7
1.20187,3.38937,7
1.20822,4.21289,7
1.21365,4.13977,7
1.21945,3.4346,7
1.22513,4.58984,7
1.22665,4.45535,7
1.23141,4.48871,7
1.23383,4.22974,7
1.23581,4.3576,7
1.2392,4.22699,7
1.24744,4.39505,7
1.25046,4.3988,7
1.25696,3.45731,7
1.26266,4.63867,7
1.26794,4.61017,7
1.27133,3.34631,7
1.28162,3.77228,7
1.28513,3.44543,7
1.29578,4.21854,7
1.29926,3.67358,7
1.30377,3.86905,7
1.30756,4.34384,7
1.31403,3.52872,7
1.31592,4.11362,7
1.32071,3.48148,7
1.32419,4.02737,7
1.32813,3.7991,7
1.33121,4.40909,7
1.33328,3.94778,7
1.33633,3.40237,7
1.34256,3.59402,7
1.34732,3.86252,7
1.35059,3.57568,7
1.35419,3.59152,7
1.36493,3.77625,7
1.36652,4.21954,7
1.37347,4.04715,7
1.3765,4.09247,7
1.38159,4.3981,7
1.38583,3.46332,7
1.38837,4.08563,7
1.39072,4.55569,7
1.39334,4.40332,7
1.40125,3.4505,7
1.40802,4.25922,7
1.41162,4.18893,7
1.42047,4.22083,7
1.42398,3.71088,7
1.42719,3.43393,7
1.42987,4.43616,7
1.43237,4.29373,7
1.43555,4.35336,7
1.4386,4.25284,7
1.44342,3.60666,7
1.44717,3.63153,7
1.45654,3.50824,7
1.46722,3.6756,7
1.47852,4.0267,7
1.48776,4.01953,7
1.49249,4.19623,7
1.50092,4.26981,7
1.51413,4.22943,7
1.5155,3.59619,7
1.51962,4.06415,7
1.53003,3.59634,7
1.53775,3.67578,7
1.54608,3.96881,7
1.55106,3.96075,7
1.55527,3.77103,7
1.56079,4.23743,7
1.5675,3.58743,7
1.57202,3.59995,7
1.5769,3.9545,7
1.57938,3.6149,7
1.59106,4.01852,7
1.59909,4.31793,7
1.60968,4.23132,7
1.61761,4.18909,7
1.62943,4.22043,7
1.63351,4.07733,7
1.64813,3.97092,7
1.65067,3.70499,7
1.65451,4.10547,7
1.66208,3.97876,7
1.66708,4.1467,7
1.68292,3.81824,7
1.69366,4.0668,7
1.70398,4.24649,7
1.71625,3.72037,7
1.73828,3.80057,7
1.75278,3.89041,7
1.75754,4.04712,7
1.76697,4.11414,7
1.79871,4.19754,7
1.81125,3.91367,7
1.81662,4.04105,7
1.83035,3.92914,7
1.8429,4.10709,7
1.85864,3.99631,7
1.87161,4.11984,7
1.89139,3.92276,7
1.90054,4.09625,7
1.90784,4.01663,7
1.91867,4.00192,7
1.95972,4.0173,7
2.09741,4.09738,8
2.11838,3.98309,8
2.1329,3.94205,8
2.16171,4.00626,8
2.17206,3.96692,8
2.19183,4.11963,8
2.21292,3.98041,8
2.22607,3.80899,8
2.24121,3.95209,8
2.26105,3.99084,8
2.26776,3.81192,8
2.276,4.19156,8
2.29001,4.07941,8
2.29672,4.03043,8
2.30362,4.04398,8
2.3179,4.05154,8
2.33499,4.24649,8
2.33908,4.0296,8
2.34485,3.68109,8
2.34961,3.82068,8
2.35718,4.10974,8
2.36304,4.25671,8
2.379,3.97583,8
2.3855,4.30252,8
2.39609,3.6846,8
2.40793,3.97443,8
2.41507,4.35187,8
2.42084,3.61884,8
2.42722,3.57529,8
2.43771,4.11853,8
2.45114,3.60303,8
2.46271,3.94608,8
2.46893,3.56729,8
2.47845,3.68735,8
2.48474,3.9585,8
2.48737,4.29907,8
2.49344,3.75671,8
2.49969,3.99072,8
2.50327,4.18588,8
2.50601,4.37421,8
2.51358,3.67108,8
2.52023,4.1702,8
2.52963,4.52316,8
2.53387,3.60571,8
2.54352,4.51395,8
2.55215,3.61932,8
2.55377,3.68671,8
2.55673,3.55002,8
2.56223,3.74466,8
2.56616,3.67813,8
2.56723,3.92554,8
2.57217,4.267,8
2.57788,4.47073,8
2.58136,4.22653,8
2.5918,4.43079,8
2.59595,3.53711,8
2.59891,3.90067,8
2.60031,3.82062,8
2.60529,4.37006,8
2.612,3.45657,8
2.61801,3.95959,8
2.62082,4.15048,8
2.62357,3.89047,8
2.62692,3.62585,8
2.63144,3.40857,8
2.63684,3.98926,8
2.63953,3.97504,8
2.64487,4.13364,8
2.65067,4.01749,8
2.65308,3.52441,8
2.65698,3.95657,8
2.66394,4.6304,8
2.66885,4.56525,8
2.67438,4.59067,8
2.67612,4.32071,8
2.68088,3.93774,8
2.69101,3.33423,8
2.6987,4.04926,8
2.70294,3.69476,8
2.70563,4.2988,8
2.71219,3.51782,8
2.71582,4.39423,8
2.71939,4.14844,8
2.72403,4.35052,8
2.72839,3.90366,8
2.73135,3.5087,8
2.7446,3.95154,8
2.75049,3.8793,8
2.7525,4.63284,8
2.759,3.63525,8
2.76083,4.5148,8
2.76288,4.02859,8
2.76587,4.71033,8
2.76996,3.5397,8
2.77689,4.39267,8
2.78165,3.96075,8
2.78531,3.65836,8
2.79001,4.01862,8
2.79153,4.50363,8
2.79779,4.59573,8
2.80304,3.85434,8
2.80817,4.556,8
2.81201,3.64612,8
2.81558,4.54608,8
2.81635,3.62726,8
2.81848,3.83963,8
2.82175,4.72943,8
2.82327,3.5618,8
2.82776,3.39532,8
2.83005,3.88754,8
2.83191,4.15143,8
2.83929,4.34705,8
2.8414,3.27496,8
2.84421,3.9957,8
2.84866,4.06271,8
2.85114,4.34103,8
2.85281,3.4791,8
2.85641,3.99689,8
2.85977,3.68015,8
2.86151,4.39856,8
2.8663,4.3924,8
2.871,4.29938,8
2.87244,3.93613,8
2.87482,3.56665,8
2.87595,3.22595,8
2.87717,3.6868,8
2.88062,3.2597,8
2.88358,3.6503,8
2.88919,4.70407,8
2.89554,3.78558,8
2.89713,3.31207,8
2.89835,4.14783,8
2.90109,3.91077,8
2.90292,4.36725,8
2.90601,3.2218,8
2.90845,4.79993,8
2.91336,4.15771,8
2.91467,3.75888,8
2.91565,4.68304,8
2.91699,3.55881,8
2.9194,4.5787,8
2.92139,3.41879,8
2.92368,3.13071,8
2.92947,3.14633,8
2.93063,3.733,8
2.93369,4.23691,8
2.93558,3.67419,8
2.9415,4.09409,8
2.94382,4.46558,8
2.95114,3.78061,8
2.95364,3.8829,8
2.95731,4.37335,8
2.95932,3.12213,8
2.96481,3.0437,8
2.96716,4.21649,8
2.96872,3.21298,8
2.97006,4.02942,8
2.97089,3.11304,8
2.97473,3.20126,8
2.97559,3.18091,8
2.97818,4.22116,8
2.98624,3.35165,8
2.98862,3.75458,8
2.98999,3.30679,8
2.99408,3.15784,8
2.99969,3.45535,8
3.00427,4.04962,8
3.00772,3.90897,8
3.00928,4.20596,8
3.0112,4.57724,8
3.01318,3.79794,8
3.01422,4.69714,8
3.02264,3.39563,8
3.02554,3.81573,8
3.02713,4.8241,8
3.02832,4.10855,8
3.03107,4.01874,8
3.03287,3.35379,8
3.03467,4.27802,8
3.03525,4.10315,8
3.03732,4.27847,8
3.03976,3.73373,8
3.04443,4.29449,8
3.04715,3.31424,8
3.05103,4.18149,8
3.05554,4.75037,8
3.05579,4.22861,8
3.06198,4.62131,8
3.06577,3.34259,8
3.06876,4.9278,8
3.07288,3.32709,8
3.07764,4.69101,8
3.07886,4.03055,8
3.08118,4.07449,8
3.08252,4.66422,8
3.08508,4.56693,8
3.08871,3.22098,8
3.09079,3.60477,8
3.09244,3.5863,8
3.09479,4.78815,8
3.09854,3.51303,8
3.10074,4.3324,8
3.10251,3.491,8
3.10349,4.7645,8
3.10779,4.39767,8
3.11111,4.5022,8
3.1124,4.37549,8
3.11349,3.67953,8
3.11853,4.49692,8
3.12488,4.3728,8
3.12952,4.52472,8
3.134,3.37024,8
3.13574,3.81909,8
3.13907,3.24063,8
3.14035,4.53134,8
3.1459,4.11298,8
3.15482,3.35513,8
3.15915,3.39764,8
3.16681,4.1015,8
3.16916,3.30643,8
3.17242,3.67303,8
3.17709,3.9176,8
3.17935,4.68555,8
3.18277,3.91367,8
3.18463,3.46475,8
3.18808,4.72861,8
3.1936,4.48117,8
3.19717,3.93832,8
3.19775,4.78711,8
3.20206,3.55539,8
3.20724,4.26633,8
3.2113,4.16278,8
3.21518,4.02115,8
3.22223,4.12558,8
3.2276,4.12769,8
3.22888,3.26666,8
3.23083,3.81879,8
3.23404,3.50754,8
3.23566,3.24753,8
3.23575,3.44748,8
3.24515,3.43021,8
3.24973,4.38345,8
3.25327,4.41348,8
3.25882,3.63553,8
3.26266,4.50421,8
3.26569,4.56558,8
3.27332,4.42831,8
3.27896,4.3085,8
3.28079,4.31857,8
3.28394,4.2305,8
3.28738,3.90521,8
3.2897,3.57761,8
3.29749,4.49643,8
3.29919,4.31058,8
3.30496,3.33823,8
3.31015,4.22372,8
3.31256,4.17111,8
3.31543,4.10229,8
3.32074,3.66614,8
3.32275,3.4657,8
3.32718,3.47134,8
3.33218,4.60052,8
3.33609,3.3623,8
3.33994,3.91257,8
3.34225,4.37668,8
3.34497,4.06244,8
3.36322,4.41302,8
3.36725,4.51129,8
3.37262,3.49179,8
3.37656,4.00333,8
3.37875,4.35999,8
3.39026,3.44839,8
3.39435,4.06113,8
3.40189,3.44955,8
3.40744,3.80838,8
3.40863,3.43887,8
3.41354,3.52487,8
3.41638,3.56039,8
3.41971,4.14337,8
3.42752,4.30399,8
3.43103,3.55637,8
3.43878,3.86951,8
3.4433,4.20026,8
3.44962,4.47537,8
3.45084,3.89151,8
3.45258,4.2959,8
3.46793,3.802,8
3.478,3.78738,8
3.48245,3.65857,8
3.49283,3.81644,8
3.50565,4.48477,8
3.51151,3.61462,8
3.51572,3.62796,8
3.52142,4.07532,8
3.5311,3.56839,8
3.53506,4.01755,8
3.54333,4.29926,8
3.55115,3.8685,8
3.55231,4.18002,8
3.55676,3.7226,8
3.56067,3.99863,8
3.56461,4.27927,8
3.56848,3.93085,8
3.57132,4.14444,8
3.57883,4.29709,8
3.58533,3.95834,8
3.59082,4.04895,8
3.59967,4.28241,8
3.60703,3.68753,8
3.61438,4.01257,8
3.61566,4.03085,8
3.61984,3.73877,8
3.62753,4.14545,8
3.6312,4.20361,8
3.6351,4.18176,8
3.6391,3.89478,8
3.65625,3.81989,8
3.68097,4.22879,8
3.71085,3.82047,8
3.72015,3.96783,8
3.7301,4.08432,8
3.73611,3.74323,8
3.75415,4.17026,8
3.76596,4.18695,8
3.77542,4.00009,8
3.78473,4.1861,8
3.79544,4.01443,8
3.80444,3.89304,8
3.80911,3.88107,8
3.83362,4.14063,8
3.84756,3.91028,8
3.89923,3.94684,8
3.93759,4.01599,8'''
s=s.split('\n')
a=[]
b=[]
c=[]
Al=[]
for i in s:
    i=i.split(',')
    a.append(float(i[0]))
    b.append(float(i[1]))
    c.append(int(i[2]))
    Al.append([a[-1],b[-1]])


n=500

pi=prob(Al,n)
a=cube(Al,pi)
X=[]
Y=[]
Z=[]
for i in range(0,len(Al)):
    xy=[]
    if i+1 in a:
        X.append(Al[i])
Y=c
numberofcluster=len(set(Y))
K=numberofcluster
print(numberofcluster)
a11=time.time()

kk=KMeans(n_clusters=K)

kk.fit(np.array(X))
#print(time.time()-a11)
c1=list(kk.predict(np.array(Al)))
print(time.time()-a11,end=',')
print(accuracy(c1,Y))
import colorsys

def _get_colors(num_colors):
    colors=[]
    for i in np.arange(0., 360., 360. / num_colors):
        hue = i/360.
        lightness = (50 + np.random.rand() * 10)/100.
        saturation = (90 + np.random.rand() * 10)/100.
        colors.append(colorsys.hls_to_rgb(hue, lightness, saturation))
    return colors
col= _get_colors(len(set(Y)))


o=0
for i in Al:
    plt.scatter(i[0],i[1],s=1,c='k')
    o+=1
plt.savefig('ARTIFn1.pdf')
import os
os.system('ARTIFn1.pdf')

# =============================================================================
# a11=time.time()
# 
# kk=KMeans(n_clusters=K)
# 
# kk.fit(np.array(Al))
# #print(time.time()-a11)
# c1=list(kk.predict(np.array(Al)))
# print(time.time()-a11,end=',')
# print(accuracy(c1,Y))
# =============================================================================
